### 백엔드 코드

#### 1. Tmap API

Geocoding - Full Text Geocoding + 길찾기

출발 위치와 도착 위치 입력받아 API 호출 - 좌표 프론트엔드로 반환 / 대중교통 길찾기 - 프론트엔드로 경로 반환

In [ ]:
# Geocoding 시도 후 실패하면 Full Text Geocoding -> 출발지 도착지 좌표 프론트엔드로 반환
# api key는 나중에 입력할 예정 

from flask import Flask, request, jsonify
import requests

app = Flask(__name__)

TMAP_API_KEY = 'YOUR_TMAP_API_KEY'
GEO_URL = "https://apis.openapi.sk.com/tmap/geo/geocoding"
FULLTEXT_GEO_URL = "https://apis.openapi.sk.com/tmap/geo/fullAddrGeo"


@app.route('/get_coordinates', methods=['POST'])
def get_coordinates():
    # 프론트엔드에서 받은 주소 정보
    address = request.form.get('address')

    # 1. 일반 지오코딩 시도
    geocode_response = geocoding(address)
    
    if geocode_response and geocode_response.get('coordinateInfo') and geocode_response['coordinateInfo'].get('coordinate'):
        # 성공적으로 좌표 데이터를 받았을 때
        coordinates = geocode_response['coordinateInfo']['coordinate'][0]
        lat = coordinates.get('lat', coordinates.get('newLat', None))
        lon = coordinates.get('lon', coordinates.get('newLon', None))
        
        if lat and lon:
            return jsonify({'lat': lat, 'lon': lon})

    # 2. 지오코딩 실패 시 풀 텍스트 지오코딩 시도
    fulltext_response = fulltext_geocoding(address)
    
    if fulltext_response and fulltext_response.get('coordinateInfo') and fulltext_response['coordinateInfo'].get('coordinate'):
        # 풀 텍스트 지오코딩으로 좌표 데이터 받았을 때
        coordinates = fulltext_response['coordinateInfo']['coordinate'][0]
        lat = coordinates.get('lat', coordinates.get('newLat', None))
        lon = coordinates.get('lon', coordinates.get('newLon', None))
        
        if lat and lon:
            return jsonify({'lat': lat, 'lon': lon})

    # 두 요청 모두 실패했을 경우
    return jsonify({'error': 'Unable to find coordinates for the given address'}), 404


def geocoding(address):
    """
    일반 Geocoding API 호출 함수
    """
    headers = {
        'appKey': TMAP_API_KEY
    }
    params = {
        'version': 1,
        'format': 'json',
        'coordType': 'WGS84GEO',
        'addressType': 'A00',  # 지번 주소를 기본으로 시도
        'fullAddr': address
    }
    
    try:
        response = requests.get(GEO_URL, headers=headers, params=params)
        return response.json()
    except requests.RequestException as e:
        print(f"Geocoding API 요청 실패: {e}")
        return None


def fulltext_geocoding(address):
    """
    Full Text Geocoding API 호출 함수
    """
    headers = {
        'appKey': TMAP_API_KEY
    }
    params = {
        'version': 1,
        'format': 'json',
        'coordType': 'WGS84GEO',
        'fullAddr': address  # 전체 텍스트로 지오코딩 시도
    }
    
    try:
        response = requests.get(FULLTEXT_GEO_URL, headers=headers, params=params)
        return response.json()
    except requests.RequestException as e:
        print(f"Full Text Geocoding API 요청 실패: {e}")
        return None


if __name__ == '__main__':
    app.run(debug=True)

ReverseGeocoding 

In [ ]:
'''
from flask import Flask, request, jsonify
import requests

app = Flask(__name__)

@app.route('/reverse_geocode', methods=['POST'])
def reverse_geocode():
    # 프론트엔드에서 받은 좌표 (출발지 좌표, 도착지 좌표)
    start_lon = request.form.get('start_lon')
    start_lat = request.form.get('start_lat')
    
    end_lon = request.form.get('end_lon')
    end_lat = request.form.get('end_lat')

    # T맵 Reverse Geocoding API 호출을 위한 URL 및 헤더 설정
    url = "https://apis.openapi.sk.com/tmap/geo/reversegeocoding?version=1&format=json"
    headers = {
        "appKey": "YOUR_TMAP_API_KEY"  # 백엔드에서 API Key를 관리
    }

    # 1. 출발지 Reverse Geocoding API 호출
    start_params = {
        "coordType": "WGS84GEO",
        "addressType": "A10",
        "lon": start_lon,
        "lat": start_lat
    }

    start_response = requests.get(url, params=start_params, headers=headers)

    if start_response.status_code == 200:
        start_address_info = start_response.json().get('addressInfo', {})
        start_address = format_address(start_address_info)
    else:
        return jsonify({'error': 'Reverse Geocoding for start location failed'}), 400

    # 2. 도착지 Reverse Geocoding API 호출
    end_params = {
        "coordType": "WGS84GEO",
        "addressType": "A10",
        "lon": end_lon,
        "lat": end_lat
    }

    end_response = requests.get(url, params=end_params, headers=headers)

    if end_response.status_code == 200:
        end_address_info = end_response.json().get('addressInfo', {})
        end_address = format_address(end_address_info)
    else:
        return jsonify({'error': 'Reverse Geocoding for end location failed'}), 400

    # 최종적으로 출발지와 도착지의 주소를 JSON으로 반환
    return jsonify({
        'start_address': start_address,
        'end_address': end_address
    })

# 주소 형식을 맞추는 함수 (새주소와 지번주소를 반환하는 형태로 포맷팅)
def format_address(address_info):
    # 새주소 만들기
    lastLegal = address_info.get('legalDong', '')[-1]
    newRoadAddr = f"{address_info.get('city_do', '')} {address_info.get('gu_gun', '')} "

    if address_info.get('eup_myun', '') == '' and (lastLegal == "읍" or lastLegal == "면"):
        newRoadAddr += address_info.get('legalDong', '')
    else:
        newRoadAddr += address_info.get('eup_myun', '')

    newRoadAddr += f" {address_info.get('roadName', '')} {address_info.get('buildingIndex', '')}"

    if address_info.get('legalDong', '') != '' and lastLegal not in ["읍", "면"]:
        if address_info.get('buildingName', '') != '':
            newRoadAddr += f" ({address_info.get('legalDong', '')}, {address_info.get('buildingName', '')})"
        else:
            newRoadAddr += f" ({address_info.get('legalDong', '')})"

    # 지번 주소 만들기
    jibunAddr = f"{address_info.get('city_do', '')} {address_info.get('gu_gun', '')} {address_info.get('legalDong', '')} {address_info.get('ri', '')} {address_info.get('bunji', '')}"
    if address_info.get('buildingName', '') != '':
        jibunAddr += f" {address_info.get('buildingName', '')}"

    return {
        'new_address': newRoadAddr,
        'jibun_address': jibunAddr
    }

if __name__ == '__main__':
    app.run(debug=True)
'''

#### 2. Tmap 대중교통 API

##### 2.1 경로탐색

In [ ]:
'''
<Body Parameters>

"startX":       출발지 좌표 (경도)
"startY":       출발지 좌표 (위도)
"endX":         도착지 좌표 (경도)
"endY":         도착지 좌표 (위도)
"lang":         응답 언어 선택- 국문 : 0(기본값), 영문 : 1
"format":       출력포맷- json(기본값), xml
"count":        최대 응답 결과 개수- 1~10, 기본값 10
"searchDttm":   타임머신 기능 검색 날짜- yyyymmddhhmi
                타임머신 요청 시간 범위: 
                년 (4자리) = 1900 ~ 9999
                월 (2자리) = 01 ~ 12
                일 (2자리) = 01 ~ 31
                시 (2자리) = 00 ~ 23
                분 (2자리) = 00 ~ 59
''' 

In [1]:
import requests

url = "https://apis.openapi.sk.com/transit/routes"

payload = {
    "startX": "126.926493082645", 
    "startY": "37.6134436427887", 
    "endX": "127.126936754911", 
    "endY": "37.5004198786564",
    "lang": 0,
    "format": "json",
    "count": 10,
    "searchDttm": "202301011200"
}
headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "appKey": "YEWVxfrK4j8xTNQZURJ4z1Te4JTZs26v45fgmfn7"
}

response = requests.post(url, json=payload, headers=headers)

print(response.text)

{"metaData":{"requestParameters":{"busCount":3,"expressbusCount":0,"subwayCount":2,"airplaneCount":0,"locale":"ko","endY":"37.5004198786564","endX":"127.126936754911","wideareaRouteCount":0,"subwayBusCount":5,"startY":"37.6134436427887","startX":"126.926493082645","ferryCount":0,"trainCount":0,"reqDttm":"20241023022117"},"plan":{"itineraries":[{"fare":{"regular":{"totalFare":1800,"currency":{"symbol":"￦","currency":"원","currencyCode":"KRW"}}},"totalTime":4002,"legs":[{"mode":"WALK","sectionTime":33,"distance":46,"start":{"name":"출발지","lon":126.926493082645,"lat":37.6134436427887},"end":{"name":"대조동우체국","lon":126.926525,"lat":37.61311111111111},"steps":[{"streetName":"보행자도로","distance":19,"description":"보행자도로 을 따라 19m 이동","linestring":"126.92651,37.61345 126.92652,37.61344 126.92654,37.613388 126.926414,37.613342"},{"streetName":"","distance":27,"description":", 27m","linestring":"126.926414,37.613342 126.926476,37.613197 126.92652,37.613117 126.92653,37.613113"}]},{"routeColor":"53B332

대중교통 api 호출 코드

In [ ]:
import requests
from datetime import datetime

url = "https://apis.openapi.sk.com/transit/routes"
API_KEY = 'Du88s82V2690hjVCJpUFf41sc3Xn94KL5rYJSE38'

# 프론트에서 값 받아와서 변수에 넣기 (경도, 위도 순서)
start_lon, start_lat, end_lon, end_lat = 0

payload = {
    "startX": start_lon, 
    "startY": start_lat, 
    "endX": end_lon, 
    "endY": end_lat,
    "lang": 0,
    "format": "json",
    "count": 10,
    "searchDttm": datetime.now().strftime("%Y%m%d%H%M")
}
headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "appKey": "YEWVxfrK4j8xTNQZURJ4z1Te4JTZs26v45fgmfn7"
}

response = requests.post(url, json=payload, headers=headers)



대중교통 api 호출 + 프론트엔드로 반환 

In [ ]:
from flask import Flask, request, jsonify
import requests
from datetime import datetime

app = Flask(__name__)

API_KEY = 'YEWVxfrK4j8xTNQZURJ4z1Te4JTZs26v45fgmfn7'
TMAP_URL = "https://apis.openapi.sk.com/transit/routes"

@app.route('/find_route', methods=['POST'])
def find_route():
    # 프론트엔드에서 경도, 위도 좌표를 받아옴
    data = request.get_json()
    start_lon = data.get('start_lon')
    start_lat = data.get('start_lat')
    end_lon = data.get('end_lon')
    end_lat = data.get('end_lat')

    # Tmap API에 요청할 페이로드
    payload = {
        "startX": start_lon,
        "startY": start_lat,
        "endX": end_lon,
        "endY": end_lat,
        "lang": 0,
        "format": "json",
        "count": 10,
        "searchDttm": datetime.now().strftime("%Y%m%d%H%M")
    }

    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "appKey": API_KEY
    }

    # Tmap API 요청
    try:
        response = requests.post(TMAP_URL, json=payload, headers=headers)
        route_data = response.json()
        return jsonify(route_data), 200
    except requests.RequestException as e:
        return jsonify({"error": "Unable to find route"}), 500

if __name__ == '__main__':
    app.run(debug=True)


In [3]:
import json

# JSON 데이터를 문자열로 할당 (트리플 따옴표 사용)
json_data = '''{
  "metaData": {
    "requestParameters": {
      "busCount": 5,
      "expressbusCount": 0,
      "subwayCount": 1,
      "airplaneCount": 0,
      "locale": "ko",
      "endY": "37.575326",
      "endX": "127.0248495",
      "wideareaRouteCount": 0,
      "subwayBusCount": 0,
      "startY": "37.573246",
      "startX": "127.016589",
      "ferryCount": 0,
      "trainCount": 0,
      "reqDttm": "20241024213754"
    },
    "plan": {
      "itineraries": [
        {
          "fare": {
            "regular": {
              "totalFare": 1400,
              "currency": {
                "symbol": "￦",
                "currency": "원",
                "currencyCode": "KRW"
              }
            }
          },
          "totalTime": 408,
          "legs": [
            {
              "mode": "WALK",
              "sectionTime": 133,
              "distance": 148,
              "start": {
                "name": "출발지",
                "lon": 127.016589,
                "lat": 37.573246
              },
              "end": {
                "name": "동묘앞",
                "lon": 127.01659444444445,
                "lat": 37.57326666666667
              },
              "steps": [
                {
                  "streetName": "",
                  "distance": 68,
                  "description": "68m 이동",
                  "linestring": "127.01658,37.57326 127.01721,37.57341 127.01731,37.573437"
                },
                {
                  "streetName": "",
                  "distance": 29,
                  "description": "동묘앞역  3번출구 에서 우회전 후 29m 이동 ",
                  "linestring": "127.01731,37.573437 127.01736,37.573322 127.0172,37.57328 127.01719,37.573273"
                },
                {
                  "streetName": "왕산로",
                  "distance": 51,
                  "description": "동묘앞역  3번출구 에서 우회전 후 왕산로 을 따라 51m 이동 ",
                  "linestring": "127.01719,37.573273 127.01706,37.57324 127.01686,37.573193 127.01664,37.57314"
                }
              ]
            },
            {
              "routeColor": "0052A4",
              "distance": 725,
              "start": {
                "name": "동묘앞",
                "lon": 127.01659444444445,
                "lat": 37.57326666666667
              },
              "Lane": [
                {
                  "routeColor": "0052A4",
                  "route": "수도권1호선(급행)",
                  "routeId": "111013004",
                  "service": 1,
                  "type": 117
                }
              ],
              "type": 1,
              "mode": "SUBWAY",
              "sectionTime": 120,
              "route": "수도권1호선",
              "routeId": "110011030",
              "service": 0,
              "passStopList": {
                "stationList": [
                  {
                    "index": 0,
                    "stationName": "동묘앞",
                    "lon": "127.016594",
                    "lat": "37.573267",
                    "stationID": "110128"
                  },
                  {
                    "index": 1,
                    "stationName": "신설동",
                    "lon": "127.024111",
                    "lat": "37.575894",
                    "stationID": "110127"
                  }
                ]
              },
              "end": {
                "name": "신설동",
                "lon": 127.02411111111111,
                "lat": 37.575894444444444
              },
              "passShape": {
                "linestring": "127.016603,37.573250 127.017108,37.573392 127.017500,37.573503 127.017731,37.573572 127.017964,37.573644 127.018194,37.573719 127.018425,37.573794 127.020608,37.574539 127.021017,37.574692 127.023233,37.575572 127.023411,37.575644 127.023575,37.575708 127.024111,37.575894"
              }
            },
            {
              "mode": "WALK",
              "sectionTime": 155,
              "distance": 185,
              "start": {
                "name": "신설동",
                "lon": 127.02411111111111,
                "lat": 37.575894444444444
              },
              "end": {
                "name": "도착지",
                "lon": 127.0248495,
                "lat": 37.575326
              },
              "steps": [
                {
                  "streetName": "",
                  "distance": 39,
                  "description": "39m 이동",
                  "linestring": "127.024086,37.575935 127.0238,37.57585 127.02368,37.575794"
                },
                {
                  "streetName": "",
                  "distance": 36,
                  "description": "좌회전 후 36m 이동 ",
                  "linestring": "127.02368,37.575794 127.02378,37.57566 127.02378,37.575623 127.02377,37.57549"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 3,
                  "description": "신설동역  6번출구 에서 우회전 후 보행자도로 을 따라 3m 이동 ",
                  "linestring": "127.02377,37.57549 127.023735,37.575485"
                },
                {
                  "streetName": "",
                  "distance": 76,
                  "description": "신설동역  6번출구 에서 좌회전 후 76m 이동 ",
                  "linestring": "127.023735,37.575485 127.023766,37.575462 127.023834,37.575428 127.02385,37.575428 127.02418,37.575283 127.02447,37.575172 127.02449,37.575165"
                },
                {
                  "streetName": "천호대로",
                  "distance": 31,
                  "description": "동진회관 에서 좌회전 후 천호대로 을 따라 31m 이동 ",
                  "linestring": "127.02449,37.575165 127.024666,37.57541"
                }
              ]
            }
          ],
          "totalWalkTime": 288,
          "transferCount": 0,
          "totalDistance": 819,
          "pathType": 1,
          "totalWalkDistance": 333
        },
        {
          "fare": {
            "regular": {
              "totalFare": 1500,
              "currency": {
                "symbol": "￦",
                "currency": "원",
                "currencyCode": "KRW"
              }
            }
          },
          "totalTime": 456,
          "legs": [
            {
              "mode": "WALK",
              "sectionTime": 202,
              "distance": 245,
              "start": {
                "name": "출발지",
                "lon": 127.016589,
                "lat": 37.573246
              },
              "end": {
                "name": "동묘앞",
                "lon": 127.01873611111111,
                "lat": 37.57380277777778
              },
              "steps": [
                {
                  "streetName": "",
                  "distance": 68,
                  "description": "68m 이동",
                  "linestring": "127.01658,37.57326 127.01721,37.57341 127.01731,37.573437"
                },
                {
                  "streetName": "",
                  "distance": 29,
                  "description": "동묘앞역  3번출구 에서 우회전 후 29m 이동 ",
                  "linestring": "127.01731,37.573437 127.01736,37.573322 127.0172,37.57328 127.01719,37.573273"
                },
                {
                  "streetName": "왕산로",
                  "distance": 148,
                  "description": "동묘앞역  3번출구 에서 좌회전 후 왕산로 을 따라 148m 이동 ",
                  "linestring": "127.01719,37.573273 127.017296,37.573296 127.01739,37.57332 127.01758,37.573368 127.0177,37.573406 127.01834,37.57362 127.01875,37.57376"
                }
              ]
            },
            {
              "routeColor": "0068B7",
              "distance": 632,
              "start": {
                "name": "동묘앞",
                "lon": 127.01873611111111,
                "lat": 37.57380277777778
              },
              "Lane": [
                {
                  "routeColor": "0068B7",
                  "route": "간선:420",
                  "routeId": "11455001",
                  "service": 1,
                  "type": 11
                },
                {
                  "routeColor": "0068B7",
                  "route": "간선:261",
                  "routeId": "11444001",
                  "service": 1,
                  "type": 11
                },
                {
                  "routeColor": "0068B7",
                  "route": "간선:720",
                  "routeId": "2870001",
                  "service": 1,
                  "type": 11
                },
                {
                  "routeColor": "0068B7",
                  "route": "간선:260",
                  "routeId": "11443001",
                  "service": 1,
                  "type": 11
                }
              ],
              "type": 11,
              "mode": "BUS",
              "sectionTime": 157,
              "route": "간선:271",
              "routeId": "11447001",
              "service": 1,
              "passStopList": {
                "stationList": [
                  {
                    "index": 0,
                    "stationName": "동묘앞",
                    "lon": "127.018736",
                    "lat": "37.573803",
                    "stationID": "775214"
                  },
                  {
                    "index": 1,
                    "stationName": "신설동역",
                    "lon": "127.025250",
                    "lat": "37.576150",
                    "stationID": "775282"
                  }
                ]
              },
              "end": {
                "name": "신설동역",
                "lon": 127.02525,
                "lat": 37.57615
              },
              "passShape": {
                "linestring": "127.018711,37.573833 127.020483,37.574436 127.022508,37.575214 127.023142,37.575464 127.023414,37.575578 127.023914,37.575775 127.025200,37.576175 127.025228,37.576183"
              }
            },
            {
              "mode": "WALK",
              "sectionTime": 97,
              "distance": 132,
              "start": {
                "name": "신설동역",
                "lon": 127.02525,
                "lat": 37.57615
              },
              "end": {
                "name": "도착지",
                "lon": 127.0248495,
                "lat": 37.575326
              },
              "steps": [
                {
                  "streetName": "왕산로",
                  "distance": 66,
                  "description": "왕산로 을 따라 66m 이동",
                  "linestring": "127.02526,37.576115 127.02499,37.57603 127.02491,37.57601 127.02481,37.57598 127.02466,37.575935 127.02457,37.575905"
                },
                {
                  "streetName": "",
                  "distance": 39,
                  "description": "해피누리 에서 좌회전 후 39m 이동 ",
                  "linestring": "127.02457,37.575905 127.024826,37.575623"
                },
                {
                  "streetName": "천호대로",
                  "distance": 27,
                  "description": "씨씨월드 신설동점 에서 우회전 후 천호대로 을 따라 27m 이동 ",
                  "linestring": "127.024826,37.575623 127.024666,37.57541"
                }
              ]
            }
          ],
          "totalWalkTime": 299,
          "transferCount": 0,
          "totalDistance": 931,
          "pathType": 2,
          "totalWalkDistance": 377
        },
        {
          "fare": {
            "regular": {
              "totalFare": 1500,
              "currency": {
                "symbol": "￦",
                "currency": "원",
                "currencyCode": "KRW"
              }
            }
          },
          "totalTime": 638,
          "legs": [
            {
              "mode": "WALK",
              "sectionTime": 202,
              "distance": 245,
              "start": {
                "name": "출발지",
                "lon": 127.016589,
                "lat": 37.573246
              },
              "end": {
                "name": "동묘앞",
                "lon": 127.01873611111111,
                "lat": 37.57380277777778
              },
              "steps": [
                {
                  "streetName": "",
                  "distance": 68,
                  "description": "68m 이동",
                  "linestring": "127.01658,37.57326 127.01721,37.57341 127.01731,37.573437"
                },
                {
                  "streetName": "",
                  "distance": 29,
                  "description": "동묘앞역  3번출구 에서 우회전 후 29m 이동 ",
                  "linestring": "127.01731,37.573437 127.01736,37.573322 127.0172,37.57328 127.01719,37.573273"
                },
                {
                  "streetName": "왕산로",
                  "distance": 148,
                  "description": "동묘앞역  3번출구 에서 좌회전 후 왕산로 을 따라 148m 이동 ",
                  "linestring": "127.01719,37.573273 127.017296,37.573296 127.01739,37.57332 127.01758,37.573368 127.0177,37.573406 127.01834,37.57362 127.01875,37.57376"
                }
              ]
            },
            {
              "routeColor": "0068B7",
              "distance": 514,
              "start": {
                "name": "동묘앞",
                "lon": 127.01873611111111,
                "lat": 37.57380277777778
              },
              "Lane": [
                {
                  "routeColor": "0068B7",
                  "route": "간선:101",
                  "routeId": "11422001",
                  "service": 1,
                  "type": 11
                },
                {
                  "routeColor": "0068B7",
                  "route": "간선:103",
                  "routeId": "11424001",
                  "service": 1,
                  "type": 11
                },
                {
                  "routeColor": "0068B7",
                  "route": "간선:144",
                  "routeId": "11434001",
                  "service": 1,
                  "type": 11
                },
                {
                  "routeColor": "0068B7",
                  "route": "간선:N15",
                  "routeId": "11467001",
                  "service": 0,
                  "type": 11
                }
              ],
              "type": 11,
              "mode": "BUS",
              "sectionTime": 157,
              "route": "간선:152",
              "routeId": "2241001",
              "service": 1,
              "passStopList": {
                "stationList": [
                  {
                    "index": 0,
                    "stationName": "동묘앞",
                    "lon": "127.018736",
                    "lat": "37.573803",
                    "stationID": "775214"
                  },
                  {
                    "index": 1,
                    "stationName": "신설동역1번출구",
                    "lon": "127.023336",
                    "lat": "37.576639",
                    "stationID": "775300"
                  }
                ]
              },
              "end": {
                "name": "신설동역1번출구",
                "lon": 127.0233361111111,
                "lat": 37.57663888888889
              },
              "passShape": {
                "linestring": "127.018711,37.573833 127.020483,37.574436 127.022508,37.575214 127.023142,37.575464 127.023272,37.575594 127.023314,37.575700 127.023292,37.576628"
              }
            },
            {
              "mode": "WALK",
              "sectionTime": 279,
              "distance": 264,
              "start": {
                "name": "신설동역1번출구",
                "lon": 127.0233361111111,
                "lat": 37.57663888888889
              },
              "end": {
                "name": "도착지",
                "lon": 127.0248495,
                "lat": 37.575326
              },
              "steps": [
                {
                  "streetName": "보문로",
                  "distance": 68,
                  "description": "보문로 을 따라 68m 이동",
                  "linestring": "127.02337,37.576637 127.02337,37.5765 127.02337,37.576435 127.02337,37.576225 127.02338,37.576145 127.023384,37.576023"
                },
                {
                  "streetName": "",
                  "distance": 7,
                  "description": "우측 횡단보도 후 7m 이동 ",
                  "linestring": "127.023445,37.575916 127.0234,37.575863"
                },
                {
                  "streetName": "",
                  "distance": 6,
                  "description": "직진 후 6m 이동 ",
                  "linestring": "127.0234,37.575863 127.02339,37.575848 127.02342,37.575836"
                },
                {
                  "streetName": "",
                  "distance": 14,
                  "description": "횡단보도 후 14m 이동 ",
                  "linestring": "127.02342,37.575836 127.02349,37.575714"
                },
                {
                  "streetName": "",
                  "distance": 13,
                  "description": "좌회전 후 13m 이동 ",
                  "linestring": "127.02349,37.575714 127.02362,37.575775"
                },
                {
                  "streetName": "",
                  "distance": 22,
                  "description": "우측 횡단보도 후 22m 이동 ",
                  "linestring": "127.02362,37.575775 127.02373,37.5756"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 13,
                  "description": "신설동역  6번출구 에서 우회전 후 보행자도로 을 따라 13m 이동 ",
                  "linestring": "127.02373,37.5756 127.02371,37.575565 127.02371,37.575523 127.02373,37.57549 127.023735,37.575485"
                },
                {
                  "streetName": "천호대로",
                  "distance": 31,
                  "description": "동진회관 에서 좌회전 후 천호대로 을 따라 31m 이동 ",
                  "linestring": "127.02449,37.575165 127.024666,37.57541"
                }
              ]
            }
          ],
          "totalWalkTime": 481,
          "transferCount": 0,
          "totalDistance": 912,
          "pathType": 2,
          "totalWalkDistance": 509
        },
        {
          "fare": {
            "regular": {
              "totalFare": 1500,
              "currency": {
                "symbol": "￦",
                "currency": "원",
                "currencyCode": "KRW"
              }
            }
          },
          "totalTime": 693,
          "legs": [
            {
              "mode": "WALK",
              "sectionTime": 263,
              "distance": 312,
              "start": {
                "name": "출발지",
                "lon": 127.016589,
                "lat": 37.573246
              },
              "end": {
                "name": "동대문역.흥인지문",
                "lon": 127.01341666666667,
                "lat": 37.57225
              },
              "steps": [
                {
                  "streetName": "",
                  "distance": 87,
                  "description": "87m 이동",
                  "linestring": "127.01658,37.57326 127.01637,37.57321 127.016106,37.573135 127.01564,37.573025"
                },
                {
                  "streetName": "",
                  "distance": 15,
                  "description": "동묘앞역  4번출구 에서 좌회전 후 15m 이동 ",
                  "linestring": "127.01564,37.573025 127.01564,37.57289"
                },
                {
                  "streetName": "",
                  "distance": 42,
                  "description": "팔레스오피스텔 에서 우회전 후 42m 이동 ",
                  "linestring": "127.01564,37.57289 127.01522,37.572773 127.015205,37.572754"
                },
                {
                  "streetName": "왕산로",
                  "distance": 168,
                  "description": "동묘앞역  7번출구 에서 2시 방향 우회전 후 왕산로 을 따라 168m 이동 ",
                  "linestring": "127.015205,37.572754 127.015144,37.572742 127.01407,37.57241 127.01384,37.572334 127.01348,37.572227 127.013435,37.572212"
                }
              ]
            },
            {
              "routeColor": "0068B7",
              "distance": 1043,
              "start": {
                "name": "동대문역.흥인지문",
                "lon": 127.01341666666667,
                "lat": 37.57225
              },
              "Lane": [
                {
                  "routeColor": "0068B7",
                  "route": "간선:721",
                  "routeId": "11463001",
                  "service": 1,
                  "type": 11
                },
                {
                  "routeColor": "53B332",
                  "route": "지선:2112",
                  "routeId": "11476001",
                  "service": 1,
                  "type": 12
                },
                {
                  "routeColor": "FF3300",
                  "route": "직행좌석:9301",
                  "routeId": "3345001",
                  "service": 1,
                  "type": 4
                },
                {
                  "routeColor": "0068B7",
                  "route": "간선:N30",
                  "routeId": "11470001",
                  "service": 0,
                  "type": 11
                }
              ],
              "type": 11,
              "mode": "BUS",
              "sectionTime": 286,
              "route": "간선:370",
              "routeId": "3302001",
              "service": 1,
              "passStopList": {
                "stationList": [
                  {
                    "index": 0,
                    "stationName": "동대문역.흥인지문",
                    "lon": "127.013417",
                    "lat": "37.572250",
                    "stationID": "775153"
                  },
                  {
                    "index": 1,
                    "stationName": "숭인동",
                    "lon": "127.020397",
                    "lat": "37.574369",
                    "stationID": "775230"
                  },
                  {
                    "index": 2,
                    "stationName": "동대문우체국.서울풍물시장",
                    "lon": "127.024692",
                    "lat": "37.574786",
                    "stationID": "775237"
                  }
                ]
              },
              "end": {
                "name": "동대문우체국.서울풍물시장",
                "lon": 127.02469166666667,
                "lat": 37.57478611111111
              },
              "passShape": {
                "linestring": "127.013406,37.572283 127.015558,37.572922 127.015733,37.572972 127.017636,37.573467 127.020367,37.574397 127.020483,37.574436 127.022508,37.575214 127.023125,37.575333 127.023303,37.575350 127.023425,37.575339 127.023817,37.575231 127.024367,37.574983 127.024719,37.574817"
              }
            },
            {
              "mode": "WALK",
              "sectionTime": 144,
              "distance": 160,
              "start": {
                "name": "동대문우체국.서울풍물시장",
                "lon": 127.02469166666667,
                "lat": 37.57478611111111
              },
              "end": {
                "name": "도착지",
                "lon": 127.0248495,
                "lat": 37.575326
              },
              "steps": [
                {
                  "streetName": "천호대로",
                  "distance": 69,
                  "description": "천호대로 을 따라 69m 이동",
                  "linestring": "127.02466,37.57475 127.02459,37.574783 127.02434,37.574905 127.024315,37.57491 127.02428,37.574917 127.02412,37.57499 127.023994,37.575058"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 30,
                  "description": "신설동역 10번출구 에서 우측 횡단보도 후 보행자도로 을 따라 30m 이동 ",
                  "linestring": "127.023994,37.575058 127.02418,37.575283"
                },
                {
                  "streetName": "",
                  "distance": 30,
                  "description": "동진회관 에서 우회전 후 30m 이동 ",
                  "linestring": "127.02418,37.575283 127.02447,37.575172 127.02449,37.575165"
                },
                {
                  "streetName": "천호대로",
                  "distance": 31,
                  "description": "동진회관 에서 좌회전 후 천호대로 을 따라 31m 이동 ",
                  "linestring": "127.02449,37.575165 127.024666,37.57541"
                }
              ]
            }
          ],
          "totalWalkTime": 407,
          "transferCount": 0,
          "totalDistance": 1407,
          "pathType": 2,
          "totalWalkDistance": 472
        },
        {
          "fare": {
            "regular": {
              "totalFare": 1400,
              "currency": {
                "symbol": "￦",
                "currency": "원",
                "currencyCode": "KRW"
              }
            }
          },
          "totalTime": 684,
          "legs": [
            {
              "mode": "WALK",
              "sectionTime": 167,
              "distance": 196,
              "start": {
                "name": "출발지",
                "lon": 127.016589,
                "lat": 37.573246
              },
              "end": {
                "name": "동묘.숭인동도깨비시장",
                "lon": 127.01822222222222,
                "lat": 37.57363055555555
              },
              "steps": [
                {
                  "streetName": "",
                  "distance": 68,
                  "description": "68m 이동",
                  "linestring": "127.01658,37.57326 127.01721,37.57341 127.01731,37.573437"
                },
                {
                  "streetName": "",
                  "distance": 29,
                  "description": "동묘앞역  3번출구 에서 우회전 후 29m 이동 ",
                  "linestring": "127.01731,37.573437 127.01736,37.573322 127.0172,37.57328 127.01719,37.573273"
                },
                {
                  "streetName": "왕산로",
                  "distance": 99,
                  "description": "동묘앞역  3번출구 에서 좌회전 후 왕산로 을 따라 99m 이동 ",
                  "linestring": "127.01719,37.573273 127.017296,37.573296 127.01739,37.57332 127.01758,37.573368 127.0177,37.573406 127.01824,37.573586"
                }
              ]
            },
            {
              "mode": "BUS",
              "routeColor": "F2B70A",
              "sectionTime": 173,
              "route": "순환:TOUR11",
              "routeId": "1025409001",
              "distance": 444,
              "service": 1,
              "start": {
                "name": "동묘.숭인동도깨비시장",
                "lon": 127.01822222222222,
                "lat": 37.57363055555555
              },
              "passStopList": {
                "stationList": [
                  {
                    "index": 0,
                    "stationName": "동묘.숭인동도깨비시장",
                    "lon": "127.018222",
                    "lat": "37.573631",
                    "stationID": "775205"
                  },
                  {
                    "index": 1,
                    "stationName": "서울풍물시장",
                    "lon": "127.023200",
                    "lat": "37.573097",
                    "stationID": "775188"
                  }
                ]
              },
              "end": {
                "name": "서울풍물시장",
                "lon": 127.0232,
                "lat": 37.57309722222222
              },
              "type": 13,
              "passShape": {
                "linestring": "127.018219,37.573667 127.020483,37.574436 127.022508,37.575214 127.023142,37.575464 127.023239,37.573117"
              }
            },
            {
              "mode": "WALK",
              "sectionTime": 344,
              "distance": 400,
              "start": {
                "name": "서울풍물시장",
                "lon": 127.0232,
                "lat": 37.57309722222222
              },
              "end": {
                "name": "도착지",
                "lon": 127.0248495,
                "lat": 37.575326
              },
              "steps": [
                {
                  "streetName": "난계로",
                  "distance": 166,
                  "description": "난계로 을 따라 166m 이동",
                  "linestring": "127.02318,37.573097 127.02313,37.57382 127.0231,37.57441 127.02312,37.574497 127.02312,37.574593"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 26,
                  "description": "강북비타에듀학원 에서 우측 횡단보도 후 보행자도로 을 따라 26m 이동 ",
                  "linestring": "127.02312,37.574593 127.02341,37.5746"
                },
                {
                  "streetName": "난계로",
                  "distance": 68,
                  "description": "좌회전 후 난계로 을 따라 68m 이동 ",
                  "linestring": "127.02341,37.5746 127.02339,37.57507 127.02342,37.575127 127.02345,37.575157 127.02349,37.575172"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 30,
                  "description": "신설동역 10번출구 에서 좌측 횡단보도 후 보행자도로 을 따라 30m 이동 ",
                  "linestring": "127.023994,37.575058 127.02418,37.575283"
                },
                {
                  "streetName": "",
                  "distance": 30,
                  "description": "동진회관 에서 우회전 후 30m 이동 ",
                  "linestring": "127.02418,37.575283 127.02447,37.575172 127.02449,37.575165"
                },
                {
                  "streetName": "천호대로",
                  "distance": 31,
                  "description": "동진회관 에서 좌회전 후 천호대로 을 따라 31m 이동 ",
                  "linestring": "127.02449,37.575165 127.024666,37.57541"
                }
              ]
            }
          ],
          "totalWalkTime": 511,
          "transferCount": 0,
          "totalDistance": 883,
          "pathType": 2,
          "totalWalkDistance": 596
        },
        {
          "fare": {
            "regular": {
              "totalFare": 1500,
              "currency": {
                "symbol": "￦",
                "currency": "원",
                "currencyCode": "KRW"
              }
            }
          },
          "totalTime": 1085,
          "legs": [
            {
              "mode": "WALK",
              "sectionTime": 446,
              "distance": 485,
              "start": {
                "name": "출발지",
                "lon": 127.016589,
                "lat": 37.573246
              },
              "end": {
                "name": "청계8가.영도교",
                "lon": 127.01876388888888,
                "lat": 37.57069444444444
              },
              "steps": [
                {
                  "streetName": "",
                  "distance": 68,
                  "description": "68m 이동",
                  "linestring": "127.01658,37.57326 127.01721,37.57341 127.01731,37.573437"
                },
                {
                  "streetName": "",
                  "distance": 29,
                  "description": "동묘앞역  3번출구 에서 우회전 후 29m 이동 ",
                  "linestring": "127.01731,37.573437 127.01736,37.573322 127.0172,37.57328 127.01719,37.573273"
                },
                {
                  "streetName": "왕산로",
                  "distance": 35,
                  "description": "동묘앞역  3번출구 에서 좌회전 후 왕산로 을 따라 35m 이동 ",
                  "linestring": "127.01719,37.573273 127.017296,37.573296 127.01739,37.57332 127.01758,37.573368"
                },
                {
                  "streetName": "청계천로",
                  "distance": 269,
                  "description": "우회전 후 청계천로 을 따라 269m 이동 ",
                  "linestring": "127.01758,37.573368 127.017715,37.573166 127.0178,37.573044 127.01796,37.57281 127.018105,37.572536 127.018196,37.57237 127.01825,37.572277 127.018456,37.5718 127.01847,37.571777 127.01862,37.57145 127.01878,37.571266 127.018715,37.57122 127.018684,37.57121"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 55,
                  "description": "한일전기 에서 횡단보도 후 보행자도로 을 따라 55m 이동 ",
                  "linestring": "127.018684,37.57121 127.018776,37.571102 127.01892,37.570904 127.01905,37.57081"
                },
                {
                  "streetName": "청계천로",
                  "distance": 29,
                  "description": "우회전 후 청계천로 을 따라 29m 이동 ",
                  "linestring": "127.01905,37.57081 127.01878,37.570667"
                }
              ]
            },
            {
              "mode": "BUS",
              "routeColor": "0068B7",
              "sectionTime": 360,
              "route": "간선:173",
              "routeId": "11442001",
              "distance": 884,
              "service": 1,
              "start": {
                "name": "청계8가.영도교",
                "lon": 127.01876388888888,
                "lat": 37.57069444444444
              },
              "passStopList": {
                "stationList": [
                  {
                    "index": 0,
                    "stationName": "청계8가.영도교",
                    "lon": "127.018764",
                    "lat": "37.570694",
                    "stationID": "775097"
                  },
                  {
                    "index": 1,
                    "stationName": "황학동롯데캐슬",
                    "lon": "127.021911",
                    "lat": "37.571689",
                    "stationID": "775126"
                  },
                  {
                    "index": 2,
                    "stationName": "신설동로터리.서울풍물시장",
                    "lon": "127.023431",
                    "lat": "37.573869",
                    "stationID": "775213"
                  },
                  {
                    "index": 3,
                    "stationName": "신설동역1번출구",
                    "lon": "127.023336",
                    "lat": "37.576639",
                    "stationID": "775300"
                  }
                ]
              },
              "end": {
                "name": "신설동역1번출구",
                "lon": 127.0233361111111,
                "lat": 37.57663888888889
              },
              "type": 11,
              "passShape": {
                "linestring": "127.018742,37.570728 127.019589,37.571172 127.020017,37.571375 127.020478,37.571528 127.020764,37.571603 127.021347,37.571703 127.021900,37.571725 127.021975,37.571728 127.022231,37.571711 127.022733,37.571642 127.023303,37.571503 127.023486,37.571450 127.023386,37.573872 127.023325,37.575431 127.023314,37.575700 127.023292,37.576628"
              }
            },
            {
              "mode": "WALK",
              "sectionTime": 279,
              "distance": 264,
              "start": {
                "name": "신설동역1번출구",
                "lon": 127.0233361111111,
                "lat": 37.57663888888889
              },
              "end": {
                "name": "도착지",
                "lon": 127.0248495,
                "lat": 37.575326
              },
              "steps": [
                {
                  "streetName": "보문로",
                  "distance": 68,
                  "description": "보문로 을 따라 68m 이동",
                  "linestring": "127.02337,37.576637 127.02337,37.5765 127.02337,37.576435 127.02337,37.576225 127.02338,37.576145 127.023384,37.576023"
                },
                {
                  "streetName": "",
                  "distance": 7,
                  "description": "우측 횡단보도 후 7m 이동 ",
                  "linestring": "127.023445,37.575916 127.0234,37.575863"
                },
                {
                  "streetName": "",
                  "distance": 6,
                  "description": "직진 후 6m 이동 ",
                  "linestring": "127.0234,37.575863 127.02339,37.575848 127.02342,37.575836"
                },
                {
                  "streetName": "",
                  "distance": 14,
                  "description": "횡단보도 후 14m 이동 ",
                  "linestring": "127.02342,37.575836 127.02349,37.575714"
                },
                {
                  "streetName": "",
                  "distance": 13,
                  "description": "좌회전 후 13m 이동 ",
                  "linestring": "127.02349,37.575714 127.02362,37.575775"
                },
                {
                  "streetName": "",
                  "distance": 22,
                  "description": "우측 횡단보도 후 22m 이동 ",
                  "linestring": "127.02362,37.575775 127.02373,37.5756"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 13,
                  "description": "신설동역  6번출구 에서 우회전 후 보행자도로 을 따라 13m 이동 ",
                  "linestring": "127.02373,37.5756 127.02371,37.575565 127.02371,37.575523 127.02373,37.57549 127.023735,37.575485"
                },
                {
                  "streetName": "천호대로",
                  "distance": 31,
                  "description": "동진회관 에서 좌회전 후 천호대로 을 따라 31m 이동 ",
                  "linestring": "127.02449,37.575165 127.024666,37.57541"
                }
              ]
            }
          ],
          "totalWalkTime": 725,
          "transferCount": 0,
          "totalDistance": 1425,
          "pathType": 2,
          "totalWalkDistance": 749
        }
      ]
    }
  }
}
'''

# JSON 문자열을 Python 딕셔너리로 변환
data = json.loads(json_data)

# 딕셔너리 확인
print(data)


{'metaData': {'requestParameters': {'busCount': 5, 'expressbusCount': 0, 'subwayCount': 1, 'airplaneCount': 0, 'locale': 'ko', 'endY': '37.575326', 'endX': '127.0248495', 'wideareaRouteCount': 0, 'subwayBusCount': 0, 'startY': '37.573246', 'startX': '127.016589', 'ferryCount': 0, 'trainCount': 0, 'reqDttm': '20241024213754'}, 'plan': {'itineraries': [{'fare': {'regular': {'totalFare': 1400, 'currency': {'symbol': '￦', 'currency': '원', 'currencyCode': 'KRW'}}}, 'totalTime': 408, 'legs': [{'mode': 'WALK', 'sectionTime': 133, 'distance': 148, 'start': {'name': '출발지', 'lon': 127.016589, 'lat': 37.573246}, 'end': {'name': '동묘앞', 'lon': 127.01659444444445, 'lat': 37.57326666666667}, 'steps': [{'streetName': '', 'distance': 68, 'description': '68m 이동', 'linestring': '127.01658,37.57326 127.01721,37.57341 127.01731,37.573437'}, {'streetName': '', 'distance': 29, 'description': '동묘앞역  3번출구 에서 우회전 후 29m 이동 ', 'linestring': '127.01731,37.573437 127.01736,37.573322 127.0172,37.57328 127.01719,37.5

테스트용 data 기본 꼴

In [ ]:
import json

# JSON 데이터를 문자열로 할당 (트리플 따옴표 사용)
json_data = ''''''

# JSON 문자열을 Python 딕셔너리로 변환
data = json.loads(json_data)

# 딕셔너리 확인
print(data)

테스트: 고잔역 - 청량리역 경로 정보 data에 할당

In [1]:
import json

# JSON 데이터를 문자열로 할당 (트리플 따옴표 사용)
json_data = '''{
  "metaData": {
    "requestParameters": {
      "busCount": 0,
      "expressbusCount": 0,
      "subwayCount": 4,
      "airplaneCount": 0,
      "locale": "ko",
      "endY": "37.581381",
      "endX": "127.048958",
      "wideareaRouteCount": 0,
      "subwayBusCount": 3,
      "startY": "37.3168267",
      "startX": "126.8230369",
      "ferryCount": 0,
      "trainCount": 3,
      "reqDttm": "20241024224044"
    },
    "plan": {
      "itineraries": [
        {
          "fare": {
            "regular": {
              "totalFare": 2200,
              "currency": {
                "symbol": "￦",
                "currency": "원",
                "currencyCode": "KRW"
              }
            }
          },
          "totalTime": 4802,
          "legs": [
            {
              "mode": "WALK",
              "sectionTime": 101,
              "distance": 121,
              "start": {
                "name": "출발지",
                "lon": 126.8230369,
                "lat": 37.3168267
              },
              "end": {
                "name": "고잔",
                "lon": 126.82304722222223,
                "lat": 37.316833333333335
              },
              "steps": [
                {
                  "streetName": "",
                  "distance": 27,
                  "description": "27m 이동",
                  "linestring": "126.82303,37.3168 126.82306,37.3168 126.82333,37.316788"
                },
                {
                  "streetName": "",
                  "distance": 15,
                  "description": "고잔역 에서 좌회전 후 15m 이동 ",
                  "linestring": "126.82333,37.316788 126.82334,37.31693"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 20,
                  "description": "직진 후 보행자도로 을 따라 20m 이동 ",
                  "linestring": "126.82334,37.31693 126.82336,37.317104"
                },
                {
                  "streetName": "중앙대로",
                  "distance": 39,
                  "description": "좌회전 후 중앙대로 을 따라 39m 이동 ",
                  "linestring": "126.82336,37.317104 126.822914,37.317135"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 20,
                  "description": "좌회전 후 보행자도로 을 따라 20m 이동 ",
                  "linestring": "126.822914,37.317135 126.82289,37.316963"
                }
              ]
            },
            {
              "mode": "SUBWAY",
              "routeColor": "00A5DE",
              "sectionTime": 3361,
              "route": "수도권4호선",
              "routeId": "110041005",
              "distance": 35317,
              "service": 1,
              "start": {
                "name": "고잔",
                "lon": 126.82304722222223,
                "lat": 37.316833333333335
              },
              "passStopList": {
                "stationList": [
                  {
                    "index": 0,
                    "stationName": "고잔",
                    "lon": "126.823047",
                    "lat": "37.316833",
                    "stationID": "110446"
                  },
                  {
                    "index": 1,
                    "stationName": "중앙",
                    "lon": "126.838481",
                    "lat": "37.316058",
                    "stationID": "110445"
                  },
                  {
                    "index": 2,
                    "stationName": "한대앞",
                    "lon": "126.853636",
                    "lat": "37.309722",
                    "stationID": "110444"
                  },
                  {
                    "index": 3,
                    "stationName": "상록수",
                    "lon": "126.866383",
                    "lat": "37.302869",
                    "stationID": "110443"
                  },
                  {
                    "index": 4,
                    "stationName": "반월",
                    "lon": "126.903367",
                    "lat": "37.311867",
                    "stationID": "110442"
                  },
                  {
                    "index": 5,
                    "stationName": "대야미",
                    "lon": "126.916878",
                    "lat": "37.327964",
                    "stationID": "110441"
                  },
                  {
                    "index": 6,
                    "stationName": "수리산",
                    "lon": "126.925617",
                    "lat": "37.350569",
                    "stationID": "110440"
                  },
                  {
                    "index": 7,
                    "stationName": "산본",
                    "lon": "126.933019",
                    "lat": "37.357989",
                    "stationID": "110439"
                  },
                  {
                    "index": 8,
                    "stationName": "금정",
                    "lon": "126.943608",
                    "lat": "37.372211",
                    "stationID": "110438"
                  },
                  {
                    "index": 9,
                    "stationName": "범계",
                    "lon": "126.950675",
                    "lat": "37.389764",
                    "stationID": "110437"
                  },
                  {
                    "index": 10,
                    "stationName": "평촌",
                    "lon": "126.964106",
                    "lat": "37.394431",
                    "stationID": "110436"
                  },
                  {
                    "index": 11,
                    "stationName": "인덕원",
                    "lon": "126.976800",
                    "lat": "37.401900",
                    "stationID": "110435"
                  },
                  {
                    "index": 12,
                    "stationName": "정부과천청사",
                    "lon": "126.989942",
                    "lat": "37.426792",
                    "stationID": "110434"
                  },
                  {
                    "index": 13,
                    "stationName": "과천",
                    "lon": "126.996669",
                    "lat": "37.433281",
                    "stationID": "110433"
                  },
                  {
                    "index": 14,
                    "stationName": "대공원",
                    "lon": "127.006531",
                    "lat": "37.435831",
                    "stationID": "110432"
                  },
                  {
                    "index": 15,
                    "stationName": "경마공원",
                    "lon": "127.007944",
                    "lat": "37.443806",
                    "stationID": "110431"
                  },
                  {
                    "index": 16,
                    "stationName": "선바위",
                    "lon": "127.001972",
                    "lat": "37.452111",
                    "stationID": "110430"
                  },
                  {
                    "index": 17,
                    "stationName": "남태령",
                    "lon": "126.988883",
                    "lat": "37.464647",
                    "stationID": "110429"
                  },
                  {
                    "index": 18,
                    "stationName": "사당",
                    "lon": "126.981667",
                    "lat": "37.476808",
                    "stationID": "110428"
                  },
                  {
                    "index": 19,
                    "stationName": "총신대입구",
                    "lon": "126.982264",
                    "lat": "37.487347",
                    "stationID": "110427"
                  },
                  {
                    "index": 20,
                    "stationName": "동작",
                    "lon": "126.980344",
                    "lat": "37.502897",
                    "stationID": "110426"
                  },
                  {
                    "index": 21,
                    "stationName": "이촌",
                    "lon": "126.974444",
                    "lat": "37.522350",
                    "stationID": "110425"
                  }
                ]
              },
              "end": {
                "name": "이촌",
                "lon": 126.97444444444444,
                "lat": 37.52235
              },
              "type": 4,
              "passShape": {
                "linestring": "126.823047,37.316833 126.824414,37.316767 126.826906,37.316628 126.830992,37.316458 126.832469,37.316378 126.833967,37.316292 126.835467,37.316217 126.836961,37.316139 126.838481,37.316061 126.839886,37.315986 126.842467,37.315819 126.844714,37.315669 126.845500,37.315581 126.846008,37.315494 126.846500,37.315406 126.846953,37.315281 126.847328,37.315158 126.847739,37.314989 126.848297,37.314700 126.848778,37.314403 126.849225,37.314086 126.849656,37.313689 126.850683,37.312733 126.851578,37.311858 126.852081,37.311344 126.852964,37.310425 126.853639,37.309722 126.853753,37.309606 126.854828,37.308542 126.855744,37.307639 126.856328,37.307072 126.857392,37.306008 126.857856,37.305514 126.858386,37.304958 126.858814,37.304581 126.859214,37.304283 126.859542,37.304086 126.859942,37.303889 126.860364,37.303700 126.860919,37.303478 126.861364,37.303333 126.861831,37.303228 126.862433,37.303131 126.863000,37.303081 126.863614,37.303056 126.864522,37.302989 126.864847,37.302964 126.866383,37.302867 126.867319,37.302806 126.871492,37.302481 126.872903,37.302453 126.873700,37.302464 126.874558,37.302494 126.875353,37.302522 126.878306,37.302639 126.879389,37.302683 126.879444,37.302686 126.880114,37.302728 126.881167,37.302778 126.881628,37.302781 126.882600,37.302847 126.883436,37.303019 126.884569,37.303317 126.885189,37.303486 126.885669,37.303642 126.886589,37.303892 126.888092,37.304297 126.888661,37.304450 126.890783,37.305022 126.892403,37.305458 126.892564,37.305503 126.892981,37.305614 126.894681,37.306069 126.895633,37.306331 126.896247,37.306492 126.896492,37.306558 126.896719,37.306622 126.896942,37.306689 126.897150,37.306756 126.897361,37.306831 126.897625,37.306928 126.897911,37.307047 126.898361,37.307239 126.898828,37.307475 126.899233,37.307703 126.899661,37.307931 126.899892,37.308075 126.900097,37.308219 126.900678,37.308725 126.900789,37.308831 126.901239,37.309314 126.901683,37.309800 126.902156,37.310414 126.902169,37.310428 126.903367,37.311867 126.904406,37.313108 126.905708,37.314681 126.907406,37.316717 126.908850,37.318425 126.910258,37.320100 126.911522,37.321614 126.912292,37.322533 126.912761,37.323092 126.913981,37.324544 126.914781,37.325519 126.915297,37.326094 126.915756,37.326639 126.916767,37.327828 126.916878,37.327964 126.917661,37.328922 126.917814,37.329117 126.918161,37.329506 126.918344,37.329744 126.918844,37.330286 126.919256,37.330789 126.919794,37.331383 126.920208,37.331872 126.920333,37.332044 126.920756,37.332558 126.921186,37.333181 126.921350,37.333408 126.921494,37.333658 126.921644,37.333936 126.921775,37.334258 126.922056,37.334978 126.922269,37.335875 126.922333,37.336169 126.922636,37.337656 126.923267,37.340964 126.923528,37.342833 126.923919,37.344722 126.924153,37.345831 126.925533,37.350419 126.925583,37.350578 126.925914,37.351614 126.926225,37.352608 126.926347,37.352931 126.926475,37.353253 126.926664,37.353675 126.926964,37.354200 126.927178,37.354453 126.927636,37.354939 126.928156,37.355353 126.930842,37.356931 126.932967,37.358056 126.935383,37.359333 126.936328,37.359833 126.939789,37.361636 126.942525,37.363175 126.944708,37.364322 126.944894,37.364464 126.945069,37.364628 126.945267,37.364825 126.945411,37.365006 126.945514,37.365156 126.945586,37.365269 126.945653,37.365397 126.945731,37.365625 126.945792,37.365825 126.945825,37.366022 126.945839,37.366267 126.945839,37.366431 126.945808,37.366608 126.945775,37.366781 126.945708,37.366978 126.945628,37.367172 126.945536,37.367378 126.945350,37.367761 126.945236,37.367994 126.945072,37.368339 126.945006,37.368506 126.943858,37.371331 126.943644,37.371906 126.943539,37.372194 126.943325,37.372775 126.942281,37.374958 126.941906,37.375700 126.941767,37.375947 126.941228,37.376814 126.940958,37.377247 126.940797,37.377472 126.940664,37.377628 126.940542,37.377758 126.940036,37.378289 126.939917,37.378406 126.938700,37.379531 126.938669,37.379575 126.938653,37.379619 126.938642,37.379664 126.938633,37.379706 126.938636,37.379767 126.938644,37.379828 126.938872,37.380197 126.939181,37.380786 126.939919,37.382556 126.940189,37.383083 126.940436,37.383478 126.940786,37.383844 126.941064,37.384042 126.941464,37.384264 126.941989,37.384469 126.942383,37.384636 126.942733,37.384825 126.942994,37.384981 126.943428,37.385258 126.943864,37.385597 126.945186,37.386678 126.947628,37.388694 126.947642,37.388703 126.947653,37.388714 126.947672,37.388722 126.947686,37.388728 126.947700,37.388733 126.950675,37.389767 126.962272,37.393797 126.964086,37.394467 126.964389,37.394578 126.968514,37.395942 126.971767,37.397033 126.973083,37.397500 126.974419,37.397997 126.975417,37.398531 126.975703,37.398761 126.975883,37.398925 126.976144,37.399208 126.976603,37.399822 126.976797,37.400136 126.976881,37.400292 126.976914,37.400467 126.976833,37.401311 126.976833,37.401350 126.976783,37.402194 126.976803,37.402544 126.976883,37.402842 126.977764,37.404747 126.977808,37.404872 126.978228,37.406642 126.978494,37.407764 126.979033,37.410022 126.979122,37.410444 126.979419,37.411742 126.979725,37.412986 126.979781,37.413208 126.979867,37.413675 126.979908,37.413847 126.980011,37.414200 126.980928,37.416297 126.981547,37.417675 126.981750,37.418144 126.982047,37.418675 126.982469,37.419406 126.982989,37.420228 126.983950,37.421275 126.984853,37.422069 126.985869,37.422925 126.987003,37.423869 126.987781,37.424625 126.988981,37.425722 126.989956,37.426783 126.990411,37.427278 126.991381,37.428233 126.992906,37.429739 126.995314,37.432433 126.995839,37.432964 126.996069,37.433125 126.996378,37.433244 126.996656,37.433317 126.996756,37.433342 126.997372,37.433425 126.997742,37.433425 126.998319,37.433353 126.999225,37.433258 126.999853,37.433225 127.000758,37.433239 127.004331,37.433214 127.004481,37.433256 127.005314,37.433958 127.005433,37.434131 127.006244,37.435392 127.006450,37.435711 127.008228,37.438322 127.008297,37.438447 127.008344,37.438575 127.008364,37.438711 127.008375,37.438969 127.008203,37.440986 127.008019,37.442944 127.007969,37.443506 127.007944,37.443831 127.007819,37.444217 127.007717,37.444408 127.007436,37.444817 127.006928,37.445475 127.004367,37.448900 127.002544,37.451400 127.001972,37.452111 127.001836,37.452281 127.001633,37.452472 127.001442,37.452575 127.000942,37.452764 126.999947,37.453253 126.998028,37.454378 126.997922,37.454442 126.996819,37.455239 126.995569,37.456381 126.994317,37.457522 126.993775,37.458058 126.993728,37.458117 126.993250,37.458667 126.991156,37.461425 126.989422,37.463706 126.989111,37.464247 126.988883,37.464647 126.987964,37.466256 126.987881,37.466394 126.987797,37.466528 126.987708,37.466664 126.987617,37.466794 126.987525,37.466928 126.987428,37.467058 126.987328,37.467186 126.987225,37.467314 126.987122,37.467442 126.987014,37.467567 126.986903,37.467689 126.986792,37.467811 126.986675,37.467931 126.986558,37.468050 126.986439,37.468167 126.986314,37.468283 126.986189,37.468394 126.986061,37.468508 126.985931,37.468619 126.985800,37.468728 126.985664,37.468833 126.985211,37.469178 126.984864,37.469464 126.984636,37.469658 126.984472,37.469808 126.984356,37.469928 126.983858,37.470544 126.983769,37.470672 126.983256,37.471453 126.982953,37.472042 126.982397,37.473183 126.982053,37.474025 126.981878,37.474547 126.981769,37.474950 126.981675,37.475608 126.981669,37.475658 126.981633,37.476083 126.981656,37.476564 126.981658,37.476653 126.982111,37.485253 126.982117,37.485383 126.982122,37.485475 126.982158,37.486308 126.982228,37.487350 126.982233,37.487400 126.982378,37.488686 126.982531,37.489653 126.982592,37.490369 126.982647,37.494967 126.982628,37.496269 126.982639,37.496469 126.982664,37.496669 126.982692,37.496817 126.982744,37.497011 126.982794,37.497156 126.982983,37.497625 126.983100,37.497950 126.983239,37.498383 126.983267,37.498531 126.983264,37.498681 126.983233,37.498825 126.983178,37.498967 126.983067,37.499131 126.981803,37.500881 126.981647,37.501097 126.981553,37.501222 126.981425,37.501369 126.980922,37.501886 126.980856,37.501964 126.980794,37.502033 126.980644,37.502233 126.980547,37.502394 126.980464,37.502575 126.980381,37.502808 126.980294,37.503031 126.980269,37.503125 126.980183,37.503464 126.980156,37.503878 126.980156,37.504058 126.980164,37.504244 126.980186,37.504425 126.980258,37.504800 126.980719,37.506508 126.983114,37.515581 126.983161,37.515808 126.983203,37.516089 126.983231,37.516306 126.983269,37.517844 126.983278,37.518414 126.983261,37.518581 126.983225,37.518736 126.983183,37.518883 126.983128,37.519019 126.983053,37.519164 126.982961,37.519292 126.982853,37.519408 126.982725,37.519531 126.982633,37.519608 126.982606,37.519633 126.982456,37.519739 126.982292,37.519836 126.982108,37.519936 126.981919,37.520033 126.981719,37.520119 126.981503,37.520206 126.976931,37.521594 126.975139,37.522128 126.974444,37.522344"
              }
            },
            {
              "mode": "WALK",
              "sectionTime": 148,
              "distance": 135,
              "start": {
                "name": "이촌",
                "lon": 126.97444444444444,
                "lat": 37.52235
              },
              "end": {
                "name": "이촌",
                "lon": 126.97354722222222,
                "lat": 37.5224
              },
              "passShape": {
                "linestring": "126.974444,37.522350 126.973544,37.522650 126.973483,37.522525 126.973397,37.522556 126.973356,37.522542 126.973547,37.522400"
              }
            },
            {
              "routeColor": "77C4A3",
              "distance": 10254,
              "start": {
                "name": "이촌",
                "lon": 126.97354722222222,
                "lat": 37.5224
              },
              "Lane": [
                {
                  "routeColor": "77C4A3",
                  "route": "경의중앙선(급행)",
                  "routeId": "111121001",
                  "service": 0,
                  "type": 122
                }
              ],
              "type": 104,
              "mode": "SUBWAY",
              "sectionTime": 900,
              "route": "경의중앙선",
              "routeId": "110121015",
              "service": 0,
              "passStopList": {
                "stationList": [
                  {
                    "index": 0,
                    "stationName": "이촌",
                    "lon": "126.973547",
                    "lat": "37.522400",
                    "stationID": "111228"
                  },
                  {
                    "index": 1,
                    "stationName": "서빙고",
                    "lon": "126.988436",
                    "lat": "37.519611",
                    "stationID": "111227"
                  },
                  {
                    "index": 2,
                    "stationName": "한남",
                    "lon": "127.009019",
                    "lat": "37.529375",
                    "stationID": "111226"
                  },
                  {
                    "index": 3,
                    "stationName": "옥수",
                    "lon": "127.018722",
                    "lat": "37.540525",
                    "stationID": "111225"
                  },
                  {
                    "index": 4,
                    "stationName": "응봉",
                    "lon": "127.034544",
                    "lat": "37.550053",
                    "stationID": "111224"
                  },
                  {
                    "index": 5,
                    "stationName": "왕십리",
                    "lon": "127.038528",
                    "lat": "37.561531",
                    "stationID": "111223"
                  },
                  {
                    "index": 6,
                    "stationName": "청량리",
                    "lon": "127.047875",
                    "lat": "37.579942",
                    "stationID": "111222"
                  }
                ]
              },
              "end": {
                "name": "청량리",
                "lon": 127.047875,
                "lat": 37.57994166666667
              },
              "passShape": {
                "linestring": "126.973550,37.522408 126.981958,37.519767 126.982122,37.519725 126.982289,37.519683 126.982456,37.519647 126.982622,37.519611 126.982633,37.519608 126.982792,37.519581 126.982958,37.519550 126.983131,37.519525 126.983300,37.519500 126.983472,37.519478 126.983644,37.519461 126.983817,37.519444 126.983989,37.519431 126.984161,37.519422 126.984333,37.519414 126.984506,37.519408 126.984681,37.519408 126.984853,37.519408 126.985025,37.519414 126.985200,37.519419 126.985372,37.519431 126.985544,37.519442 126.987156,37.519569 126.988433,37.519614 126.989928,37.519667 126.990758,37.519683 126.992892,37.519725 126.992978,37.519725 126.993067,37.519728 126.993156,37.519731 126.993242,37.519733 126.993331,37.519739 126.993417,37.519744 126.993503,37.519753 126.993592,37.519764 126.993678,37.519772 126.993764,37.519783 126.993850,37.519797 126.993936,37.519814 126.994019,37.519828 126.994106,37.519844 126.994192,37.519864 126.994275,37.519883 126.994358,37.519903 126.994442,37.519925 126.994525,37.519950 126.994606,37.519975 126.994689,37.520000 126.995186,37.520194 127.000000,37.522139 127.000111,37.522186 127.000222,37.522236 127.000331,37.522289 127.000439,37.522344 127.000544,37.522403 127.000647,37.522464 127.000747,37.522528 127.000844,37.522594 127.000939,37.522661 127.001031,37.522731 127.001119,37.522806 127.001208,37.522878 127.001292,37.522956 127.001372,37.523033 127.001450,37.523117 127.001525,37.523197 127.001597,37.523283 127.001664,37.523369 127.001731,37.523456 127.001792,37.523544 127.001850,37.523636 127.001956,37.523800 127.002581,37.524836 127.002617,37.524897 127.002656,37.524958 127.002697,37.525022 127.002739,37.525081 127.002781,37.525142 127.002828,37.525203 127.002872,37.525261 127.002919,37.525319 127.002967,37.525378 127.003017,37.525433 127.003067,37.525492 127.003117,37.525547 127.003169,37.525603 127.003225,37.525656 127.003281,37.525711 127.003336,37.525764 127.003392,37.525817 127.003453,37.525867 127.003511,37.525917 127.003572,37.525967 127.003633,37.526017 127.006289,37.527975 127.006358,37.528031 127.006433,37.528083 127.006506,37.528139 127.006581,37.528192 127.006656,37.528244 127.006733,37.528297 127.006811,37.528350 127.006889,37.528400 127.006969,37.528447 127.007050,37.528497 127.007131,37.528544 127.007211,37.528592 127.007294,37.528636 127.007378,37.528681 127.007461,37.528725 127.007544,37.528769 127.007631,37.528811 127.007717,37.528853 127.007806,37.528892 127.007892,37.528931 127.007981,37.528969 127.009019,37.529375 127.010064,37.529781 127.010194,37.529831 127.010325,37.529889 127.010450,37.529947 127.010575,37.530008 127.010694,37.530075 127.010814,37.530144 127.010928,37.530217 127.011042,37.530292 127.011150,37.530369 127.011256,37.530450 127.011358,37.530533 127.011458,37.530619 127.011553,37.530708 127.011644,37.530803 127.011731,37.530894 127.011814,37.530992 127.011894,37.531089 127.011969,37.531189 127.012042,37.531292 127.012108,37.531394 127.012169,37.531500 127.016536,37.538567 127.016575,37.538631 127.016614,37.538694 127.016653,37.538758 127.016694,37.538822 127.016739,37.538883 127.016781,37.538947 127.016828,37.539008 127.016875,37.539069 127.016925,37.539128 127.016975,37.539189 127.017025,37.539244 127.017078,37.539303 127.017133,37.539358 127.017189,37.539417 127.017244,37.539469 127.017306,37.539525 127.017364,37.539578 127.017425,37.539631 127.017486,37.539683 127.017550,37.539733 127.017614,37.539783 127.017792,37.539903 127.018283,37.540219 127.018736,37.540511 127.020686,37.541767 127.021619,37.542442 127.021953,37.542683 127.023269,37.543683 127.023519,37.543864 127.023800,37.544064 127.024036,37.544217 127.024267,37.544347 127.024558,37.544503 127.024833,37.544636 127.025114,37.544750 127.025381,37.544839 127.025669,37.544931 127.026336,37.545111 127.027953,37.545569 127.028078,37.545611 127.028139,37.545631 127.028200,37.545653 127.028261,37.545675 127.028322,37.545697 127.028383,37.545719 127.028444,37.545744 127.028503,37.545767 127.028564,37.545792 127.028622,37.545817 127.028683,37.545842 127.028742,37.545867 127.028800,37.545894 127.028858,37.545919 127.028917,37.545947 127.028975,37.545975 127.029031,37.546003 127.029089,37.546031 127.032158,37.547544 127.032247,37.547589 127.032336,37.547636 127.032425,37.547683 127.032508,37.547733 127.032594,37.547783 127.032678,37.547836 127.032758,37.547892 127.032839,37.547944 127.032917,37.548003 127.032992,37.548061 127.033067,37.548119 127.033142,37.548181 127.033214,37.548244 127.033281,37.548308 127.033350,37.548372 127.033392,37.548414 127.033417,37.548439 127.033478,37.548506 127.033542,37.548575 127.033600,37.548644 127.033658,37.548717 127.033714,37.548786 127.034525,37.550017 127.034547,37.550050 127.034947,37.550611 127.034992,37.550667 127.035039,37.550719 127.035089,37.550775 127.035136,37.550828 127.035186,37.550881 127.035236,37.550931 127.035289,37.550981 127.035342,37.551033 127.035397,37.551081 127.035453,37.551131 127.035508,37.551178 127.035567,37.551225 127.035625,37.551272 127.035683,37.551319 127.035744,37.551364 127.035806,37.551408 127.035867,37.551450 127.035931,37.551492 127.035994,37.551533 127.036058,37.551575 127.036125,37.551617 127.037769,37.552650 127.038211,37.552967 127.038356,37.553083 127.038494,37.553203 127.038625,37.553331 127.038747,37.553458 127.038864,37.553594 127.038972,37.553733 127.039072,37.553875 127.039164,37.554019 127.039250,37.554169 127.039325,37.554319 127.039389,37.554475 127.039447,37.554631 127.039497,37.554789 127.039536,37.554950 127.039567,37.555111 127.039589,37.555272 127.039603,37.555436 127.039606,37.555600 127.039600,37.555761 127.039583,37.555925 127.039558,37.556086 127.039492,37.556456 127.038567,37.561286 127.038522,37.561528 127.038308,37.562647 127.037778,37.565417 127.037761,37.565531 127.037750,37.565644 127.037744,37.565758 127.037742,37.565872 127.037742,37.565986 127.037750,37.566103 127.037761,37.566217 127.037778,37.566328 127.037797,37.566442 127.037819,37.566556 127.037850,37.566667 127.037883,37.566778 127.037919,37.566889 127.037964,37.567000 127.038008,37.567106 127.038058,37.567214 127.038114,37.567319 127.038172,37.567425 127.038236,37.567528 127.038303,37.567628 127.038375,37.567728 127.038722,37.568219 127.040781,37.570906 127.042075,37.572578 127.043939,37.574981 127.046050,37.577747 127.046983,37.578828 127.047394,37.579303 127.047594,37.579567"
              }
            },
            {
              "mode": "WALK",
              "sectionTime": 292,
              "distance": 369,
              "start": {
                "name": "청량리",
                "lon": 127.047875,
                "lat": 37.57994166666667
              },
              "end": {
                "name": "도착지",
                "lon": 127.048958,
                "lat": 37.581381
              },
              "steps": [
                {
                  "streetName": "",
                  "distance": 58,
                  "description": "58m 이동",
                  "linestring": "127.047844,37.57991 127.04787,37.5799 127.04841,37.57964"
                },
                {
                  "streetName": "",
                  "distance": 16,
                  "description": "직진 후 16m 이동 ",
                  "linestring": "127.04841,37.57964 127.04856,37.579563"
                },
                {
                  "streetName": "",
                  "distance": 242,
                  "description": "좌회전 후 242m 이동 ",
                  "linestring": "127.04856,37.579563 127.04963,37.580936 127.04958,37.58105 127.04961,37.58109 127.04965,37.58115 127.04975,37.58128 127.04977,37.581333 127.04976,37.581387 127.04971,37.58145"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 53,
                  "description": "10시 방향 좌회전 후 보행자도로 을 따라 53m 이동 ",
                  "linestring": "127.04971,37.58145 127.04921,37.581703"
                }
              ]
            }
          ],
          "totalWalkTime": 541,
          "transferCount": 1,
          "totalDistance": 45895,
          "pathType": 1,
          "totalWalkDistance": 625
        },
        {
          "fare": {
            "regular": {
              "totalFare": 5000,
              "currency": {
                "symbol": "￦",
                "currency": "원",
                "currencyCode": "KRW"
              }
            }
          },
          "totalTime": 5449,
          "legs": [
            {
              "mode": "WALK",
              "sectionTime": 101,
              "distance": 121,
              "start": {
                "name": "출발지",
                "lon": 126.8230369,
                "lat": 37.3168267
              },
              "end": {
                "name": "고잔",
                "lon": 126.82304722222223,
                "lat": 37.316833333333335
              },
              "steps": [
                {
                  "streetName": "",
                  "distance": 27,
                  "description": "27m 이동",
                  "linestring": "126.82303,37.3168 126.82306,37.3168 126.82333,37.316788"
                },
                {
                  "streetName": "",
                  "distance": 15,
                  "description": "고잔역 에서 좌회전 후 15m 이동 ",
                  "linestring": "126.82333,37.316788 126.82334,37.31693"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 20,
                  "description": "직진 후 보행자도로 을 따라 20m 이동 ",
                  "linestring": "126.82334,37.31693 126.82336,37.317104"
                },
                {
                  "streetName": "중앙대로",
                  "distance": 39,
                  "description": "좌회전 후 중앙대로 을 따라 39m 이동 ",
                  "linestring": "126.82336,37.317104 126.822914,37.317135"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 20,
                  "description": "좌회전 후 보행자도로 을 따라 20m 이동 ",
                  "linestring": "126.822914,37.317135 126.82289,37.316963"
                }
              ]
            },
            {
              "mode": "SUBWAY",
              "routeColor": "00A5DE",
              "sectionTime": 3539,
              "route": "수도권4호선",
              "routeId": "110041005",
              "distance": 36273,
              "service": 1,
              "start": {
                "name": "고잔",
                "lon": 126.82304722222223,
                "lat": 37.316833333333335
              },
              "passStopList": {
                "stationList": [
                  {
                    "index": 0,
                    "stationName": "고잔",
                    "lon": "126.823047",
                    "lat": "37.316833",
                    "stationID": "110446"
                  },
                  {
                    "index": 1,
                    "stationName": "중앙",
                    "lon": "126.838481",
                    "lat": "37.316058",
                    "stationID": "110445"
                  },
                  {
                    "index": 2,
                    "stationName": "한대앞",
                    "lon": "126.853636",
                    "lat": "37.309722",
                    "stationID": "110444"
                  },
                  {
                    "index": 3,
                    "stationName": "상록수",
                    "lon": "126.866383",
                    "lat": "37.302869",
                    "stationID": "110443"
                  },
                  {
                    "index": 4,
                    "stationName": "반월",
                    "lon": "126.903367",
                    "lat": "37.311867",
                    "stationID": "110442"
                  },
                  {
                    "index": 5,
                    "stationName": "대야미",
                    "lon": "126.916878",
                    "lat": "37.327964",
                    "stationID": "110441"
                  },
                  {
                    "index": 6,
                    "stationName": "수리산",
                    "lon": "126.925617",
                    "lat": "37.350569",
                    "stationID": "110440"
                  },
                  {
                    "index": 7,
                    "stationName": "산본",
                    "lon": "126.933019",
                    "lat": "37.357989",
                    "stationID": "110439"
                  },
                  {
                    "index": 8,
                    "stationName": "금정",
                    "lon": "126.943608",
                    "lat": "37.372211",
                    "stationID": "110438"
                  },
                  {
                    "index": 9,
                    "stationName": "범계",
                    "lon": "126.950675",
                    "lat": "37.389764",
                    "stationID": "110437"
                  },
                  {
                    "index": 10,
                    "stationName": "평촌",
                    "lon": "126.964106",
                    "lat": "37.394431",
                    "stationID": "110436"
                  },
                  {
                    "index": 11,
                    "stationName": "인덕원",
                    "lon": "126.976800",
                    "lat": "37.401900",
                    "stationID": "110435"
                  },
                  {
                    "index": 12,
                    "stationName": "정부과천청사",
                    "lon": "126.989942",
                    "lat": "37.426792",
                    "stationID": "110434"
                  },
                  {
                    "index": 13,
                    "stationName": "과천",
                    "lon": "126.996669",
                    "lat": "37.433281",
                    "stationID": "110433"
                  },
                  {
                    "index": 14,
                    "stationName": "대공원",
                    "lon": "127.006531",
                    "lat": "37.435831",
                    "stationID": "110432"
                  },
                  {
                    "index": 15,
                    "stationName": "경마공원",
                    "lon": "127.007944",
                    "lat": "37.443806",
                    "stationID": "110431"
                  },
                  {
                    "index": 16,
                    "stationName": "선바위",
                    "lon": "127.001972",
                    "lat": "37.452111",
                    "stationID": "110430"
                  },
                  {
                    "index": 17,
                    "stationName": "남태령",
                    "lon": "126.988883",
                    "lat": "37.464647",
                    "stationID": "110429"
                  },
                  {
                    "index": 18,
                    "stationName": "사당",
                    "lon": "126.981667",
                    "lat": "37.476808",
                    "stationID": "110428"
                  },
                  {
                    "index": 19,
                    "stationName": "총신대입구",
                    "lon": "126.982264",
                    "lat": "37.487347",
                    "stationID": "110427"
                  },
                  {
                    "index": 20,
                    "stationName": "동작",
                    "lon": "126.980344",
                    "lat": "37.502897",
                    "stationID": "110426"
                  },
                  {
                    "index": 21,
                    "stationName": "이촌",
                    "lon": "126.974444",
                    "lat": "37.522350",
                    "stationID": "110425"
                  },
                  {
                    "index": 22,
                    "stationName": "신용산",
                    "lon": "126.967928",
                    "lat": "37.529222",
                    "stationID": "110424"
                  }
                ]
              },
              "end": {
                "name": "신용산",
                "lon": 126.96792777777777,
                "lat": 37.529222222222224
              },
              "type": 4,
              "passShape": {
                "linestring": "126.823047,37.316833 126.824414,37.316767 126.826906,37.316628 126.830992,37.316458 126.832469,37.316378 126.833967,37.316292 126.835467,37.316217 126.836961,37.316139 126.838481,37.316061 126.839886,37.315986 126.842467,37.315819 126.844714,37.315669 126.845500,37.315581 126.846008,37.315494 126.846500,37.315406 126.846953,37.315281 126.847328,37.315158 126.847739,37.314989 126.848297,37.314700 126.848778,37.314403 126.849225,37.314086 126.849656,37.313689 126.850683,37.312733 126.851578,37.311858 126.852081,37.311344 126.852964,37.310425 126.853639,37.309722 126.853753,37.309606 126.854828,37.308542 126.855744,37.307639 126.856328,37.307072 126.857392,37.306008 126.857856,37.305514 126.858386,37.304958 126.858814,37.304581 126.859214,37.304283 126.859542,37.304086 126.859942,37.303889 126.860364,37.303700 126.860919,37.303478 126.861364,37.303333 126.861831,37.303228 126.862433,37.303131 126.863000,37.303081 126.863614,37.303056 126.864522,37.302989 126.864847,37.302964 126.866383,37.302867 126.867319,37.302806 126.871492,37.302481 126.872903,37.302453 126.873700,37.302464 126.874558,37.302494 126.875353,37.302522 126.878306,37.302639 126.879389,37.302683 126.879444,37.302686 126.880114,37.302728 126.881167,37.302778 126.881628,37.302781 126.882600,37.302847 126.883436,37.303019 126.884569,37.303317 126.885189,37.303486 126.885669,37.303642 126.886589,37.303892 126.888092,37.304297 126.888661,37.304450 126.890783,37.305022 126.892403,37.305458 126.892564,37.305503 126.892981,37.305614 126.894681,37.306069 126.895633,37.306331 126.896247,37.306492 126.896492,37.306558 126.896719,37.306622 126.896942,37.306689 126.897150,37.306756 126.897361,37.306831 126.897625,37.306928 126.897911,37.307047 126.898361,37.307239 126.898828,37.307475 126.899233,37.307703 126.899661,37.307931 126.899892,37.308075 126.900097,37.308219 126.900678,37.308725 126.900789,37.308831 126.901239,37.309314 126.901683,37.309800 126.902156,37.310414 126.902169,37.310428 126.903367,37.311867 126.904406,37.313108 126.905708,37.314681 126.907406,37.316717 126.908850,37.318425 126.910258,37.320100 126.911522,37.321614 126.912292,37.322533 126.912761,37.323092 126.913981,37.324544 126.914781,37.325519 126.915297,37.326094 126.915756,37.326639 126.916767,37.327828 126.916878,37.327964 126.917661,37.328922 126.917814,37.329117 126.918161,37.329506 126.918344,37.329744 126.918844,37.330286 126.919256,37.330789 126.919794,37.331383 126.920208,37.331872 126.920333,37.332044 126.920756,37.332558 126.921186,37.333181 126.921350,37.333408 126.921494,37.333658 126.921644,37.333936 126.921775,37.334258 126.922056,37.334978 126.922269,37.335875 126.922333,37.336169 126.922636,37.337656 126.923267,37.340964 126.923528,37.342833 126.923919,37.344722 126.924153,37.345831 126.925533,37.350419 126.925583,37.350578 126.925914,37.351614 126.926225,37.352608 126.926347,37.352931 126.926475,37.353253 126.926664,37.353675 126.926964,37.354200 126.927178,37.354453 126.927636,37.354939 126.928156,37.355353 126.930842,37.356931 126.932967,37.358056 126.935383,37.359333 126.936328,37.359833 126.939789,37.361636 126.942525,37.363175 126.944708,37.364322 126.944894,37.364464 126.945069,37.364628 126.945267,37.364825 126.945411,37.365006 126.945514,37.365156 126.945586,37.365269 126.945653,37.365397 126.945731,37.365625 126.945792,37.365825 126.945825,37.366022 126.945839,37.366267 126.945839,37.366431 126.945808,37.366608 126.945775,37.366781 126.945708,37.366978 126.945628,37.367172 126.945536,37.367378 126.945350,37.367761 126.945236,37.367994 126.945072,37.368339 126.945006,37.368506 126.943858,37.371331 126.943644,37.371906 126.943539,37.372194 126.943325,37.372775 126.942281,37.374958 126.941906,37.375700 126.941767,37.375947 126.941228,37.376814 126.940958,37.377247 126.940797,37.377472 126.940664,37.377628 126.940542,37.377758 126.940036,37.378289 126.939917,37.378406 126.938700,37.379531 126.938669,37.379575 126.938653,37.379619 126.938642,37.379664 126.938633,37.379706 126.938636,37.379767 126.938644,37.379828 126.938872,37.380197 126.939181,37.380786 126.939919,37.382556 126.940189,37.383083 126.940436,37.383478 126.940786,37.383844 126.941064,37.384042 126.941464,37.384264 126.941989,37.384469 126.942383,37.384636 126.942733,37.384825 126.942994,37.384981 126.943428,37.385258 126.943864,37.385597 126.945186,37.386678 126.947628,37.388694 126.947642,37.388703 126.947653,37.388714 126.947672,37.388722 126.947686,37.388728 126.947700,37.388733 126.950675,37.389767 126.962272,37.393797 126.964086,37.394467 126.964389,37.394578 126.968514,37.395942 126.971767,37.397033 126.973083,37.397500 126.974419,37.397997 126.975417,37.398531 126.975703,37.398761 126.975883,37.398925 126.976144,37.399208 126.976603,37.399822 126.976797,37.400136 126.976881,37.400292 126.976914,37.400467 126.976833,37.401311 126.976833,37.401350 126.976783,37.402194 126.976803,37.402544 126.976883,37.402842 126.977764,37.404747 126.977808,37.404872 126.978228,37.406642 126.978494,37.407764 126.979033,37.410022 126.979122,37.410444 126.979419,37.411742 126.979725,37.412986 126.979781,37.413208 126.979867,37.413675 126.979908,37.413847 126.980011,37.414200 126.980928,37.416297 126.981547,37.417675 126.981750,37.418144 126.982047,37.418675 126.982469,37.419406 126.982989,37.420228 126.983950,37.421275 126.984853,37.422069 126.985869,37.422925 126.987003,37.423869 126.987781,37.424625 126.988981,37.425722 126.989956,37.426783 126.990411,37.427278 126.991381,37.428233 126.992906,37.429739 126.995314,37.432433 126.995839,37.432964 126.996069,37.433125 126.996378,37.433244 126.996656,37.433317 126.996756,37.433342 126.997372,37.433425 126.997742,37.433425 126.998319,37.433353 126.999225,37.433258 126.999853,37.433225 127.000758,37.433239 127.004331,37.433214 127.004481,37.433256 127.005314,37.433958 127.005433,37.434131 127.006244,37.435392 127.006450,37.435711 127.008228,37.438322 127.008297,37.438447 127.008344,37.438575 127.008364,37.438711 127.008375,37.438969 127.008203,37.440986 127.008019,37.442944 127.007969,37.443506 127.007944,37.443831 127.007819,37.444217 127.007717,37.444408 127.007436,37.444817 127.006928,37.445475 127.004367,37.448900 127.002544,37.451400 127.001972,37.452111 127.001836,37.452281 127.001633,37.452472 127.001442,37.452575 127.000942,37.452764 126.999947,37.453253 126.998028,37.454378 126.997922,37.454442 126.996819,37.455239 126.995569,37.456381 126.994317,37.457522 126.993775,37.458058 126.993728,37.458117 126.993250,37.458667 126.991156,37.461425 126.989422,37.463706 126.989111,37.464247 126.988883,37.464647 126.987964,37.466256 126.987881,37.466394 126.987797,37.466528 126.987708,37.466664 126.987617,37.466794 126.987525,37.466928 126.987428,37.467058 126.987328,37.467186 126.987225,37.467314 126.987122,37.467442 126.987014,37.467567 126.986903,37.467689 126.986792,37.467811 126.986675,37.467931 126.986558,37.468050 126.986439,37.468167 126.986314,37.468283 126.986189,37.468394 126.986061,37.468508 126.985931,37.468619 126.985800,37.468728 126.985664,37.468833 126.985211,37.469178 126.984864,37.469464 126.984636,37.469658 126.984472,37.469808 126.984356,37.469928 126.983858,37.470544 126.983769,37.470672 126.983256,37.471453 126.982953,37.472042 126.982397,37.473183 126.982053,37.474025 126.981878,37.474547 126.981769,37.474950 126.981675,37.475608 126.981669,37.475658 126.981633,37.476083 126.981656,37.476564 126.981658,37.476653 126.982111,37.485253 126.982117,37.485383 126.982122,37.485475 126.982158,37.486308 126.982228,37.487350 126.982233,37.487400 126.982378,37.488686 126.982531,37.489653 126.982592,37.490369 126.982647,37.494967 126.982628,37.496269 126.982639,37.496469 126.982664,37.496669 126.982692,37.496817 126.982744,37.497011 126.982794,37.497156 126.982983,37.497625 126.983100,37.497950 126.983239,37.498383 126.983267,37.498531 126.983264,37.498681 126.983233,37.498825 126.983178,37.498967 126.983067,37.499131 126.981803,37.500881 126.981647,37.501097 126.981553,37.501222 126.981425,37.501369 126.980922,37.501886 126.980856,37.501964 126.980794,37.502033 126.980644,37.502233 126.980547,37.502394 126.980464,37.502575 126.980381,37.502808 126.980294,37.503031 126.980269,37.503125 126.980183,37.503464 126.980156,37.503878 126.980156,37.504058 126.980164,37.504244 126.980186,37.504425 126.980258,37.504800 126.980719,37.506508 126.983114,37.515581 126.983161,37.515808 126.983203,37.516089 126.983231,37.516306 126.983269,37.517844 126.983278,37.518414 126.983261,37.518581 126.983225,37.518736 126.983183,37.518883 126.983128,37.519019 126.983053,37.519164 126.982961,37.519292 126.982853,37.519408 126.982725,37.519531 126.982633,37.519608 126.982606,37.519633 126.982456,37.519739 126.982292,37.519836 126.982108,37.519936 126.981919,37.520033 126.981719,37.520119 126.981503,37.520206 126.976931,37.521594 126.975139,37.522128 126.974444,37.522344 126.972900,37.522833 126.972414,37.522969 126.972056,37.523053 126.971644,37.523133 126.971183,37.523214 126.970789,37.523292 126.970444,37.523367 126.970233,37.523428 126.970031,37.523500 126.969831,37.523583 126.969533,37.523728 126.966744,37.525375 126.966603,37.525464 126.966583,37.525478 126.966564,37.525489 126.966547,37.525503 126.966531,37.525517 126.966514,37.525531 126.966500,37.525547 126.966486,37.525564 126.966475,37.525581 126.966461,37.525594 126.966450,37.525614 126.966442,37.525631 126.966431,37.525650 126.966425,37.525667 126.966417,37.525686 126.966411,37.525703 126.966406,37.525722 126.966403,37.525742 126.966400,37.525761 126.966400,37.525781 126.966400,37.525800 126.966400,37.525819 126.966425,37.527406 126.966428,37.527431 126.966431,37.527456 126.966433,37.527481 126.966439,37.527506 126.966444,37.527528 126.966453,37.527553 126.966458,37.527578 126.966467,37.527600 126.966478,37.527625 126.966486,37.527647 126.966497,37.527669 126.966508,37.527692 126.966522,37.527714 126.966533,37.527739 126.966550,37.527758 126.966564,37.527781 126.966581,37.527803 126.966594,37.527822 126.966614,37.527844 126.966631,37.527864 126.966650,37.527883 126.967931,37.529222"
              }
            },
            {
              "mode": "WALK",
              "sectionTime": 340,
              "distance": 400,
              "start": {
                "name": "신용산",
                "lon": 126.96792777777777,
                "lat": 37.529222222222224
              },
              "end": {
                "name": "용산",
                "lon": 126.96482777777777,
                "lat": 37.530097222222224
              },
              "passShape": {
                "linestring": "126.967928,37.529222 126.967900,37.529211 126.967708,37.529331 126.967542,37.529153 126.967517,37.529150 126.967183,37.528806 126.966986,37.528597 126.966931,37.528539 126.966750,37.528356 126.966714,37.528356 126.966683,37.528367 126.966542,37.528453 126.966319,37.528589 126.966164,37.528686 126.966142,37.528719 126.966133,37.528742 126.965697,37.528986 126.965831,37.529128 126.965178,37.529669 126.964828,37.530097"
              }
            },
            {
              "routeColor": "F75C4C",
              "distance": 9287,
              "start": {
                "name": "용산",
                "lon": 126.96482777777777,
                "lat": 37.530097222222224
              },
              "type": 16,
              "mode": "TRAIN",
              "sectionTime": 1054,
              "route": "ITX청춘:용산-청량리",
              "routeId": "500002-500010-ITXCC001",
              "service": 1,
              "passStopList": {
                "stationList": [
                  {
                    "index": 0,
                    "stationName": "용산",
                    "lon": "126.964828",
                    "lat": "37.530097",
                    "stationID": "500002"
                  },
                  {
                    "index": 1,
                    "stationName": "청량리",
                    "lon": "127.048500",
                    "lat": "37.580742",
                    "stationID": "500010"
                  }
                ]
              },
              "end": {
                "name": "청량리",
                "lon": 127.0485,
                "lat": 37.58074166666667
              },
              "passShape": {
                "linestring": "126.964828,37.530097 127.048500,37.580742"
              },
              "routePayment": 3000
            },
            {
              "mode": "WALK",
              "sectionTime": 415,
              "distance": 492,
              "start": {
                "name": "청량리",
                "lon": 127.0485,
                "lat": 37.58074166666667
              },
              "end": {
                "name": "도착지",
                "lon": 127.048958,
                "lat": 37.581381
              },
              "steps": [
                {
                  "streetName": "",
                  "distance": 109,
                  "description": "109m 이동",
                  "linestring": "127.04837,37.580803 127.04826,37.58067 127.04796,37.580284 127.047714,37.579975"
                },
                {
                  "streetName": "",
                  "distance": 72,
                  "description": "던킨도너츠 청량리역사점 에서 좌회전 후 72m 이동 ",
                  "linestring": "127.047714,37.579975 127.04787,37.5799 127.04841,37.57964"
                },
                {
                  "streetName": "",
                  "distance": 16,
                  "description": "직진 후 16m 이동 ",
                  "linestring": "127.04841,37.57964 127.04856,37.579563"
                },
                {
                  "streetName": "",
                  "distance": 242,
                  "description": "좌회전 후 242m 이동 ",
                  "linestring": "127.04856,37.579563 127.04963,37.580936 127.04958,37.58105 127.04961,37.58109 127.04965,37.58115 127.04975,37.58128 127.04977,37.581333 127.04976,37.581387 127.04971,37.58145"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 53,
                  "description": "10시 방향 좌회전 후 보행자도로 을 따라 53m 이동 ",
                  "linestring": "127.04971,37.58145 127.04921,37.581703"
                }
              ]
            }
          ],
          "totalWalkTime": 856,
          "transferCount": 1,
          "totalDistance": 46044,
          "pathType": 5,
          "totalWalkDistance": 1013
        },
        {
          "fare": {
            "regular": {
              "totalFare": 2200,
              "currency": {
                "symbol": "￦",
                "currency": "원",
                "currencyCode": "KRW"
              }
            }
          },
          "totalTime": 5728,
          "legs": [
            {
              "mode": "WALK",
              "sectionTime": 101,
              "distance": 121,
              "start": {
                "name": "출발지",
                "lon": 126.8230369,
                "lat": 37.3168267
              },
              "end": {
                "name": "고잔",
                "lon": 126.82304722222223,
                "lat": 37.316833333333335
              },
              "steps": [
                {
                  "streetName": "",
                  "distance": 27,
                  "description": "27m 이동",
                  "linestring": "126.82303,37.3168 126.82306,37.3168 126.82333,37.316788"
                },
                {
                  "streetName": "",
                  "distance": 15,
                  "description": "고잔역 에서 좌회전 후 15m 이동 ",
                  "linestring": "126.82333,37.316788 126.82334,37.31693"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 20,
                  "description": "직진 후 보행자도로 을 따라 20m 이동 ",
                  "linestring": "126.82334,37.31693 126.82336,37.317104"
                },
                {
                  "streetName": "중앙대로",
                  "distance": 39,
                  "description": "좌회전 후 중앙대로 을 따라 39m 이동 ",
                  "linestring": "126.82336,37.317104 126.822914,37.317135"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 20,
                  "description": "좌회전 후 보행자도로 을 따라 20m 이동 ",
                  "linestring": "126.822914,37.317135 126.82289,37.316963"
                }
              ]
            },
            {
              "mode": "SUBWAY",
              "routeColor": "00A5DE",
              "sectionTime": 1379,
              "route": "수도권4호선",
              "routeId": "110041005",
              "distance": 15342,
              "service": 1,
              "start": {
                "name": "고잔",
                "lon": 126.82304722222223,
                "lat": 37.316833333333335
              },
              "passStopList": {
                "stationList": [
                  {
                    "index": 0,
                    "stationName": "고잔",
                    "lon": "126.823047",
                    "lat": "37.316833",
                    "stationID": "110446"
                  },
                  {
                    "index": 1,
                    "stationName": "중앙",
                    "lon": "126.838481",
                    "lat": "37.316058",
                    "stationID": "110445"
                  },
                  {
                    "index": 2,
                    "stationName": "한대앞",
                    "lon": "126.853636",
                    "lat": "37.309722",
                    "stationID": "110444"
                  },
                  {
                    "index": 3,
                    "stationName": "상록수",
                    "lon": "126.866383",
                    "lat": "37.302869",
                    "stationID": "110443"
                  },
                  {
                    "index": 4,
                    "stationName": "반월",
                    "lon": "126.903367",
                    "lat": "37.311867",
                    "stationID": "110442"
                  },
                  {
                    "index": 5,
                    "stationName": "대야미",
                    "lon": "126.916878",
                    "lat": "37.327964",
                    "stationID": "110441"
                  },
                  {
                    "index": 6,
                    "stationName": "수리산",
                    "lon": "126.925617",
                    "lat": "37.350569",
                    "stationID": "110440"
                  },
                  {
                    "index": 7,
                    "stationName": "산본",
                    "lon": "126.933019",
                    "lat": "37.357989",
                    "stationID": "110439"
                  },
                  {
                    "index": 8,
                    "stationName": "금정",
                    "lon": "126.943608",
                    "lat": "37.372211",
                    "stationID": "110438"
                  }
                ]
              },
              "end": {
                "name": "금정",
                "lon": 126.94360833333333,
                "lat": 37.37221111111111
              },
              "type": 4,
              "passShape": {
                "linestring": "126.823047,37.316833 126.824414,37.316767 126.826906,37.316628 126.830992,37.316458 126.832469,37.316378 126.833967,37.316292 126.835467,37.316217 126.836961,37.316139 126.838481,37.316061 126.839886,37.315986 126.842467,37.315819 126.844714,37.315669 126.845500,37.315581 126.846008,37.315494 126.846500,37.315406 126.846953,37.315281 126.847328,37.315158 126.847739,37.314989 126.848297,37.314700 126.848778,37.314403 126.849225,37.314086 126.849656,37.313689 126.850683,37.312733 126.851578,37.311858 126.852081,37.311344 126.852964,37.310425 126.853639,37.309722 126.853753,37.309606 126.854828,37.308542 126.855744,37.307639 126.856328,37.307072 126.857392,37.306008 126.857856,37.305514 126.858386,37.304958 126.858814,37.304581 126.859214,37.304283 126.859542,37.304086 126.859942,37.303889 126.860364,37.303700 126.860919,37.303478 126.861364,37.303333 126.861831,37.303228 126.862433,37.303131 126.863000,37.303081 126.863614,37.303056 126.864522,37.302989 126.864847,37.302964 126.866383,37.302867 126.867319,37.302806 126.871492,37.302481 126.872903,37.302453 126.873700,37.302464 126.874558,37.302494 126.875353,37.302522 126.878306,37.302639 126.879389,37.302683 126.879444,37.302686 126.880114,37.302728 126.881167,37.302778 126.881628,37.302781 126.882600,37.302847 126.883436,37.303019 126.884569,37.303317 126.885189,37.303486 126.885669,37.303642 126.886589,37.303892 126.888092,37.304297 126.888661,37.304450 126.890783,37.305022 126.892403,37.305458 126.892564,37.305503 126.892981,37.305614 126.894681,37.306069 126.895633,37.306331 126.896247,37.306492 126.896492,37.306558 126.896719,37.306622 126.896942,37.306689 126.897150,37.306756 126.897361,37.306831 126.897625,37.306928 126.897911,37.307047 126.898361,37.307239 126.898828,37.307475 126.899233,37.307703 126.899661,37.307931 126.899892,37.308075 126.900097,37.308219 126.900678,37.308725 126.900789,37.308831 126.901239,37.309314 126.901683,37.309800 126.902156,37.310414 126.902169,37.310428 126.903367,37.311867 126.904406,37.313108 126.905708,37.314681 126.907406,37.316717 126.908850,37.318425 126.910258,37.320100 126.911522,37.321614 126.912292,37.322533 126.912761,37.323092 126.913981,37.324544 126.914781,37.325519 126.915297,37.326094 126.915756,37.326639 126.916767,37.327828 126.916878,37.327964 126.917661,37.328922 126.917814,37.329117 126.918161,37.329506 126.918344,37.329744 126.918844,37.330286 126.919256,37.330789 126.919794,37.331383 126.920208,37.331872 126.920333,37.332044 126.920756,37.332558 126.921186,37.333181 126.921350,37.333408 126.921494,37.333658 126.921644,37.333936 126.921775,37.334258 126.922056,37.334978 126.922269,37.335875 126.922333,37.336169 126.922636,37.337656 126.923267,37.340964 126.923528,37.342833 126.923919,37.344722 126.924153,37.345831 126.925533,37.350419 126.925583,37.350578 126.925914,37.351614 126.926225,37.352608 126.926347,37.352931 126.926475,37.353253 126.926664,37.353675 126.926964,37.354200 126.927178,37.354453 126.927636,37.354939 126.928156,37.355353 126.930842,37.356931 126.932967,37.358056 126.935383,37.359333 126.936328,37.359833 126.939789,37.361636 126.942525,37.363175 126.944708,37.364322 126.944894,37.364464 126.945069,37.364628 126.945267,37.364825 126.945411,37.365006 126.945514,37.365156 126.945586,37.365269 126.945653,37.365397 126.945731,37.365625 126.945792,37.365825 126.945825,37.366022 126.945839,37.366267 126.945839,37.366431 126.945808,37.366608 126.945775,37.366781 126.945708,37.366978 126.945628,37.367172 126.945536,37.367378 126.945350,37.367761 126.945236,37.367994 126.945072,37.368339 126.945006,37.368506 126.943858,37.371331 126.943644,37.371906 126.943539,37.372194"
              }
            },
            {
              "mode": "WALK",
              "sectionTime": 136,
              "distance": 151,
              "start": {
                "name": "금정",
                "lon": 126.94360833333333,
                "lat": 37.37221111111111
              },
              "end": {
                "name": "금정",
                "lon": 126.94342222222222,
                "lat": 37.37238611111111
              },
              "passShape": {
                "linestring": "126.943608,37.372211 126.943814,37.371767 126.944025,37.371833 126.943922,37.372047 126.943947,37.372067 126.943844,37.372283 126.943422,37.372386"
              }
            },
            {
              "routeColor": "0052A4",
              "distance": 34712,
              "start": {
                "name": "금정",
                "lon": 126.94342222222222,
                "lat": 37.37238611111111
              },
              "Lane": [
                {
                  "routeColor": "0052A4",
                  "route": "수도권1호선",
                  "routeId": "110014012",
                  "service": 1,
                  "type": 1
                }
              ],
              "type": 117,
              "mode": "SUBWAY",
              "sectionTime": 3482,
              "route": "수도권1호선(급행)",
              "routeId": "111013004",
              "service": 1,
              "passStopList": {
                "stationList": [
                  {
                    "index": 0,
                    "stationName": "금정",
                    "lon": "126.943422",
                    "lat": "37.372386",
                    "stationID": "110170"
                  },
                  {
                    "index": 1,
                    "stationName": "명학",
                    "lon": "126.935531",
                    "lat": "37.384597",
                    "stationID": "110169"
                  },
                  {
                    "index": 2,
                    "stationName": "안양",
                    "lon": "126.922828",
                    "lat": "37.401717",
                    "stationID": "110168"
                  },
                  {
                    "index": 3,
                    "stationName": "관악",
                    "lon": "126.908533",
                    "lat": "37.419542",
                    "stationID": "110167"
                  },
                  {
                    "index": 4,
                    "stationName": "석수",
                    "lon": "126.902353",
                    "lat": "37.434136",
                    "stationID": "110166"
                  },
                  {
                    "index": 5,
                    "stationName": "금천구청",
                    "lon": "126.893942",
                    "lat": "37.455681",
                    "stationID": "110165"
                  },
                  {
                    "index": 6,
                    "stationName": "독산",
                    "lon": "126.889394",
                    "lat": "37.466203",
                    "stationID": "110164"
                  },
                  {
                    "index": 7,
                    "stationName": "가산디지털단지",
                    "lon": "126.882661",
                    "lat": "37.481408",
                    "stationID": "110163"
                  },
                  {
                    "index": 8,
                    "stationName": "구로",
                    "lon": "126.881619",
                    "lat": "37.502419",
                    "stationID": "110142"
                  },
                  {
                    "index": 9,
                    "stationName": "신도림",
                    "lon": "126.891569",
                    "lat": "37.509014",
                    "stationID": "110141"
                  },
                  {
                    "index": 10,
                    "stationName": "영등포",
                    "lon": "126.907289",
                    "lat": "37.515689",
                    "stationID": "110140"
                  },
                  {
                    "index": 11,
                    "stationName": "신길",
                    "lon": "126.917708",
                    "lat": "37.517036",
                    "stationID": "110139"
                  },
                  {
                    "index": 12,
                    "stationName": "대방",
                    "lon": "126.926450",
                    "lat": "37.513408",
                    "stationID": "110138"
                  },
                  {
                    "index": 13,
                    "stationName": "노량진",
                    "lon": "126.941633",
                    "lat": "37.513950",
                    "stationID": "110137"
                  },
                  {
                    "index": 14,
                    "stationName": "용산",
                    "lon": "126.964481",
                    "lat": "37.529669",
                    "stationID": "110136"
                  },
                  {
                    "index": 15,
                    "stationName": "남영",
                    "lon": "126.971181",
                    "lat": "37.540525",
                    "stationID": "110135"
                  },
                  {
                    "index": 16,
                    "stationName": "서울역",
                    "lon": "126.972275",
                    "lat": "37.555956",
                    "stationID": "110134"
                  },
                  {
                    "index": 17,
                    "stationName": "시청",
                    "lon": "126.977125",
                    "lat": "37.565539",
                    "stationID": "110133"
                  },
                  {
                    "index": 18,
                    "stationName": "종각",
                    "lon": "126.983194",
                    "lat": "37.570183",
                    "stationID": "110132"
                  },
                  {
                    "index": 19,
                    "stationName": "종로3가",
                    "lon": "126.992206",
                    "lat": "37.570436",
                    "stationID": "110131"
                  },
                  {
                    "index": 20,
                    "stationName": "종로5가",
                    "lon": "127.001942",
                    "lat": "37.571008",
                    "stationID": "110130"
                  },
                  {
                    "index": 21,
                    "stationName": "동대문",
                    "lon": "127.010639",
                    "lat": "37.571675",
                    "stationID": "110129"
                  },
                  {
                    "index": 22,
                    "stationName": "동묘앞",
                    "lon": "127.016594",
                    "lat": "37.573267",
                    "stationID": "110128"
                  },
                  {
                    "index": 23,
                    "stationName": "신설동",
                    "lon": "127.024111",
                    "lat": "37.575894",
                    "stationID": "110127"
                  },
                  {
                    "index": 24,
                    "stationName": "제기동",
                    "lon": "127.034792",
                    "lat": "37.578169",
                    "stationID": "110126"
                  },
                  {
                    "index": 25,
                    "stationName": "청량리",
                    "lon": "127.044556",
                    "lat": "37.579967",
                    "stationID": "110125"
                  }
                ]
              },
              "end": {
                "name": "청량리",
                "lon": 127.04455555555556,
                "lat": 37.579966666666664
              },
              "passShape": {
                "linestring": "126.943644,37.371906 126.943092,37.373092 126.942500,37.374344 126.942392,37.374561 126.941839,37.375678 126.941683,37.375950 126.941164,37.376781 126.940897,37.377225 126.940731,37.377439 126.940478,37.377742 126.940067,37.378186 126.937881,37.380233 126.937703,37.380419 126.937228,37.380986 126.936914,37.381392 126.936667,37.381781 126.936514,37.382047 126.936303,37.382439 126.935936,37.383467 126.935731,37.384058 126.935531,37.384597 126.935189,37.385517 126.935147,37.385628 126.934942,37.386119 126.934769,37.386611 126.934586,37.387103 126.934383,37.387589 126.934203,37.388081 126.934022,37.388578 126.933836,37.389069 126.933653,37.389569 126.933458,37.390097 126.933256,37.390619 126.933069,37.391144 126.932908,37.391503 126.932825,37.391661 126.932714,37.391836 126.932592,37.392022 126.932464,37.392189 126.932347,37.392331 126.932197,37.392511 126.931939,37.392778 126.931706,37.392983 126.931481,37.393178 126.931275,37.393350 126.931142,37.393450 126.930572,37.393903 126.930222,37.394178 126.929906,37.394431 126.929569,37.394692 126.929206,37.394983 126.928894,37.395225 126.928619,37.395444 126.928203,37.395775 126.927922,37.395989 126.927258,37.396569 126.926772,37.396992 126.926381,37.397342 126.925992,37.397758 126.925636,37.398175 126.925083,37.398861 126.924533,37.399544 126.923992,37.400292 126.923625,37.400756 126.923517,37.400878 126.923031,37.401469 126.922725,37.401842 126.921953,37.402806 126.921464,37.403442 126.920797,37.404311 126.919831,37.405564 126.917611,37.408150 126.914039,37.412678 126.913650,37.413172 126.911069,37.416419 126.909853,37.417931 126.909481,37.418369 126.909211,37.418689 126.908797,37.419211 126.908597,37.419461 126.908531,37.419542 126.908350,37.419764 126.907961,37.420244 126.907725,37.420553 126.907489,37.420856 126.907203,37.421256 126.907081,37.421433 126.906758,37.421981 126.906519,37.422422 126.906219,37.422964 126.905989,37.423378 126.905767,37.423764 126.905531,37.424175 126.905336,37.424503 126.904828,37.425400 126.904431,37.426133 126.904233,37.426544 126.903864,37.427414 126.903722,37.427814 126.903644,37.428106 126.903436,37.429131 126.902606,37.433319 126.902453,37.434147 126.902411,37.434372 126.902075,37.436031 126.901989,37.436458 126.901739,37.437611 126.901694,37.437806 126.901642,37.438000 126.901589,37.438192 126.901558,37.438289 126.901500,37.438478 126.901467,37.438575 126.901400,37.438764 126.901328,37.438953 126.901250,37.439142 126.901169,37.439328 126.901083,37.439514 126.900697,37.440375 126.898817,37.444614 126.898364,37.445639 126.894867,37.453564 126.894169,37.455281 126.893989,37.455692 126.893125,37.457642 126.890858,37.462756 126.889358,37.466192 126.885583,37.474828 126.884383,37.477619 126.884267,37.477889 126.883067,37.480519 126.882661,37.481408 126.882522,37.481711 126.881003,37.485200 126.880572,37.486014 126.880525,37.486114 126.880475,37.486214 126.880431,37.486317 126.880389,37.486417 126.880347,37.486519 126.880308,37.486622 126.880272,37.486725 126.880236,37.486831 126.880206,37.486933 126.880078,37.487247 126.880025,37.487422 126.879989,37.487603 126.879408,37.495628 126.879394,37.495808 126.879372,37.495919 126.879292,37.496178 126.878497,37.498450 126.878472,37.498514 126.878453,37.498578 126.878436,37.498642 126.878422,37.498706 126.878414,37.498769 126.878408,37.498836 126.878406,37.498900 126.878408,37.498967 126.878417,37.499031 126.878428,37.499097 126.878442,37.499161 126.878458,37.499225 126.878481,37.499289 126.878506,37.499350 126.878533,37.499411 126.878567,37.499469 126.878603,37.499531 126.878644,37.499586 126.878686,37.499642 126.878733,37.499697 126.878781,37.499750 126.879811,37.500717 126.881608,37.502408 126.881617,37.502417 126.881619,37.502419 126.881867,37.502650 126.882189,37.502953 126.883911,37.504564 126.884392,37.504842 126.884906,37.505142 126.889300,37.507842 126.889489,37.507956 126.889781,37.508119 126.890075,37.508283 126.890375,37.508439 126.890575,37.508542 126.890775,37.508644 126.890981,37.508742 126.891111,37.508806 126.891183,37.508839 126.891197,37.508844 126.891392,37.508936 126.892072,37.509233 126.900297,37.512678 126.905769,37.515103 126.907467,37.515758 126.907483,37.515764 126.908981,37.516344 126.909103,37.516389 126.909225,37.516431 126.909350,37.516469 126.909475,37.516508 126.909600,37.516544 126.909725,37.516581 126.909853,37.516614 126.909981,37.516644 126.910108,37.516675 126.910239,37.516706 126.910367,37.516731 126.910497,37.516756 126.910628,37.516778 126.910758,37.516800 126.910889,37.516819 126.911022,37.516836 126.911153,37.516853 126.911286,37.516867 126.911419,37.516878 126.911553,37.516889 126.915689,37.517158 126.916231,37.517178 126.916467,37.517178 126.916700,37.517172 126.916933,37.517158 126.917167,37.517139 126.917397,37.517111 126.917628,37.517078 126.917717,37.517061 126.917856,37.517036 126.918081,37.516989 126.918306,37.516936 126.918525,37.516875 126.918744,37.516806 126.918958,37.516733 126.919169,37.516653 126.919378,37.516567 126.919581,37.516475 126.919778,37.516375 126.919972,37.516269 126.920161,37.516161 126.920344,37.516044 126.920522,37.515925 126.921106,37.515472 126.922514,37.514372 126.922619,37.514292 126.922731,37.514214 126.922844,37.514139 126.922961,37.514067 126.923081,37.514000 126.923206,37.513933 126.923331,37.513872 126.923458,37.513814 126.923589,37.513758 126.923722,37.513708 126.923858,37.513661 126.923994,37.513619 126.924133,37.513581 126.924275,37.513544 126.924417,37.513514 126.924561,37.513486 126.924706,37.513461 126.924853,37.513442 126.924997,37.513428 126.925144,37.513417 126.925617,37.513392 126.926450,37.513400 126.926794,37.513406 126.927922,37.513467 126.933408,37.513575 126.934053,37.513589 126.934225,37.513594 126.939428,37.513811 126.940331,37.513850 126.941300,37.513917 126.941633,37.513950 126.943844,37.514178 126.944692,37.514289 126.946094,37.514508 126.946214,37.514531 126.946328,37.514553 126.946442,37.514578 126.946556,37.514606 126.946669,37.514639 126.946781,37.514672 126.946889,37.514708 126.946997,37.514750 126.947106,37.514792 126.947208,37.514836 126.947314,37.514883 126.947414,37.514933 126.947514,37.514989 126.947611,37.515042 126.947706,37.515100 126.947800,37.515158 126.947889,37.515222 126.947978,37.515286 126.948061,37.515353 126.948144,37.515419 126.948225,37.515492 126.952994,37.519900 126.953592,37.520486 126.954914,37.522044 126.957992,37.524903 126.958469,37.525317 126.958808,37.525586 126.959608,37.526178 126.960453,37.526767 126.964453,37.529697 126.964458,37.529700 126.969106,37.533106 126.969267,37.533222 126.969431,37.533378 126.969592,37.533539 126.969742,37.533703 126.969889,37.533869 126.970025,37.534042 126.970156,37.534217 126.970278,37.534397 126.970394,37.534578 126.970503,37.534764 126.970603,37.534953 126.970694,37.535142 126.970778,37.535333 126.970853,37.535528 126.970919,37.535725 126.970981,37.535925 126.971031,37.536125 126.971075,37.536325 126.971108,37.536528 126.971114,37.536578 126.971133,37.536731 126.971150,37.536933 126.971158,37.537136 126.971186,37.540525 126.971214,37.543814 126.971228,37.545825 126.971233,37.546511 126.971289,37.548153 126.971331,37.549453 126.971342,37.549681 126.971358,37.549908 126.971383,37.550133 126.971417,37.550361 126.971456,37.550589 126.971486,37.550739 126.971519,37.550889 126.971733,37.551761 126.971753,37.551842 126.971786,37.552008 126.971833,37.552256 126.971864,37.552419 126.971900,37.552669 126.971925,37.552836 126.971953,37.553086 126.971964,37.553192 126.971969,37.553253 126.971992,37.553503 126.972125,37.554750 126.972133,37.554817 126.972136,37.554850 126.972333,37.556442 126.972344,37.556494 126.972369,37.556594 126.972403,37.556694 126.972442,37.556794 126.972489,37.556892 126.972539,37.556989 126.972594,37.557083 126.972656,37.557175 126.972725,37.557264 126.972797,37.557350 126.972875,37.557433 126.974806,37.559606 126.974925,37.559733 126.974981,37.559800 126.975092,37.559933 126.975197,37.560069 126.975247,37.560139 126.975344,37.560281 126.975436,37.560422 126.975481,37.560494 126.975564,37.560639 126.975642,37.560786 126.975714,37.560936 126.975781,37.561086 126.976911,37.564133 126.976972,37.564317 126.977078,37.564619 126.977128,37.564908 126.977150,37.565297 126.977144,37.565539 126.977119,37.566797 126.977000,37.569767 126.977003,37.569797 126.977008,37.569831 126.977017,37.569861 126.977031,37.569892 126.977044,37.569922 126.977064,37.569953 126.977083,37.569981 126.977106,37.570008 126.977131,37.570033 126.977156,37.570058 126.977186,37.570081 126.977217,37.570103 126.977250,37.570122 126.977283,37.570139 126.977319,37.570156 126.977358,37.570169 126.977397,37.570181 126.977436,37.570189 126.977475,37.570194 126.977517,37.570200 126.977556,37.570203 126.982975,37.570175 126.984011,37.570178 126.987717,37.570192 126.990142,37.570294 126.990417,37.570314 126.992147,37.570433 126.992167,37.570436 126.992206,37.570436 126.994897,37.570522 126.997125,37.570689 126.997889,37.570747 127.000419,37.570917 127.001083,37.570958 127.001483,37.570983 127.002136,37.571003 127.002889,37.571017 127.007889,37.571125 127.007994,37.571128 127.008100,37.571133 127.008206,37.571142 127.008308,37.571156 127.008414,37.571169 127.008517,37.571189 127.008619,37.571208 127.008722,37.571231 127.008872,37.571269 127.008992,37.571308 127.009072,37.571331 127.009225,37.571375 127.009378,37.571417 127.009533,37.571453 127.009689,37.571486 127.009794,37.571508 127.009900,37.571528 127.010056,37.571553 127.010636,37.571681 127.012117,37.572006 127.012317,37.572050 127.012464,37.572089 127.012611,37.572125 127.012806,37.572181 127.012953,37.572225 127.013144,37.572283 127.015642,37.572981 127.016603,37.573250 127.017108,37.573392 127.017500,37.573503 127.017731,37.573572 127.017964,37.573644 127.018194,37.573719 127.018425,37.573794 127.020608,37.574539 127.021017,37.574692 127.023233,37.575572 127.023411,37.575644 127.023575,37.575708 127.024111,37.575894 127.024403,37.575994 127.025333,37.576289 127.027267,37.576822 127.030839,37.577864 127.030981,37.577908 127.031122,37.577947 127.031194,37.577967 127.031339,37.578000 127.031411,37.578017 127.031556,37.578047 127.031706,37.578072 127.031853,37.578094 127.032003,37.578114 127.032153,37.578128 127.032228,37.578133 127.032378,37.578142 127.034792,37.578169 127.036183,37.578183 127.038053,37.578169 127.038119,37.578169 127.038253,37.578175 127.038386,37.578181 127.038519,37.578192 127.038653,37.578206 127.038783,37.578225 127.038850,37.578236 127.038978,37.578258 127.039108,37.578289 127.039236,37.578319 127.039300,37.578336 127.039425,37.578372 127.044556,37.579967"
              }
            },
            {
              "mode": "WALK",
              "sectionTime": 630,
              "distance": 692,
              "start": {
                "name": "청량리",
                "lon": 127.04455555555556,
                "lat": 37.579966666666664
              },
              "end": {
                "name": "도착지",
                "lon": 127.048958,
                "lat": 37.581381
              },
              "steps": [
                {
                  "streetName": "",
                  "distance": 128,
                  "description": "128m 이동",
                  "linestring": "127.044556,37.579964 127.044846,37.580055 127.04525,37.580185 127.045395,37.580257 127.045845,37.58048"
                },
                {
                  "streetName": "",
                  "distance": 42,
                  "description": "청량리역  4번출구 에서 우회전 후 42m 이동 ",
                  "linestring": "127.045845,37.58048 127.04597,37.580322 127.046165,37.580433"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 6,
                  "description": "청량리역  4번출구 에서 좌회전 후 보행자도로 을 따라 6m 이동 ",
                  "linestring": "127.046165,37.580433 127.046104,37.58046"
                },
                {
                  "streetName": "왕산로",
                  "distance": 87,
                  "description": "청량리역  4번출구 에서 우회전 후 왕산로 을 따라 87m 이동 ",
                  "linestring": "127.046104,37.58046 127.04615,37.580517 127.046196,37.580578 127.046295,37.58071 127.04644,37.580803 127.04654,37.58086 127.04664,37.58092 127.04674,37.580936 127.046814,37.580925"
                },
                {
                  "streetName": "",
                  "distance": 24,
                  "description": "KFC 청량리점 에서 좌측 횡단보도 후 24m 이동 ",
                  "linestring": "127.046814,37.580925 127.04683,37.580956 127.04684,37.580994 127.04685,37.58104 127.04687,37.581104 127.046875,37.58114"
                },
                {
                  "streetName": "",
                  "distance": 17,
                  "description": "직진 후 17m 이동 ",
                  "linestring": "127.046875,37.58114 127.04689,37.58125 127.046906,37.581284"
                },
                {
                  "streetName": "",
                  "distance": 14,
                  "description": "횡단보도 후 14m 이동 ",
                  "linestring": "127.046906,37.581284 127.04703,37.581375"
                },
                {
                  "streetName": "",
                  "distance": 304,
                  "description": "훼미리마트 청량리역점 에서 우회전 후 304m 이동 ",
                  "linestring": "127.04703,37.581375 127.04703,37.581326 127.04703,37.581295 127.04706,37.581257 127.0471,37.581226 127.047165,37.581207 127.047226,37.581196 127.04729,37.581196 127.047356,37.581207 127.047424,37.581226 127.04754,37.581272 127.047646,37.581345 127.04775,37.581417 127.04815,37.581753 127.04833,37.581894 127.04845,37.581947 127.04858,37.581978 127.04877,37.581997 127.04888,37.58199 127.04908,37.581936 127.04913,37.58191 127.04942,37.58177 127.04982,37.581577"
                },
                {
                  "streetName": "",
                  "distance": 17,
                  "description": "우측 횡단보도 후 17m 이동 ",
                  "linestring": "127.04982,37.581577 127.04971,37.58145"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 53,
                  "description": "우회전 후 보행자도로 을 따라 53m 이동 ",
                  "linestring": "127.04971,37.58145 127.04921,37.581703"
                }
              ]
            }
          ],
          "totalWalkTime": 867,
          "transferCount": 1,
          "totalDistance": 50627,
          "pathType": 1,
          "totalWalkDistance": 964
        },
        {
          "fare": {
            "regular": {
              "totalFare": 2200,
              "currency": {
                "symbol": "￦",
                "currency": "원",
                "currencyCode": "KRW"
              }
            }
          },
          "totalTime": 5785,
          "legs": [
            {
              "mode": "WALK",
              "sectionTime": 101,
              "distance": 121,
              "start": {
                "name": "출발지",
                "lon": 126.8230369,
                "lat": 37.3168267
              },
              "end": {
                "name": "고잔",
                "lon": 126.82304722222223,
                "lat": 37.316833333333335
              },
              "steps": [
                {
                  "streetName": "",
                  "distance": 27,
                  "description": "27m 이동",
                  "linestring": "126.82303,37.3168 126.82306,37.3168 126.82333,37.316788"
                },
                {
                  "streetName": "",
                  "distance": 15,
                  "description": "고잔역 에서 좌회전 후 15m 이동 ",
                  "linestring": "126.82333,37.316788 126.82334,37.31693"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 20,
                  "description": "직진 후 보행자도로 을 따라 20m 이동 ",
                  "linestring": "126.82334,37.31693 126.82336,37.317104"
                },
                {
                  "streetName": "중앙대로",
                  "distance": 39,
                  "description": "좌회전 후 중앙대로 을 따라 39m 이동 ",
                  "linestring": "126.82336,37.317104 126.822914,37.317135"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 20,
                  "description": "좌회전 후 보행자도로 을 따라 20m 이동 ",
                  "linestring": "126.822914,37.317135 126.82289,37.316963"
                }
              ]
            },
            {
              "mode": "SUBWAY",
              "routeColor": "00A5DE",
              "sectionTime": 4381,
              "route": "수도권4호선",
              "routeId": "110041005",
              "distance": 43266,
              "service": 1,
              "start": {
                "name": "고잔",
                "lon": 126.82304722222223,
                "lat": 37.316833333333335
              },
              "passStopList": {
                "stationList": [
                  {
                    "index": 0,
                    "stationName": "고잔",
                    "lon": "126.823047",
                    "lat": "37.316833",
                    "stationID": "110446"
                  },
                  {
                    "index": 1,
                    "stationName": "중앙",
                    "lon": "126.838481",
                    "lat": "37.316058",
                    "stationID": "110445"
                  },
                  {
                    "index": 2,
                    "stationName": "한대앞",
                    "lon": "126.853636",
                    "lat": "37.309722",
                    "stationID": "110444"
                  },
                  {
                    "index": 3,
                    "stationName": "상록수",
                    "lon": "126.866383",
                    "lat": "37.302869",
                    "stationID": "110443"
                  },
                  {
                    "index": 4,
                    "stationName": "반월",
                    "lon": "126.903367",
                    "lat": "37.311867",
                    "stationID": "110442"
                  },
                  {
                    "index": 5,
                    "stationName": "대야미",
                    "lon": "126.916878",
                    "lat": "37.327964",
                    "stationID": "110441"
                  },
                  {
                    "index": 6,
                    "stationName": "수리산",
                    "lon": "126.925617",
                    "lat": "37.350569",
                    "stationID": "110440"
                  },
                  {
                    "index": 7,
                    "stationName": "산본",
                    "lon": "126.933019",
                    "lat": "37.357989",
                    "stationID": "110439"
                  },
                  {
                    "index": 8,
                    "stationName": "금정",
                    "lon": "126.943608",
                    "lat": "37.372211",
                    "stationID": "110438"
                  },
                  {
                    "index": 9,
                    "stationName": "범계",
                    "lon": "126.950675",
                    "lat": "37.389764",
                    "stationID": "110437"
                  },
                  {
                    "index": 10,
                    "stationName": "평촌",
                    "lon": "126.964106",
                    "lat": "37.394431",
                    "stationID": "110436"
                  },
                  {
                    "index": 11,
                    "stationName": "인덕원",
                    "lon": "126.976800",
                    "lat": "37.401900",
                    "stationID": "110435"
                  },
                  {
                    "index": 12,
                    "stationName": "정부과천청사",
                    "lon": "126.989942",
                    "lat": "37.426792",
                    "stationID": "110434"
                  },
                  {
                    "index": 13,
                    "stationName": "과천",
                    "lon": "126.996669",
                    "lat": "37.433281",
                    "stationID": "110433"
                  },
                  {
                    "index": 14,
                    "stationName": "대공원",
                    "lon": "127.006531",
                    "lat": "37.435831",
                    "stationID": "110432"
                  },
                  {
                    "index": 15,
                    "stationName": "경마공원",
                    "lon": "127.007944",
                    "lat": "37.443806",
                    "stationID": "110431"
                  },
                  {
                    "index": 16,
                    "stationName": "선바위",
                    "lon": "127.001972",
                    "lat": "37.452111",
                    "stationID": "110430"
                  },
                  {
                    "index": 17,
                    "stationName": "남태령",
                    "lon": "126.988883",
                    "lat": "37.464647",
                    "stationID": "110429"
                  },
                  {
                    "index": 18,
                    "stationName": "사당",
                    "lon": "126.981667",
                    "lat": "37.476808",
                    "stationID": "110428"
                  },
                  {
                    "index": 19,
                    "stationName": "총신대입구",
                    "lon": "126.982264",
                    "lat": "37.487347",
                    "stationID": "110427"
                  },
                  {
                    "index": 20,
                    "stationName": "동작",
                    "lon": "126.980344",
                    "lat": "37.502897",
                    "stationID": "110426"
                  },
                  {
                    "index": 21,
                    "stationName": "이촌",
                    "lon": "126.974444",
                    "lat": "37.522350",
                    "stationID": "110425"
                  },
                  {
                    "index": 22,
                    "stationName": "신용산",
                    "lon": "126.967928",
                    "lat": "37.529222",
                    "stationID": "110424"
                  },
                  {
                    "index": 23,
                    "stationName": "삼각지",
                    "lon": "126.972633",
                    "lat": "37.534133",
                    "stationID": "110423"
                  },
                  {
                    "index": 24,
                    "stationName": "숙대입구",
                    "lon": "126.972111",
                    "lat": "37.544600",
                    "stationID": "110422"
                  },
                  {
                    "index": 25,
                    "stationName": "서울역",
                    "lon": "126.972797",
                    "lat": "37.553497",
                    "stationID": "110421"
                  },
                  {
                    "index": 26,
                    "stationName": "회현",
                    "lon": "126.978564",
                    "lat": "37.558886",
                    "stationID": "110420"
                  },
                  {
                    "index": 27,
                    "stationName": "명동",
                    "lon": "126.986489",
                    "lat": "37.560981",
                    "stationID": "110419"
                  },
                  {
                    "index": 28,
                    "stationName": "충무로",
                    "lon": "126.994172",
                    "lat": "37.561256",
                    "stationID": "110418"
                  },
                  {
                    "index": 29,
                    "stationName": "동대문역사문화공원",
                    "lon": "127.007356",
                    "lat": "37.564644",
                    "stationID": "110417"
                  },
                  {
                    "index": 30,
                    "stationName": "동대문",
                    "lon": "127.009161",
                    "lat": "37.571050",
                    "stationID": "110416"
                  }
                ]
              },
              "end": {
                "name": "동대문",
                "lon": 127.00916111111111,
                "lat": 37.57105
              },
              "type": 4,
              "passShape": {
                "linestring": "126.823047,37.316833 126.824414,37.316767 126.826906,37.316628 126.830992,37.316458 126.832469,37.316378 126.833967,37.316292 126.835467,37.316217 126.836961,37.316139 126.838481,37.316061 126.839886,37.315986 126.842467,37.315819 126.844714,37.315669 126.845500,37.315581 126.846008,37.315494 126.846500,37.315406 126.846953,37.315281 126.847328,37.315158 126.847739,37.314989 126.848297,37.314700 126.848778,37.314403 126.849225,37.314086 126.849656,37.313689 126.850683,37.312733 126.851578,37.311858 126.852081,37.311344 126.852964,37.310425 126.853639,37.309722 126.853753,37.309606 126.854828,37.308542 126.855744,37.307639 126.856328,37.307072 126.857392,37.306008 126.857856,37.305514 126.858386,37.304958 126.858814,37.304581 126.859214,37.304283 126.859542,37.304086 126.859942,37.303889 126.860364,37.303700 126.860919,37.303478 126.861364,37.303333 126.861831,37.303228 126.862433,37.303131 126.863000,37.303081 126.863614,37.303056 126.864522,37.302989 126.864847,37.302964 126.866383,37.302867 126.867319,37.302806 126.871492,37.302481 126.872903,37.302453 126.873700,37.302464 126.874558,37.302494 126.875353,37.302522 126.878306,37.302639 126.879389,37.302683 126.879444,37.302686 126.880114,37.302728 126.881167,37.302778 126.881628,37.302781 126.882600,37.302847 126.883436,37.303019 126.884569,37.303317 126.885189,37.303486 126.885669,37.303642 126.886589,37.303892 126.888092,37.304297 126.888661,37.304450 126.890783,37.305022 126.892403,37.305458 126.892564,37.305503 126.892981,37.305614 126.894681,37.306069 126.895633,37.306331 126.896247,37.306492 126.896492,37.306558 126.896719,37.306622 126.896942,37.306689 126.897150,37.306756 126.897361,37.306831 126.897625,37.306928 126.897911,37.307047 126.898361,37.307239 126.898828,37.307475 126.899233,37.307703 126.899661,37.307931 126.899892,37.308075 126.900097,37.308219 126.900678,37.308725 126.900789,37.308831 126.901239,37.309314 126.901683,37.309800 126.902156,37.310414 126.902169,37.310428 126.903367,37.311867 126.904406,37.313108 126.905708,37.314681 126.907406,37.316717 126.908850,37.318425 126.910258,37.320100 126.911522,37.321614 126.912292,37.322533 126.912761,37.323092 126.913981,37.324544 126.914781,37.325519 126.915297,37.326094 126.915756,37.326639 126.916767,37.327828 126.916878,37.327964 126.917661,37.328922 126.917814,37.329117 126.918161,37.329506 126.918344,37.329744 126.918844,37.330286 126.919256,37.330789 126.919794,37.331383 126.920208,37.331872 126.920333,37.332044 126.920756,37.332558 126.921186,37.333181 126.921350,37.333408 126.921494,37.333658 126.921644,37.333936 126.921775,37.334258 126.922056,37.334978 126.922269,37.335875 126.922333,37.336169 126.922636,37.337656 126.923267,37.340964 126.923528,37.342833 126.923919,37.344722 126.924153,37.345831 126.925533,37.350419 126.925583,37.350578 126.925914,37.351614 126.926225,37.352608 126.926347,37.352931 126.926475,37.353253 126.926664,37.353675 126.926964,37.354200 126.927178,37.354453 126.927636,37.354939 126.928156,37.355353 126.930842,37.356931 126.932967,37.358056 126.935383,37.359333 126.936328,37.359833 126.939789,37.361636 126.942525,37.363175 126.944708,37.364322 126.944894,37.364464 126.945069,37.364628 126.945267,37.364825 126.945411,37.365006 126.945514,37.365156 126.945586,37.365269 126.945653,37.365397 126.945731,37.365625 126.945792,37.365825 126.945825,37.366022 126.945839,37.366267 126.945839,37.366431 126.945808,37.366608 126.945775,37.366781 126.945708,37.366978 126.945628,37.367172 126.945536,37.367378 126.945350,37.367761 126.945236,37.367994 126.945072,37.368339 126.945006,37.368506 126.943858,37.371331 126.943644,37.371906 126.943539,37.372194 126.943325,37.372775 126.942281,37.374958 126.941906,37.375700 126.941767,37.375947 126.941228,37.376814 126.940958,37.377247 126.940797,37.377472 126.940664,37.377628 126.940542,37.377758 126.940036,37.378289 126.939917,37.378406 126.938700,37.379531 126.938669,37.379575 126.938653,37.379619 126.938642,37.379664 126.938633,37.379706 126.938636,37.379767 126.938644,37.379828 126.938872,37.380197 126.939181,37.380786 126.939919,37.382556 126.940189,37.383083 126.940436,37.383478 126.940786,37.383844 126.941064,37.384042 126.941464,37.384264 126.941989,37.384469 126.942383,37.384636 126.942733,37.384825 126.942994,37.384981 126.943428,37.385258 126.943864,37.385597 126.945186,37.386678 126.947628,37.388694 126.947642,37.388703 126.947653,37.388714 126.947672,37.388722 126.947686,37.388728 126.947700,37.388733 126.950675,37.389767 126.962272,37.393797 126.964086,37.394467 126.964389,37.394578 126.968514,37.395942 126.971767,37.397033 126.973083,37.397500 126.974419,37.397997 126.975417,37.398531 126.975703,37.398761 126.975883,37.398925 126.976144,37.399208 126.976603,37.399822 126.976797,37.400136 126.976881,37.400292 126.976914,37.400467 126.976833,37.401311 126.976833,37.401350 126.976783,37.402194 126.976803,37.402544 126.976883,37.402842 126.977764,37.404747 126.977808,37.404872 126.978228,37.406642 126.978494,37.407764 126.979033,37.410022 126.979122,37.410444 126.979419,37.411742 126.979725,37.412986 126.979781,37.413208 126.979867,37.413675 126.979908,37.413847 126.980011,37.414200 126.980928,37.416297 126.981547,37.417675 126.981750,37.418144 126.982047,37.418675 126.982469,37.419406 126.982989,37.420228 126.983950,37.421275 126.984853,37.422069 126.985869,37.422925 126.987003,37.423869 126.987781,37.424625 126.988981,37.425722 126.989956,37.426783 126.990411,37.427278 126.991381,37.428233 126.992906,37.429739 126.995314,37.432433 126.995839,37.432964 126.996069,37.433125 126.996378,37.433244 126.996656,37.433317 126.996756,37.433342 126.997372,37.433425 126.997742,37.433425 126.998319,37.433353 126.999225,37.433258 126.999853,37.433225 127.000758,37.433239 127.004331,37.433214 127.004481,37.433256 127.005314,37.433958 127.005433,37.434131 127.006244,37.435392 127.006450,37.435711 127.008228,37.438322 127.008297,37.438447 127.008344,37.438575 127.008364,37.438711 127.008375,37.438969 127.008203,37.440986 127.008019,37.442944 127.007969,37.443506 127.007944,37.443831 127.007819,37.444217 127.007717,37.444408 127.007436,37.444817 127.006928,37.445475 127.004367,37.448900 127.002544,37.451400 127.001972,37.452111 127.001836,37.452281 127.001633,37.452472 127.001442,37.452575 127.000942,37.452764 126.999947,37.453253 126.998028,37.454378 126.997922,37.454442 126.996819,37.455239 126.995569,37.456381 126.994317,37.457522 126.993775,37.458058 126.993728,37.458117 126.993250,37.458667 126.991156,37.461425 126.989422,37.463706 126.989111,37.464247 126.988883,37.464647 126.987964,37.466256 126.987881,37.466394 126.987797,37.466528 126.987708,37.466664 126.987617,37.466794 126.987525,37.466928 126.987428,37.467058 126.987328,37.467186 126.987225,37.467314 126.987122,37.467442 126.987014,37.467567 126.986903,37.467689 126.986792,37.467811 126.986675,37.467931 126.986558,37.468050 126.986439,37.468167 126.986314,37.468283 126.986189,37.468394 126.986061,37.468508 126.985931,37.468619 126.985800,37.468728 126.985664,37.468833 126.985211,37.469178 126.984864,37.469464 126.984636,37.469658 126.984472,37.469808 126.984356,37.469928 126.983858,37.470544 126.983769,37.470672 126.983256,37.471453 126.982953,37.472042 126.982397,37.473183 126.982053,37.474025 126.981878,37.474547 126.981769,37.474950 126.981675,37.475608 126.981669,37.475658 126.981633,37.476083 126.981656,37.476564 126.981658,37.476653 126.982111,37.485253 126.982117,37.485383 126.982122,37.485475 126.982158,37.486308 126.982228,37.487350 126.982233,37.487400 126.982378,37.488686 126.982531,37.489653 126.982592,37.490369 126.982647,37.494967 126.982628,37.496269 126.982639,37.496469 126.982664,37.496669 126.982692,37.496817 126.982744,37.497011 126.982794,37.497156 126.982983,37.497625 126.983100,37.497950 126.983239,37.498383 126.983267,37.498531 126.983264,37.498681 126.983233,37.498825 126.983178,37.498967 126.983067,37.499131 126.981803,37.500881 126.981647,37.501097 126.981553,37.501222 126.981425,37.501369 126.980922,37.501886 126.980856,37.501964 126.980794,37.502033 126.980644,37.502233 126.980547,37.502394 126.980464,37.502575 126.980381,37.502808 126.980294,37.503031 126.980269,37.503125 126.980183,37.503464 126.980156,37.503878 126.980156,37.504058 126.980164,37.504244 126.980186,37.504425 126.980258,37.504800 126.980719,37.506508 126.983114,37.515581 126.983161,37.515808 126.983203,37.516089 126.983231,37.516306 126.983269,37.517844 126.983278,37.518414 126.983261,37.518581 126.983225,37.518736 126.983183,37.518883 126.983128,37.519019 126.983053,37.519164 126.982961,37.519292 126.982853,37.519408 126.982725,37.519531 126.982633,37.519608 126.982606,37.519633 126.982456,37.519739 126.982292,37.519836 126.982108,37.519936 126.981919,37.520033 126.981719,37.520119 126.981503,37.520206 126.976931,37.521594 126.975139,37.522128 126.974444,37.522344 126.972900,37.522833 126.972414,37.522969 126.972056,37.523053 126.971644,37.523133 126.971183,37.523214 126.970789,37.523292 126.970444,37.523367 126.970233,37.523428 126.970031,37.523500 126.969831,37.523583 126.969533,37.523728 126.966744,37.525375 126.966603,37.525464 126.966583,37.525478 126.966564,37.525489 126.966547,37.525503 126.966531,37.525517 126.966514,37.525531 126.966500,37.525547 126.966486,37.525564 126.966475,37.525581 126.966461,37.525594 126.966450,37.525614 126.966442,37.525631 126.966431,37.525650 126.966425,37.525667 126.966417,37.525686 126.966411,37.525703 126.966406,37.525722 126.966403,37.525742 126.966400,37.525761 126.966400,37.525781 126.966400,37.525800 126.966400,37.525819 126.966425,37.527406 126.966428,37.527431 126.966431,37.527456 126.966433,37.527481 126.966439,37.527506 126.966444,37.527528 126.966453,37.527553 126.966458,37.527578 126.966467,37.527600 126.966478,37.527625 126.966486,37.527647 126.966497,37.527669 126.966508,37.527692 126.966522,37.527714 126.966533,37.527739 126.966550,37.527758 126.966564,37.527781 126.966581,37.527803 126.966594,37.527822 126.966614,37.527844 126.966631,37.527864 126.966650,37.527883 126.967931,37.529222 126.968436,37.529753 126.971272,37.532750 126.972633,37.534133 126.973353,37.534864 126.973986,37.535603 126.973989,37.535606 126.974167,37.535861 126.974181,37.535883 126.974194,37.535906 126.974206,37.535928 126.974217,37.535950 126.974228,37.535975 126.974239,37.535997 126.974247,37.536019 126.974256,37.536044 126.974261,37.536069 126.974267,37.536092 126.974272,37.536117 126.974278,37.536142 126.974281,37.536167 126.974281,37.536192 126.974283,37.536217 126.974281,37.536239 126.974281,37.536264 126.974278,37.536289 126.974275,37.536314 126.974269,37.536339 126.974267,37.536364 126.972572,37.542839 126.972311,37.543833 126.972008,37.544989 126.971953,37.545361 126.971900,37.546225 126.971819,37.548156 126.971794,37.548819 126.971797,37.549114 126.971819,37.549381 126.971856,37.549678 126.971928,37.550069 126.972000,37.550425 126.972294,37.551683 126.972739,37.553319 126.972747,37.553344 126.972939,37.553928 126.973025,37.554339 126.973044,37.554517 126.973072,37.555861 126.973075,37.555892 126.973081,37.555922 126.973089,37.555953 126.973097,37.555981 126.973108,37.556008 126.973122,37.556039 126.973136,37.556067 126.973153,37.556094 126.973172,37.556119 126.973192,37.556144 126.973214,37.556169 126.973236,37.556194 126.973261,37.556219 126.973289,37.556242 126.973314,37.556261 126.973344,37.556281 126.973375,37.556300 126.973406,37.556317 126.973436,37.556333 126.973469,37.556350 126.973506,37.556361 126.973986,37.556536 126.977178,37.557736 126.977364,37.557817 126.977617,37.557967 126.977814,37.558117 126.978078,37.558394 126.978444,37.558789 126.978564,37.558883 126.978656,37.558956 126.978692,37.558981 126.978731,37.559006 126.978769,37.559028 126.978806,37.559050 126.978844,37.559075 126.978886,37.559094 126.978925,37.559117 126.978967,37.559139 126.979006,37.559158 126.979047,37.559178 126.979089,37.559197 126.979131,37.559217 126.979172,37.559233 126.979217,37.559253 126.979258,37.559269 126.979303,37.559286 126.979344,37.559303 126.979389,37.559319 126.979433,37.559333 126.979478,37.559350 126.979522,37.559364 126.980778,37.559744 126.980817,37.559756 126.980856,37.559769 126.980894,37.559783 126.980933,37.559797 126.980969,37.559811 126.981008,37.559825 126.981044,37.559839 126.981083,37.559856 126.981119,37.559869 126.981158,37.559886 126.981194,37.559903 126.981231,37.559919 126.981267,37.559936 126.981303,37.559953 126.981339,37.559972 126.981375,37.559989 126.981408,37.560008 126.981444,37.560028 126.981478,37.560044 126.981511,37.560067 126.981544,37.560083 126.981869,37.560261 126.982136,37.560417 126.982164,37.560431 126.982189,37.560444 126.982217,37.560456 126.982244,37.560469 126.982269,37.560481 126.982300,37.560494 126.982328,37.560506 126.982356,37.560514 126.982383,37.560525 126.982414,37.560533 126.982442,37.560544 126.982472,37.560553 126.982503,37.560561 126.982531,37.560569 126.982561,37.560575 126.982592,37.560583 126.982622,37.560589 126.982653,37.560594 126.982683,37.560600 126.982714,37.560603 126.982744,37.560608 126.984639,37.560767 126.986489,37.560978 126.986525,37.560983 126.988339,37.561144 126.989236,37.561203 126.989272,37.561206 126.989308,37.561206 126.989344,37.561208 126.989381,37.561208 126.989417,37.561211 126.989453,37.561211 126.989489,37.561211 126.989525,37.561208 126.989561,37.561208 126.989597,37.561208 126.989633,37.561206 126.989669,37.561203 126.989706,37.561203 126.989742,37.561200 126.989778,37.561194 126.989814,37.561192 126.989850,37.561189 126.989886,37.561183 126.989922,37.561178 126.989958,37.561175 126.989994,37.561167 126.990364,37.561100 126.990394,37.561094 126.990431,37.561092 126.990464,37.561086 126.990478,37.561083 126.990497,37.561083 126.990531,37.561078 126.990564,37.561075 126.990597,37.561072 126.990631,37.561069 126.990667,37.561067 126.990700,37.561064 126.990733,37.561061 126.990767,37.561058 126.990800,37.561058 126.990833,37.561056 126.990869,37.561056 126.990903,37.561053 126.990936,37.561053 126.990969,37.561053 126.991006,37.561053 126.991039,37.561053 126.991072,37.561053 126.992567,37.561075 126.993236,37.561078 126.993447,37.561100 126.994111,37.561242 126.994172,37.561256 126.995656,37.561569 127.004903,37.563719 127.005561,37.563858 127.006442,37.564022 127.006633,37.564058 127.006664,37.564067 127.006692,37.564075 127.006719,37.564081 127.006750,37.564092 127.006778,37.564100 127.006803,37.564111 127.006831,37.564122 127.006856,37.564133 127.006883,37.564147 127.006908,37.564158 127.006931,37.564175 127.006956,37.564189 127.006978,37.564206 127.007000,37.564219 127.007014,37.564231 127.007022,37.564239 127.007044,37.564256 127.007064,37.564272 127.007083,37.564292 127.007100,37.564311 127.007119,37.564331 127.007133,37.564350 127.007333,37.564619 127.007356,37.564644 127.008444,37.565839 127.008606,37.566017 127.008781,37.566247 127.008933,37.566497 127.009106,37.566825 127.009281,37.567164 127.009450,37.567483 127.009697,37.567964 127.009717,37.568006 127.009733,37.568050 127.009747,37.568092 127.009761,37.568133 127.009775,37.568178 127.009783,37.568219 127.009794,37.568264 127.009803,37.568308 127.009808,37.568353 127.009814,37.568397 127.009817,37.568442 127.009819,37.568483 127.009819,37.568528 127.009817,37.568575 127.009814,37.568617 127.009811,37.568661 127.009742,37.569244 127.009617,37.570203 127.009486,37.570547 127.009258,37.570911 127.009114,37.571139"
              }
            },
            {
              "mode": "WALK",
              "sectionTime": 193,
              "distance": 194,
              "start": {
                "name": "동대문",
                "lon": 127.00916111111111,
                "lat": 37.57105
              },
              "end": {
                "name": "동대문",
                "lon": 127.01063888888889,
                "lat": 37.571675
              },
              "passShape": {
                "linestring": "127.009161,37.571050 127.009056,37.571261 127.010494,37.571619 127.010442,37.571761 127.010581,37.571792 127.010611,37.571825 127.010639,37.571675"
              }
            },
            {
              "routeColor": "0052A4",
              "distance": 3142,
              "start": {
                "name": "동대문",
                "lon": 127.01063888888889,
                "lat": 37.571675
              },
              "Lane": [
                {
                  "routeColor": "0052A4",
                  "route": "수도권1호선(급행)",
                  "routeId": "111013004",
                  "service": 1,
                  "type": 117
                }
              ],
              "type": 1,
              "mode": "SUBWAY",
              "sectionTime": 480,
              "route": "수도권1호선",
              "routeId": "110011030",
              "service": 0,
              "passStopList": {
                "stationList": [
                  {
                    "index": 0,
                    "stationName": "동대문",
                    "lon": "127.010639",
                    "lat": "37.571675",
                    "stationID": "110129"
                  },
                  {
                    "index": 1,
                    "stationName": "동묘앞",
                    "lon": "127.016594",
                    "lat": "37.573267",
                    "stationID": "110128"
                  },
                  {
                    "index": 2,
                    "stationName": "신설동",
                    "lon": "127.024111",
                    "lat": "37.575894",
                    "stationID": "110127"
                  },
                  {
                    "index": 3,
                    "stationName": "제기동",
                    "lon": "127.034792",
                    "lat": "37.578169",
                    "stationID": "110126"
                  },
                  {
                    "index": 4,
                    "stationName": "청량리",
                    "lon": "127.044556",
                    "lat": "37.579967",
                    "stationID": "110125"
                  }
                ]
              },
              "end": {
                "name": "청량리",
                "lon": 127.04455555555556,
                "lat": 37.579966666666664
              },
              "passShape": {
                "linestring": "127.010636,37.571681 127.012117,37.572006 127.012317,37.572050 127.012464,37.572089 127.012611,37.572125 127.012806,37.572181 127.012953,37.572225 127.013144,37.572283 127.015642,37.572981 127.016603,37.573250 127.017108,37.573392 127.017500,37.573503 127.017731,37.573572 127.017964,37.573644 127.018194,37.573719 127.018425,37.573794 127.020608,37.574539 127.021017,37.574692 127.023233,37.575572 127.023411,37.575644 127.023575,37.575708 127.024111,37.575894 127.024403,37.575994 127.025333,37.576289 127.027267,37.576822 127.030839,37.577864 127.030981,37.577908 127.031122,37.577947 127.031194,37.577967 127.031339,37.578000 127.031411,37.578017 127.031556,37.578047 127.031706,37.578072 127.031853,37.578094 127.032003,37.578114 127.032153,37.578128 127.032228,37.578133 127.032378,37.578142 127.034792,37.578169 127.036183,37.578183 127.038053,37.578169 127.038119,37.578169 127.038253,37.578175 127.038386,37.578181 127.038519,37.578192 127.038653,37.578206 127.038783,37.578225 127.038850,37.578236 127.038978,37.578258 127.039108,37.578289 127.039236,37.578319 127.039300,37.578336 127.039425,37.578372 127.044556,37.579967"
              }
            },
            {
              "mode": "WALK",
              "sectionTime": 630,
              "distance": 692,
              "start": {
                "name": "청량리",
                "lon": 127.04455555555556,
                "lat": 37.579966666666664
              },
              "end": {
                "name": "도착지",
                "lon": 127.048958,
                "lat": 37.581381
              },
              "steps": [
                {
                  "streetName": "",
                  "distance": 128,
                  "description": "128m 이동",
                  "linestring": "127.044556,37.579964 127.044846,37.580055 127.04525,37.580185 127.045395,37.580257 127.045845,37.58048"
                },
                {
                  "streetName": "",
                  "distance": 42,
                  "description": "청량리역  4번출구 에서 우회전 후 42m 이동 ",
                  "linestring": "127.045845,37.58048 127.04597,37.580322 127.046165,37.580433"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 6,
                  "description": "청량리역  4번출구 에서 좌회전 후 보행자도로 을 따라 6m 이동 ",
                  "linestring": "127.046165,37.580433 127.046104,37.58046"
                },
                {
                  "streetName": "왕산로",
                  "distance": 87,
                  "description": "청량리역  4번출구 에서 우회전 후 왕산로 을 따라 87m 이동 ",
                  "linestring": "127.046104,37.58046 127.04615,37.580517 127.046196,37.580578 127.046295,37.58071 127.04644,37.580803 127.04654,37.58086 127.04664,37.58092 127.04674,37.580936 127.046814,37.580925"
                },
                {
                  "streetName": "",
                  "distance": 24,
                  "description": "KFC 청량리점 에서 좌측 횡단보도 후 24m 이동 ",
                  "linestring": "127.046814,37.580925 127.04683,37.580956 127.04684,37.580994 127.04685,37.58104 127.04687,37.581104 127.046875,37.58114"
                },
                {
                  "streetName": "",
                  "distance": 17,
                  "description": "직진 후 17m 이동 ",
                  "linestring": "127.046875,37.58114 127.04689,37.58125 127.046906,37.581284"
                },
                {
                  "streetName": "",
                  "distance": 14,
                  "description": "횡단보도 후 14m 이동 ",
                  "linestring": "127.046906,37.581284 127.04703,37.581375"
                },
                {
                  "streetName": "",
                  "distance": 304,
                  "description": "훼미리마트 청량리역점 에서 우회전 후 304m 이동 ",
                  "linestring": "127.04703,37.581375 127.04703,37.581326 127.04703,37.581295 127.04706,37.581257 127.0471,37.581226 127.047165,37.581207 127.047226,37.581196 127.04729,37.581196 127.047356,37.581207 127.047424,37.581226 127.04754,37.581272 127.047646,37.581345 127.04775,37.581417 127.04815,37.581753 127.04833,37.581894 127.04845,37.581947 127.04858,37.581978 127.04877,37.581997 127.04888,37.58199 127.04908,37.581936 127.04913,37.58191 127.04942,37.58177 127.04982,37.581577"
                },
                {
                  "streetName": "",
                  "distance": 17,
                  "description": "우측 횡단보도 후 17m 이동 ",
                  "linestring": "127.04982,37.581577 127.04971,37.58145"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 53,
                  "description": "우회전 후 보행자도로 을 따라 53m 이동 ",
                  "linestring": "127.04971,37.58145 127.04921,37.581703"
                }
              ]
            }
          ],
          "totalWalkTime": 924,
          "transferCount": 1,
          "totalDistance": 47024,
          "pathType": 1,
          "totalWalkDistance": 1007
        },
        {
          "fare": {
            "regular": {
              "totalFare": 2300,
              "currency": {
                "symbol": "￦",
                "currency": "원",
                "currencyCode": "KRW"
              }
            }
          },
          "totalTime": 6172,
          "legs": [
            {
              "mode": "WALK",
              "sectionTime": 101,
              "distance": 121,
              "start": {
                "name": "출발지",
                "lon": 126.8230369,
                "lat": 37.3168267
              },
              "end": {
                "name": "고잔",
                "lon": 126.82304722222223,
                "lat": 37.316833333333335
              },
              "steps": [
                {
                  "streetName": "",
                  "distance": 27,
                  "description": "27m 이동",
                  "linestring": "126.82303,37.3168 126.82306,37.3168 126.82333,37.316788"
                },
                {
                  "streetName": "",
                  "distance": 15,
                  "description": "고잔역 에서 좌회전 후 15m 이동 ",
                  "linestring": "126.82333,37.316788 126.82334,37.31693"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 20,
                  "description": "직진 후 보행자도로 을 따라 20m 이동 ",
                  "linestring": "126.82334,37.31693 126.82336,37.317104"
                },
                {
                  "streetName": "중앙대로",
                  "distance": 39,
                  "description": "좌회전 후 중앙대로 을 따라 39m 이동 ",
                  "linestring": "126.82336,37.317104 126.822914,37.317135"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 20,
                  "description": "좌회전 후 보행자도로 을 따라 20m 이동 ",
                  "linestring": "126.822914,37.317135 126.82289,37.316963"
                }
              ]
            },
            {
              "mode": "SUBWAY",
              "routeColor": "00A5DE",
              "sectionTime": 4319,
              "route": "수도권4호선",
              "routeId": "110041005",
              "distance": 42538,
              "service": 1,
              "start": {
                "name": "고잔",
                "lon": 126.82304722222223,
                "lat": 37.316833333333335
              },
              "passStopList": {
                "stationList": [
                  {
                    "index": 0,
                    "stationName": "고잔",
                    "lon": "126.823047",
                    "lat": "37.316833",
                    "stationID": "110446"
                  },
                  {
                    "index": 1,
                    "stationName": "중앙",
                    "lon": "126.838481",
                    "lat": "37.316058",
                    "stationID": "110445"
                  },
                  {
                    "index": 2,
                    "stationName": "한대앞",
                    "lon": "126.853636",
                    "lat": "37.309722",
                    "stationID": "110444"
                  },
                  {
                    "index": 3,
                    "stationName": "상록수",
                    "lon": "126.866383",
                    "lat": "37.302869",
                    "stationID": "110443"
                  },
                  {
                    "index": 4,
                    "stationName": "반월",
                    "lon": "126.903367",
                    "lat": "37.311867",
                    "stationID": "110442"
                  },
                  {
                    "index": 5,
                    "stationName": "대야미",
                    "lon": "126.916878",
                    "lat": "37.327964",
                    "stationID": "110441"
                  },
                  {
                    "index": 6,
                    "stationName": "수리산",
                    "lon": "126.925617",
                    "lat": "37.350569",
                    "stationID": "110440"
                  },
                  {
                    "index": 7,
                    "stationName": "산본",
                    "lon": "126.933019",
                    "lat": "37.357989",
                    "stationID": "110439"
                  },
                  {
                    "index": 8,
                    "stationName": "금정",
                    "lon": "126.943608",
                    "lat": "37.372211",
                    "stationID": "110438"
                  },
                  {
                    "index": 9,
                    "stationName": "범계",
                    "lon": "126.950675",
                    "lat": "37.389764",
                    "stationID": "110437"
                  },
                  {
                    "index": 10,
                    "stationName": "평촌",
                    "lon": "126.964106",
                    "lat": "37.394431",
                    "stationID": "110436"
                  },
                  {
                    "index": 11,
                    "stationName": "인덕원",
                    "lon": "126.976800",
                    "lat": "37.401900",
                    "stationID": "110435"
                  },
                  {
                    "index": 12,
                    "stationName": "정부과천청사",
                    "lon": "126.989942",
                    "lat": "37.426792",
                    "stationID": "110434"
                  },
                  {
                    "index": 13,
                    "stationName": "과천",
                    "lon": "126.996669",
                    "lat": "37.433281",
                    "stationID": "110433"
                  },
                  {
                    "index": 14,
                    "stationName": "대공원",
                    "lon": "127.006531",
                    "lat": "37.435831",
                    "stationID": "110432"
                  },
                  {
                    "index": 15,
                    "stationName": "경마공원",
                    "lon": "127.007944",
                    "lat": "37.443806",
                    "stationID": "110431"
                  },
                  {
                    "index": 16,
                    "stationName": "선바위",
                    "lon": "127.001972",
                    "lat": "37.452111",
                    "stationID": "110430"
                  },
                  {
                    "index": 17,
                    "stationName": "남태령",
                    "lon": "126.988883",
                    "lat": "37.464647",
                    "stationID": "110429"
                  },
                  {
                    "index": 18,
                    "stationName": "사당",
                    "lon": "126.981667",
                    "lat": "37.476808",
                    "stationID": "110428"
                  },
                  {
                    "index": 19,
                    "stationName": "총신대입구",
                    "lon": "126.982264",
                    "lat": "37.487347",
                    "stationID": "110427"
                  },
                  {
                    "index": 20,
                    "stationName": "동작",
                    "lon": "126.980344",
                    "lat": "37.502897",
                    "stationID": "110426"
                  },
                  {
                    "index": 21,
                    "stationName": "이촌",
                    "lon": "126.974444",
                    "lat": "37.522350",
                    "stationID": "110425"
                  },
                  {
                    "index": 22,
                    "stationName": "신용산",
                    "lon": "126.967928",
                    "lat": "37.529222",
                    "stationID": "110424"
                  },
                  {
                    "index": 23,
                    "stationName": "삼각지",
                    "lon": "126.972633",
                    "lat": "37.534133",
                    "stationID": "110423"
                  },
                  {
                    "index": 24,
                    "stationName": "숙대입구",
                    "lon": "126.972111",
                    "lat": "37.544600",
                    "stationID": "110422"
                  },
                  {
                    "index": 25,
                    "stationName": "서울역",
                    "lon": "126.972797",
                    "lat": "37.553497",
                    "stationID": "110421"
                  },
                  {
                    "index": 26,
                    "stationName": "회현",
                    "lon": "126.978564",
                    "lat": "37.558886",
                    "stationID": "110420"
                  },
                  {
                    "index": 27,
                    "stationName": "명동",
                    "lon": "126.986489",
                    "lat": "37.560981",
                    "stationID": "110419"
                  },
                  {
                    "index": 28,
                    "stationName": "충무로",
                    "lon": "126.994172",
                    "lat": "37.561256",
                    "stationID": "110418"
                  },
                  {
                    "index": 29,
                    "stationName": "동대문역사문화공원",
                    "lon": "127.007356",
                    "lat": "37.564644",
                    "stationID": "110417"
                  }
                ]
              },
              "end": {
                "name": "동대문역사문화공원",
                "lon": 127.00735555555555,
                "lat": 37.56464444444445
              },
              "type": 4,
              "passShape": {
                "linestring": "126.823047,37.316833 126.824414,37.316767 126.826906,37.316628 126.830992,37.316458 126.832469,37.316378 126.833967,37.316292 126.835467,37.316217 126.836961,37.316139 126.838481,37.316061 126.839886,37.315986 126.842467,37.315819 126.844714,37.315669 126.845500,37.315581 126.846008,37.315494 126.846500,37.315406 126.846953,37.315281 126.847328,37.315158 126.847739,37.314989 126.848297,37.314700 126.848778,37.314403 126.849225,37.314086 126.849656,37.313689 126.850683,37.312733 126.851578,37.311858 126.852081,37.311344 126.852964,37.310425 126.853639,37.309722 126.853753,37.309606 126.854828,37.308542 126.855744,37.307639 126.856328,37.307072 126.857392,37.306008 126.857856,37.305514 126.858386,37.304958 126.858814,37.304581 126.859214,37.304283 126.859542,37.304086 126.859942,37.303889 126.860364,37.303700 126.860919,37.303478 126.861364,37.303333 126.861831,37.303228 126.862433,37.303131 126.863000,37.303081 126.863614,37.303056 126.864522,37.302989 126.864847,37.302964 126.866383,37.302867 126.867319,37.302806 126.871492,37.302481 126.872903,37.302453 126.873700,37.302464 126.874558,37.302494 126.875353,37.302522 126.878306,37.302639 126.879389,37.302683 126.879444,37.302686 126.880114,37.302728 126.881167,37.302778 126.881628,37.302781 126.882600,37.302847 126.883436,37.303019 126.884569,37.303317 126.885189,37.303486 126.885669,37.303642 126.886589,37.303892 126.888092,37.304297 126.888661,37.304450 126.890783,37.305022 126.892403,37.305458 126.892564,37.305503 126.892981,37.305614 126.894681,37.306069 126.895633,37.306331 126.896247,37.306492 126.896492,37.306558 126.896719,37.306622 126.896942,37.306689 126.897150,37.306756 126.897361,37.306831 126.897625,37.306928 126.897911,37.307047 126.898361,37.307239 126.898828,37.307475 126.899233,37.307703 126.899661,37.307931 126.899892,37.308075 126.900097,37.308219 126.900678,37.308725 126.900789,37.308831 126.901239,37.309314 126.901683,37.309800 126.902156,37.310414 126.902169,37.310428 126.903367,37.311867 126.904406,37.313108 126.905708,37.314681 126.907406,37.316717 126.908850,37.318425 126.910258,37.320100 126.911522,37.321614 126.912292,37.322533 126.912761,37.323092 126.913981,37.324544 126.914781,37.325519 126.915297,37.326094 126.915756,37.326639 126.916767,37.327828 126.916878,37.327964 126.917661,37.328922 126.917814,37.329117 126.918161,37.329506 126.918344,37.329744 126.918844,37.330286 126.919256,37.330789 126.919794,37.331383 126.920208,37.331872 126.920333,37.332044 126.920756,37.332558 126.921186,37.333181 126.921350,37.333408 126.921494,37.333658 126.921644,37.333936 126.921775,37.334258 126.922056,37.334978 126.922269,37.335875 126.922333,37.336169 126.922636,37.337656 126.923267,37.340964 126.923528,37.342833 126.923919,37.344722 126.924153,37.345831 126.925533,37.350419 126.925583,37.350578 126.925914,37.351614 126.926225,37.352608 126.926347,37.352931 126.926475,37.353253 126.926664,37.353675 126.926964,37.354200 126.927178,37.354453 126.927636,37.354939 126.928156,37.355353 126.930842,37.356931 126.932967,37.358056 126.935383,37.359333 126.936328,37.359833 126.939789,37.361636 126.942525,37.363175 126.944708,37.364322 126.944894,37.364464 126.945069,37.364628 126.945267,37.364825 126.945411,37.365006 126.945514,37.365156 126.945586,37.365269 126.945653,37.365397 126.945731,37.365625 126.945792,37.365825 126.945825,37.366022 126.945839,37.366267 126.945839,37.366431 126.945808,37.366608 126.945775,37.366781 126.945708,37.366978 126.945628,37.367172 126.945536,37.367378 126.945350,37.367761 126.945236,37.367994 126.945072,37.368339 126.945006,37.368506 126.943858,37.371331 126.943644,37.371906 126.943539,37.372194 126.943325,37.372775 126.942281,37.374958 126.941906,37.375700 126.941767,37.375947 126.941228,37.376814 126.940958,37.377247 126.940797,37.377472 126.940664,37.377628 126.940542,37.377758 126.940036,37.378289 126.939917,37.378406 126.938700,37.379531 126.938669,37.379575 126.938653,37.379619 126.938642,37.379664 126.938633,37.379706 126.938636,37.379767 126.938644,37.379828 126.938872,37.380197 126.939181,37.380786 126.939919,37.382556 126.940189,37.383083 126.940436,37.383478 126.940786,37.383844 126.941064,37.384042 126.941464,37.384264 126.941989,37.384469 126.942383,37.384636 126.942733,37.384825 126.942994,37.384981 126.943428,37.385258 126.943864,37.385597 126.945186,37.386678 126.947628,37.388694 126.947642,37.388703 126.947653,37.388714 126.947672,37.388722 126.947686,37.388728 126.947700,37.388733 126.950675,37.389767 126.962272,37.393797 126.964086,37.394467 126.964389,37.394578 126.968514,37.395942 126.971767,37.397033 126.973083,37.397500 126.974419,37.397997 126.975417,37.398531 126.975703,37.398761 126.975883,37.398925 126.976144,37.399208 126.976603,37.399822 126.976797,37.400136 126.976881,37.400292 126.976914,37.400467 126.976833,37.401311 126.976833,37.401350 126.976783,37.402194 126.976803,37.402544 126.976883,37.402842 126.977764,37.404747 126.977808,37.404872 126.978228,37.406642 126.978494,37.407764 126.979033,37.410022 126.979122,37.410444 126.979419,37.411742 126.979725,37.412986 126.979781,37.413208 126.979867,37.413675 126.979908,37.413847 126.980011,37.414200 126.980928,37.416297 126.981547,37.417675 126.981750,37.418144 126.982047,37.418675 126.982469,37.419406 126.982989,37.420228 126.983950,37.421275 126.984853,37.422069 126.985869,37.422925 126.987003,37.423869 126.987781,37.424625 126.988981,37.425722 126.989956,37.426783 126.990411,37.427278 126.991381,37.428233 126.992906,37.429739 126.995314,37.432433 126.995839,37.432964 126.996069,37.433125 126.996378,37.433244 126.996656,37.433317 126.996756,37.433342 126.997372,37.433425 126.997742,37.433425 126.998319,37.433353 126.999225,37.433258 126.999853,37.433225 127.000758,37.433239 127.004331,37.433214 127.004481,37.433256 127.005314,37.433958 127.005433,37.434131 127.006244,37.435392 127.006450,37.435711 127.008228,37.438322 127.008297,37.438447 127.008344,37.438575 127.008364,37.438711 127.008375,37.438969 127.008203,37.440986 127.008019,37.442944 127.007969,37.443506 127.007944,37.443831 127.007819,37.444217 127.007717,37.444408 127.007436,37.444817 127.006928,37.445475 127.004367,37.448900 127.002544,37.451400 127.001972,37.452111 127.001836,37.452281 127.001633,37.452472 127.001442,37.452575 127.000942,37.452764 126.999947,37.453253 126.998028,37.454378 126.997922,37.454442 126.996819,37.455239 126.995569,37.456381 126.994317,37.457522 126.993775,37.458058 126.993728,37.458117 126.993250,37.458667 126.991156,37.461425 126.989422,37.463706 126.989111,37.464247 126.988883,37.464647 126.987964,37.466256 126.987881,37.466394 126.987797,37.466528 126.987708,37.466664 126.987617,37.466794 126.987525,37.466928 126.987428,37.467058 126.987328,37.467186 126.987225,37.467314 126.987122,37.467442 126.987014,37.467567 126.986903,37.467689 126.986792,37.467811 126.986675,37.467931 126.986558,37.468050 126.986439,37.468167 126.986314,37.468283 126.986189,37.468394 126.986061,37.468508 126.985931,37.468619 126.985800,37.468728 126.985664,37.468833 126.985211,37.469178 126.984864,37.469464 126.984636,37.469658 126.984472,37.469808 126.984356,37.469928 126.983858,37.470544 126.983769,37.470672 126.983256,37.471453 126.982953,37.472042 126.982397,37.473183 126.982053,37.474025 126.981878,37.474547 126.981769,37.474950 126.981675,37.475608 126.981669,37.475658 126.981633,37.476083 126.981656,37.476564 126.981658,37.476653 126.982111,37.485253 126.982117,37.485383 126.982122,37.485475 126.982158,37.486308 126.982228,37.487350 126.982233,37.487400 126.982378,37.488686 126.982531,37.489653 126.982592,37.490369 126.982647,37.494967 126.982628,37.496269 126.982639,37.496469 126.982664,37.496669 126.982692,37.496817 126.982744,37.497011 126.982794,37.497156 126.982983,37.497625 126.983100,37.497950 126.983239,37.498383 126.983267,37.498531 126.983264,37.498681 126.983233,37.498825 126.983178,37.498967 126.983067,37.499131 126.981803,37.500881 126.981647,37.501097 126.981553,37.501222 126.981425,37.501369 126.980922,37.501886 126.980856,37.501964 126.980794,37.502033 126.980644,37.502233 126.980547,37.502394 126.980464,37.502575 126.980381,37.502808 126.980294,37.503031 126.980269,37.503125 126.980183,37.503464 126.980156,37.503878 126.980156,37.504058 126.980164,37.504244 126.980186,37.504425 126.980258,37.504800 126.980719,37.506508 126.983114,37.515581 126.983161,37.515808 126.983203,37.516089 126.983231,37.516306 126.983269,37.517844 126.983278,37.518414 126.983261,37.518581 126.983225,37.518736 126.983183,37.518883 126.983128,37.519019 126.983053,37.519164 126.982961,37.519292 126.982853,37.519408 126.982725,37.519531 126.982633,37.519608 126.982606,37.519633 126.982456,37.519739 126.982292,37.519836 126.982108,37.519936 126.981919,37.520033 126.981719,37.520119 126.981503,37.520206 126.976931,37.521594 126.975139,37.522128 126.974444,37.522344 126.972900,37.522833 126.972414,37.522969 126.972056,37.523053 126.971644,37.523133 126.971183,37.523214 126.970789,37.523292 126.970444,37.523367 126.970233,37.523428 126.970031,37.523500 126.969831,37.523583 126.969533,37.523728 126.966744,37.525375 126.966603,37.525464 126.966583,37.525478 126.966564,37.525489 126.966547,37.525503 126.966531,37.525517 126.966514,37.525531 126.966500,37.525547 126.966486,37.525564 126.966475,37.525581 126.966461,37.525594 126.966450,37.525614 126.966442,37.525631 126.966431,37.525650 126.966425,37.525667 126.966417,37.525686 126.966411,37.525703 126.966406,37.525722 126.966403,37.525742 126.966400,37.525761 126.966400,37.525781 126.966400,37.525800 126.966400,37.525819 126.966425,37.527406 126.966428,37.527431 126.966431,37.527456 126.966433,37.527481 126.966439,37.527506 126.966444,37.527528 126.966453,37.527553 126.966458,37.527578 126.966467,37.527600 126.966478,37.527625 126.966486,37.527647 126.966497,37.527669 126.966508,37.527692 126.966522,37.527714 126.966533,37.527739 126.966550,37.527758 126.966564,37.527781 126.966581,37.527803 126.966594,37.527822 126.966614,37.527844 126.966631,37.527864 126.966650,37.527883 126.967931,37.529222 126.968436,37.529753 126.971272,37.532750 126.972633,37.534133 126.973353,37.534864 126.973986,37.535603 126.973989,37.535606 126.974167,37.535861 126.974181,37.535883 126.974194,37.535906 126.974206,37.535928 126.974217,37.535950 126.974228,37.535975 126.974239,37.535997 126.974247,37.536019 126.974256,37.536044 126.974261,37.536069 126.974267,37.536092 126.974272,37.536117 126.974278,37.536142 126.974281,37.536167 126.974281,37.536192 126.974283,37.536217 126.974281,37.536239 126.974281,37.536264 126.974278,37.536289 126.974275,37.536314 126.974269,37.536339 126.974267,37.536364 126.972572,37.542839 126.972311,37.543833 126.972008,37.544989 126.971953,37.545361 126.971900,37.546225 126.971819,37.548156 126.971794,37.548819 126.971797,37.549114 126.971819,37.549381 126.971856,37.549678 126.971928,37.550069 126.972000,37.550425 126.972294,37.551683 126.972739,37.553319 126.972747,37.553344 126.972939,37.553928 126.973025,37.554339 126.973044,37.554517 126.973072,37.555861 126.973075,37.555892 126.973081,37.555922 126.973089,37.555953 126.973097,37.555981 126.973108,37.556008 126.973122,37.556039 126.973136,37.556067 126.973153,37.556094 126.973172,37.556119 126.973192,37.556144 126.973214,37.556169 126.973236,37.556194 126.973261,37.556219 126.973289,37.556242 126.973314,37.556261 126.973344,37.556281 126.973375,37.556300 126.973406,37.556317 126.973436,37.556333 126.973469,37.556350 126.973506,37.556361 126.973986,37.556536 126.977178,37.557736 126.977364,37.557817 126.977617,37.557967 126.977814,37.558117 126.978078,37.558394 126.978444,37.558789 126.978564,37.558883 126.978656,37.558956 126.978692,37.558981 126.978731,37.559006 126.978769,37.559028 126.978806,37.559050 126.978844,37.559075 126.978886,37.559094 126.978925,37.559117 126.978967,37.559139 126.979006,37.559158 126.979047,37.559178 126.979089,37.559197 126.979131,37.559217 126.979172,37.559233 126.979217,37.559253 126.979258,37.559269 126.979303,37.559286 126.979344,37.559303 126.979389,37.559319 126.979433,37.559333 126.979478,37.559350 126.979522,37.559364 126.980778,37.559744 126.980817,37.559756 126.980856,37.559769 126.980894,37.559783 126.980933,37.559797 126.980969,37.559811 126.981008,37.559825 126.981044,37.559839 126.981083,37.559856 126.981119,37.559869 126.981158,37.559886 126.981194,37.559903 126.981231,37.559919 126.981267,37.559936 126.981303,37.559953 126.981339,37.559972 126.981375,37.559989 126.981408,37.560008 126.981444,37.560028 126.981478,37.560044 126.981511,37.560067 126.981544,37.560083 126.981869,37.560261 126.982136,37.560417 126.982164,37.560431 126.982189,37.560444 126.982217,37.560456 126.982244,37.560469 126.982269,37.560481 126.982300,37.560494 126.982328,37.560506 126.982356,37.560514 126.982383,37.560525 126.982414,37.560533 126.982442,37.560544 126.982472,37.560553 126.982503,37.560561 126.982531,37.560569 126.982561,37.560575 126.982592,37.560583 126.982622,37.560589 126.982653,37.560594 126.982683,37.560600 126.982714,37.560603 126.982744,37.560608 126.984639,37.560767 126.986489,37.560978 126.986525,37.560983 126.988339,37.561144 126.989236,37.561203 126.989272,37.561206 126.989308,37.561206 126.989344,37.561208 126.989381,37.561208 126.989417,37.561211 126.989453,37.561211 126.989489,37.561211 126.989525,37.561208 126.989561,37.561208 126.989597,37.561208 126.989633,37.561206 126.989669,37.561203 126.989706,37.561203 126.989742,37.561200 126.989778,37.561194 126.989814,37.561192 126.989850,37.561189 126.989886,37.561183 126.989922,37.561178 126.989958,37.561175 126.989994,37.561167 126.990364,37.561100 126.990394,37.561094 126.990431,37.561092 126.990464,37.561086 126.990478,37.561083 126.990497,37.561083 126.990531,37.561078 126.990564,37.561075 126.990597,37.561072 126.990631,37.561069 126.990667,37.561067 126.990700,37.561064 126.990733,37.561061 126.990767,37.561058 126.990800,37.561058 126.990833,37.561056 126.990869,37.561056 126.990903,37.561053 126.990936,37.561053 126.990969,37.561053 126.991006,37.561053 126.991039,37.561053 126.991072,37.561053 126.992567,37.561075 126.993236,37.561078 126.993447,37.561100 126.994111,37.561242 126.994172,37.561256 126.995656,37.561569 127.004903,37.563719 127.005561,37.563858 127.006442,37.564022 127.006633,37.564058 127.006664,37.564067 127.006692,37.564075 127.006719,37.564081 127.006750,37.564092 127.006778,37.564100 127.006803,37.564111 127.006831,37.564122 127.006856,37.564133 127.006883,37.564147 127.006908,37.564158 127.006931,37.564175 127.006956,37.564189 127.006978,37.564206 127.007000,37.564219 127.007014,37.564231 127.007022,37.564239 127.007044,37.564256 127.007064,37.564272 127.007083,37.564292 127.007100,37.564311 127.007119,37.564331 127.007133,37.564350 127.007333,37.564619 127.007356,37.564644"
              }
            },
            {
              "mode": "WALK",
              "sectionTime": 132,
              "distance": 161,
              "start": {
                "name": "동대문역사문화공원",
                "lon": 127.00735555555555,
                "lat": 37.56464444444445
              },
              "end": {
                "name": "광희동",
                "lon": 127.00770555555556,
                "lat": 37.56516388888889
              },
              "passShape": {
                "linestring": "127.007356,37.564644 127.007281,37.564508 127.007428,37.564425 127.007717,37.564403 127.007728,37.564414 127.007611,37.564425 127.007544,37.564433 127.007519,37.564464 127.007497,37.564483 127.007481,37.564508 127.007500,37.564578 127.007581,37.564803 127.007642,37.564956 127.007678,37.565058 127.007706,37.565164"
              }
            },
            {
              "routeColor": "0068B7",
              "distance": 3979,
              "start": {
                "name": "광희동",
                "lon": 127.00770555555556,
                "lat": 37.56516388888889
              },
              "Lane": [
                {
                  "routeColor": "0068B7",
                  "route": "간선:N13",
                  "routeId": "11466001",
                  "service": 0,
                  "type": 11
                }
              ],
              "type": 11,
              "mode": "BUS",
              "sectionTime": 1124,
              "route": "간선:420",
              "routeId": "11455001",
              "service": 1,
              "passStopList": {
                "stationList": [
                  {
                    "index": 0,
                    "stationName": "광희동",
                    "lon": "127.007706",
                    "lat": "37.565164",
                    "stationID": "774916"
                  },
                  {
                    "index": 1,
                    "stationName": "동대문역사문화공원",
                    "lon": "127.009017",
                    "lat": "37.567569",
                    "stationID": "775011"
                  },
                  {
                    "index": 2,
                    "stationName": "동대문역.흥인지문",
                    "lon": "127.013417",
                    "lat": "37.572250",
                    "stationID": "775153"
                  },
                  {
                    "index": 3,
                    "stationName": "동묘앞",
                    "lon": "127.018736",
                    "lat": "37.573803",
                    "stationID": "775214"
                  },
                  {
                    "index": 4,
                    "stationName": "신설동역",
                    "lon": "127.025250",
                    "lat": "37.576150",
                    "stationID": "775282"
                  },
                  {
                    "index": 5,
                    "stationName": "용두동사거리",
                    "lon": "127.030111",
                    "lat": "37.577539",
                    "stationID": "775322"
                  },
                  {
                    "index": 6,
                    "stationName": "제기동역.서울약령시",
                    "lon": "127.037442",
                    "lat": "37.578072",
                    "stationID": "775336"
                  },
                  {
                    "index": 7,
                    "stationName": "청량리수산시장",
                    "lon": "127.041825",
                    "lat": "37.579014",
                    "stationID": "775365"
                  },
                  {
                    "index": 8,
                    "stationName": "청량리역환승센터(3번승강장)",
                    "lon": "127.045953",
                    "lat": "37.580481",
                    "stationID": "775408"
                  }
                ]
              },
              "end": {
                "name": "청량리역환승센터(3번승강장)",
                "lon": 127.04595277777777,
                "lat": 37.58048055555555
              },
              "passShape": {
                "linestring": "127.007614,37.565164 127.007772,37.565600 127.007889,37.565917 127.007939,37.566056 127.008106,37.566431 127.008947,37.567617 127.009025,37.567725 127.009686,37.568731 127.009750,37.568869 127.009828,37.569236 127.009844,37.569428 127.009833,37.569608 127.009778,37.569958 127.009708,37.570239 127.009578,37.570614 127.009378,37.571011 127.009239,37.571231 127.009181,37.571314 127.009489,37.571400 127.010361,37.571564 127.010828,37.571633 127.012281,37.571950 127.013406,37.572283 127.015558,37.572922 127.015733,37.572972 127.017636,37.573467 127.018711,37.573833 127.020483,37.574436 127.022508,37.575214 127.023142,37.575464 127.023414,37.575578 127.023914,37.575775 127.025200,37.576175 127.025228,37.576183 127.028344,37.577056 127.030089,37.577569 127.030400,37.577661 127.030581,37.577714 127.031100,37.577856 127.031703,37.577994 127.032378,37.578072 127.036372,37.578106 127.037419,37.578106 127.038231,37.578103 127.038414,37.578111 127.038728,37.578144 127.039303,37.578253 127.041822,37.579050 127.042503,37.579267 127.042672,37.579319 127.045083,37.580072 127.045256,37.580131 127.045558,37.580256 127.045828,37.580419 127.045908,37.580489"
              }
            },
            {
              "mode": "WALK",
              "sectionTime": 496,
              "distance": 528,
              "start": {
                "name": "청량리역환승센터(3번승강장)",
                "lon": 127.04595277777777,
                "lat": 37.58048055555555
              },
              "end": {
                "name": "도착지",
                "lon": 127.048958,
                "lat": 37.581381
              },
              "steps": [
                {
                  "streetName": "보행자도로",
                  "distance": 21,
                  "description": "보행자도로 을 따라 21m 이동",
                  "linestring": "127.04594,37.580486 127.045975,37.580517 127.04613,37.5806"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 6,
                  "description": "우측 횡단보도 후 보행자도로 을 따라 6m 이동 ",
                  "linestring": "127.04613,37.5806 127.046196,37.580578"
                },
                {
                  "streetName": "왕산로",
                  "distance": 72,
                  "description": "좌회전 후 왕산로 을 따라 72m 이동 ",
                  "linestring": "127.046196,37.580578 127.046295,37.58071 127.04644,37.580803 127.04654,37.58086 127.04664,37.58092 127.04674,37.580936 127.046814,37.580925"
                },
                {
                  "streetName": "",
                  "distance": 24,
                  "description": "KFC 청량리점 에서 좌측 횡단보도 후 24m 이동 ",
                  "linestring": "127.046814,37.580925 127.04683,37.580956 127.04684,37.580994 127.04685,37.58104 127.04687,37.581104 127.046875,37.58114"
                },
                {
                  "streetName": "",
                  "distance": 17,
                  "description": "직진 후 17m 이동 ",
                  "linestring": "127.046875,37.58114 127.04689,37.58125 127.046906,37.581284"
                },
                {
                  "streetName": "",
                  "distance": 14,
                  "description": "횡단보도 후 14m 이동 ",
                  "linestring": "127.046906,37.581284 127.04703,37.581375"
                },
                {
                  "streetName": "",
                  "distance": 304,
                  "description": "훼미리마트 청량리역점 에서 우회전 후 304m 이동 ",
                  "linestring": "127.04703,37.581375 127.04703,37.581326 127.04703,37.581295 127.04706,37.581257 127.0471,37.581226 127.047165,37.581207 127.047226,37.581196 127.04729,37.581196 127.047356,37.581207 127.047424,37.581226 127.04754,37.581272 127.047646,37.581345 127.04775,37.581417 127.04815,37.581753 127.04833,37.581894 127.04845,37.581947 127.04858,37.581978 127.04877,37.581997 127.04888,37.58199 127.04908,37.581936 127.04913,37.58191 127.04942,37.58177 127.04982,37.581577"
                },
                {
                  "streetName": "",
                  "distance": 17,
                  "description": "우측 횡단보도 후 17m 이동 ",
                  "linestring": "127.04982,37.581577 127.04971,37.58145"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 53,
                  "description": "우회전 후 보행자도로 을 따라 53m 이동 ",
                  "linestring": "127.04971,37.58145 127.04921,37.581703"
                }
              ]
            }
          ],
          "totalWalkTime": 729,
          "transferCount": 1,
          "totalDistance": 46964,
          "pathType": 3,
          "totalWalkDistance": 810
        },
        {
          "fare": {
            "regular": {
              "totalFare": 2300,
              "currency": {
                "symbol": "￦",
                "currency": "원",
                "currencyCode": "KRW"
              }
            }
          },
          "totalTime": 6161,
          "legs": [
            {
              "mode": "WALK",
              "sectionTime": 101,
              "distance": 121,
              "start": {
                "name": "출발지",
                "lon": 126.8230369,
                "lat": 37.3168267
              },
              "end": {
                "name": "고잔",
                "lon": 126.82304722222223,
                "lat": 37.316833333333335
              },
              "steps": [
                {
                  "streetName": "",
                  "distance": 27,
                  "description": "27m 이동",
                  "linestring": "126.82303,37.3168 126.82306,37.3168 126.82333,37.316788"
                },
                {
                  "streetName": "",
                  "distance": 15,
                  "description": "고잔역 에서 좌회전 후 15m 이동 ",
                  "linestring": "126.82333,37.316788 126.82334,37.31693"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 20,
                  "description": "직진 후 보행자도로 을 따라 20m 이동 ",
                  "linestring": "126.82334,37.31693 126.82336,37.317104"
                },
                {
                  "streetName": "중앙대로",
                  "distance": 39,
                  "description": "좌회전 후 중앙대로 을 따라 39m 이동 ",
                  "linestring": "126.82336,37.317104 126.822914,37.317135"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 20,
                  "description": "좌회전 후 보행자도로 을 따라 20m 이동 ",
                  "linestring": "126.822914,37.317135 126.82289,37.316963"
                }
              ]
            },
            {
              "mode": "SUBWAY",
              "routeColor": "00A5DE",
              "sectionTime": 4381,
              "route": "수도권4호선",
              "routeId": "110041005",
              "distance": 43266,
              "service": 1,
              "start": {
                "name": "고잔",
                "lon": 126.82304722222223,
                "lat": 37.316833333333335
              },
              "passStopList": {
                "stationList": [
                  {
                    "index": 0,
                    "stationName": "고잔",
                    "lon": "126.823047",
                    "lat": "37.316833",
                    "stationID": "110446"
                  },
                  {
                    "index": 1,
                    "stationName": "중앙",
                    "lon": "126.838481",
                    "lat": "37.316058",
                    "stationID": "110445"
                  },
                  {
                    "index": 2,
                    "stationName": "한대앞",
                    "lon": "126.853636",
                    "lat": "37.309722",
                    "stationID": "110444"
                  },
                  {
                    "index": 3,
                    "stationName": "상록수",
                    "lon": "126.866383",
                    "lat": "37.302869",
                    "stationID": "110443"
                  },
                  {
                    "index": 4,
                    "stationName": "반월",
                    "lon": "126.903367",
                    "lat": "37.311867",
                    "stationID": "110442"
                  },
                  {
                    "index": 5,
                    "stationName": "대야미",
                    "lon": "126.916878",
                    "lat": "37.327964",
                    "stationID": "110441"
                  },
                  {
                    "index": 6,
                    "stationName": "수리산",
                    "lon": "126.925617",
                    "lat": "37.350569",
                    "stationID": "110440"
                  },
                  {
                    "index": 7,
                    "stationName": "산본",
                    "lon": "126.933019",
                    "lat": "37.357989",
                    "stationID": "110439"
                  },
                  {
                    "index": 8,
                    "stationName": "금정",
                    "lon": "126.943608",
                    "lat": "37.372211",
                    "stationID": "110438"
                  },
                  {
                    "index": 9,
                    "stationName": "범계",
                    "lon": "126.950675",
                    "lat": "37.389764",
                    "stationID": "110437"
                  },
                  {
                    "index": 10,
                    "stationName": "평촌",
                    "lon": "126.964106",
                    "lat": "37.394431",
                    "stationID": "110436"
                  },
                  {
                    "index": 11,
                    "stationName": "인덕원",
                    "lon": "126.976800",
                    "lat": "37.401900",
                    "stationID": "110435"
                  },
                  {
                    "index": 12,
                    "stationName": "정부과천청사",
                    "lon": "126.989942",
                    "lat": "37.426792",
                    "stationID": "110434"
                  },
                  {
                    "index": 13,
                    "stationName": "과천",
                    "lon": "126.996669",
                    "lat": "37.433281",
                    "stationID": "110433"
                  },
                  {
                    "index": 14,
                    "stationName": "대공원",
                    "lon": "127.006531",
                    "lat": "37.435831",
                    "stationID": "110432"
                  },
                  {
                    "index": 15,
                    "stationName": "경마공원",
                    "lon": "127.007944",
                    "lat": "37.443806",
                    "stationID": "110431"
                  },
                  {
                    "index": 16,
                    "stationName": "선바위",
                    "lon": "127.001972",
                    "lat": "37.452111",
                    "stationID": "110430"
                  },
                  {
                    "index": 17,
                    "stationName": "남태령",
                    "lon": "126.988883",
                    "lat": "37.464647",
                    "stationID": "110429"
                  },
                  {
                    "index": 18,
                    "stationName": "사당",
                    "lon": "126.981667",
                    "lat": "37.476808",
                    "stationID": "110428"
                  },
                  {
                    "index": 19,
                    "stationName": "총신대입구",
                    "lon": "126.982264",
                    "lat": "37.487347",
                    "stationID": "110427"
                  },
                  {
                    "index": 20,
                    "stationName": "동작",
                    "lon": "126.980344",
                    "lat": "37.502897",
                    "stationID": "110426"
                  },
                  {
                    "index": 21,
                    "stationName": "이촌",
                    "lon": "126.974444",
                    "lat": "37.522350",
                    "stationID": "110425"
                  },
                  {
                    "index": 22,
                    "stationName": "신용산",
                    "lon": "126.967928",
                    "lat": "37.529222",
                    "stationID": "110424"
                  },
                  {
                    "index": 23,
                    "stationName": "삼각지",
                    "lon": "126.972633",
                    "lat": "37.534133",
                    "stationID": "110423"
                  },
                  {
                    "index": 24,
                    "stationName": "숙대입구",
                    "lon": "126.972111",
                    "lat": "37.544600",
                    "stationID": "110422"
                  },
                  {
                    "index": 25,
                    "stationName": "서울역",
                    "lon": "126.972797",
                    "lat": "37.553497",
                    "stationID": "110421"
                  },
                  {
                    "index": 26,
                    "stationName": "회현",
                    "lon": "126.978564",
                    "lat": "37.558886",
                    "stationID": "110420"
                  },
                  {
                    "index": 27,
                    "stationName": "명동",
                    "lon": "126.986489",
                    "lat": "37.560981",
                    "stationID": "110419"
                  },
                  {
                    "index": 28,
                    "stationName": "충무로",
                    "lon": "126.994172",
                    "lat": "37.561256",
                    "stationID": "110418"
                  },
                  {
                    "index": 29,
                    "stationName": "동대문역사문화공원",
                    "lon": "127.007356",
                    "lat": "37.564644",
                    "stationID": "110417"
                  },
                  {
                    "index": 30,
                    "stationName": "동대문",
                    "lon": "127.009161",
                    "lat": "37.571050",
                    "stationID": "110416"
                  }
                ]
              },
              "end": {
                "name": "동대문",
                "lon": 127.00916111111111,
                "lat": 37.57105
              },
              "type": 4,
              "passShape": {
                "linestring": "126.823047,37.316833 126.824414,37.316767 126.826906,37.316628 126.830992,37.316458 126.832469,37.316378 126.833967,37.316292 126.835467,37.316217 126.836961,37.316139 126.838481,37.316061 126.839886,37.315986 126.842467,37.315819 126.844714,37.315669 126.845500,37.315581 126.846008,37.315494 126.846500,37.315406 126.846953,37.315281 126.847328,37.315158 126.847739,37.314989 126.848297,37.314700 126.848778,37.314403 126.849225,37.314086 126.849656,37.313689 126.850683,37.312733 126.851578,37.311858 126.852081,37.311344 126.852964,37.310425 126.853639,37.309722 126.853753,37.309606 126.854828,37.308542 126.855744,37.307639 126.856328,37.307072 126.857392,37.306008 126.857856,37.305514 126.858386,37.304958 126.858814,37.304581 126.859214,37.304283 126.859542,37.304086 126.859942,37.303889 126.860364,37.303700 126.860919,37.303478 126.861364,37.303333 126.861831,37.303228 126.862433,37.303131 126.863000,37.303081 126.863614,37.303056 126.864522,37.302989 126.864847,37.302964 126.866383,37.302867 126.867319,37.302806 126.871492,37.302481 126.872903,37.302453 126.873700,37.302464 126.874558,37.302494 126.875353,37.302522 126.878306,37.302639 126.879389,37.302683 126.879444,37.302686 126.880114,37.302728 126.881167,37.302778 126.881628,37.302781 126.882600,37.302847 126.883436,37.303019 126.884569,37.303317 126.885189,37.303486 126.885669,37.303642 126.886589,37.303892 126.888092,37.304297 126.888661,37.304450 126.890783,37.305022 126.892403,37.305458 126.892564,37.305503 126.892981,37.305614 126.894681,37.306069 126.895633,37.306331 126.896247,37.306492 126.896492,37.306558 126.896719,37.306622 126.896942,37.306689 126.897150,37.306756 126.897361,37.306831 126.897625,37.306928 126.897911,37.307047 126.898361,37.307239 126.898828,37.307475 126.899233,37.307703 126.899661,37.307931 126.899892,37.308075 126.900097,37.308219 126.900678,37.308725 126.900789,37.308831 126.901239,37.309314 126.901683,37.309800 126.902156,37.310414 126.902169,37.310428 126.903367,37.311867 126.904406,37.313108 126.905708,37.314681 126.907406,37.316717 126.908850,37.318425 126.910258,37.320100 126.911522,37.321614 126.912292,37.322533 126.912761,37.323092 126.913981,37.324544 126.914781,37.325519 126.915297,37.326094 126.915756,37.326639 126.916767,37.327828 126.916878,37.327964 126.917661,37.328922 126.917814,37.329117 126.918161,37.329506 126.918344,37.329744 126.918844,37.330286 126.919256,37.330789 126.919794,37.331383 126.920208,37.331872 126.920333,37.332044 126.920756,37.332558 126.921186,37.333181 126.921350,37.333408 126.921494,37.333658 126.921644,37.333936 126.921775,37.334258 126.922056,37.334978 126.922269,37.335875 126.922333,37.336169 126.922636,37.337656 126.923267,37.340964 126.923528,37.342833 126.923919,37.344722 126.924153,37.345831 126.925533,37.350419 126.925583,37.350578 126.925914,37.351614 126.926225,37.352608 126.926347,37.352931 126.926475,37.353253 126.926664,37.353675 126.926964,37.354200 126.927178,37.354453 126.927636,37.354939 126.928156,37.355353 126.930842,37.356931 126.932967,37.358056 126.935383,37.359333 126.936328,37.359833 126.939789,37.361636 126.942525,37.363175 126.944708,37.364322 126.944894,37.364464 126.945069,37.364628 126.945267,37.364825 126.945411,37.365006 126.945514,37.365156 126.945586,37.365269 126.945653,37.365397 126.945731,37.365625 126.945792,37.365825 126.945825,37.366022 126.945839,37.366267 126.945839,37.366431 126.945808,37.366608 126.945775,37.366781 126.945708,37.366978 126.945628,37.367172 126.945536,37.367378 126.945350,37.367761 126.945236,37.367994 126.945072,37.368339 126.945006,37.368506 126.943858,37.371331 126.943644,37.371906 126.943539,37.372194 126.943325,37.372775 126.942281,37.374958 126.941906,37.375700 126.941767,37.375947 126.941228,37.376814 126.940958,37.377247 126.940797,37.377472 126.940664,37.377628 126.940542,37.377758 126.940036,37.378289 126.939917,37.378406 126.938700,37.379531 126.938669,37.379575 126.938653,37.379619 126.938642,37.379664 126.938633,37.379706 126.938636,37.379767 126.938644,37.379828 126.938872,37.380197 126.939181,37.380786 126.939919,37.382556 126.940189,37.383083 126.940436,37.383478 126.940786,37.383844 126.941064,37.384042 126.941464,37.384264 126.941989,37.384469 126.942383,37.384636 126.942733,37.384825 126.942994,37.384981 126.943428,37.385258 126.943864,37.385597 126.945186,37.386678 126.947628,37.388694 126.947642,37.388703 126.947653,37.388714 126.947672,37.388722 126.947686,37.388728 126.947700,37.388733 126.950675,37.389767 126.962272,37.393797 126.964086,37.394467 126.964389,37.394578 126.968514,37.395942 126.971767,37.397033 126.973083,37.397500 126.974419,37.397997 126.975417,37.398531 126.975703,37.398761 126.975883,37.398925 126.976144,37.399208 126.976603,37.399822 126.976797,37.400136 126.976881,37.400292 126.976914,37.400467 126.976833,37.401311 126.976833,37.401350 126.976783,37.402194 126.976803,37.402544 126.976883,37.402842 126.977764,37.404747 126.977808,37.404872 126.978228,37.406642 126.978494,37.407764 126.979033,37.410022 126.979122,37.410444 126.979419,37.411742 126.979725,37.412986 126.979781,37.413208 126.979867,37.413675 126.979908,37.413847 126.980011,37.414200 126.980928,37.416297 126.981547,37.417675 126.981750,37.418144 126.982047,37.418675 126.982469,37.419406 126.982989,37.420228 126.983950,37.421275 126.984853,37.422069 126.985869,37.422925 126.987003,37.423869 126.987781,37.424625 126.988981,37.425722 126.989956,37.426783 126.990411,37.427278 126.991381,37.428233 126.992906,37.429739 126.995314,37.432433 126.995839,37.432964 126.996069,37.433125 126.996378,37.433244 126.996656,37.433317 126.996756,37.433342 126.997372,37.433425 126.997742,37.433425 126.998319,37.433353 126.999225,37.433258 126.999853,37.433225 127.000758,37.433239 127.004331,37.433214 127.004481,37.433256 127.005314,37.433958 127.005433,37.434131 127.006244,37.435392 127.006450,37.435711 127.008228,37.438322 127.008297,37.438447 127.008344,37.438575 127.008364,37.438711 127.008375,37.438969 127.008203,37.440986 127.008019,37.442944 127.007969,37.443506 127.007944,37.443831 127.007819,37.444217 127.007717,37.444408 127.007436,37.444817 127.006928,37.445475 127.004367,37.448900 127.002544,37.451400 127.001972,37.452111 127.001836,37.452281 127.001633,37.452472 127.001442,37.452575 127.000942,37.452764 126.999947,37.453253 126.998028,37.454378 126.997922,37.454442 126.996819,37.455239 126.995569,37.456381 126.994317,37.457522 126.993775,37.458058 126.993728,37.458117 126.993250,37.458667 126.991156,37.461425 126.989422,37.463706 126.989111,37.464247 126.988883,37.464647 126.987964,37.466256 126.987881,37.466394 126.987797,37.466528 126.987708,37.466664 126.987617,37.466794 126.987525,37.466928 126.987428,37.467058 126.987328,37.467186 126.987225,37.467314 126.987122,37.467442 126.987014,37.467567 126.986903,37.467689 126.986792,37.467811 126.986675,37.467931 126.986558,37.468050 126.986439,37.468167 126.986314,37.468283 126.986189,37.468394 126.986061,37.468508 126.985931,37.468619 126.985800,37.468728 126.985664,37.468833 126.985211,37.469178 126.984864,37.469464 126.984636,37.469658 126.984472,37.469808 126.984356,37.469928 126.983858,37.470544 126.983769,37.470672 126.983256,37.471453 126.982953,37.472042 126.982397,37.473183 126.982053,37.474025 126.981878,37.474547 126.981769,37.474950 126.981675,37.475608 126.981669,37.475658 126.981633,37.476083 126.981656,37.476564 126.981658,37.476653 126.982111,37.485253 126.982117,37.485383 126.982122,37.485475 126.982158,37.486308 126.982228,37.487350 126.982233,37.487400 126.982378,37.488686 126.982531,37.489653 126.982592,37.490369 126.982647,37.494967 126.982628,37.496269 126.982639,37.496469 126.982664,37.496669 126.982692,37.496817 126.982744,37.497011 126.982794,37.497156 126.982983,37.497625 126.983100,37.497950 126.983239,37.498383 126.983267,37.498531 126.983264,37.498681 126.983233,37.498825 126.983178,37.498967 126.983067,37.499131 126.981803,37.500881 126.981647,37.501097 126.981553,37.501222 126.981425,37.501369 126.980922,37.501886 126.980856,37.501964 126.980794,37.502033 126.980644,37.502233 126.980547,37.502394 126.980464,37.502575 126.980381,37.502808 126.980294,37.503031 126.980269,37.503125 126.980183,37.503464 126.980156,37.503878 126.980156,37.504058 126.980164,37.504244 126.980186,37.504425 126.980258,37.504800 126.980719,37.506508 126.983114,37.515581 126.983161,37.515808 126.983203,37.516089 126.983231,37.516306 126.983269,37.517844 126.983278,37.518414 126.983261,37.518581 126.983225,37.518736 126.983183,37.518883 126.983128,37.519019 126.983053,37.519164 126.982961,37.519292 126.982853,37.519408 126.982725,37.519531 126.982633,37.519608 126.982606,37.519633 126.982456,37.519739 126.982292,37.519836 126.982108,37.519936 126.981919,37.520033 126.981719,37.520119 126.981503,37.520206 126.976931,37.521594 126.975139,37.522128 126.974444,37.522344 126.972900,37.522833 126.972414,37.522969 126.972056,37.523053 126.971644,37.523133 126.971183,37.523214 126.970789,37.523292 126.970444,37.523367 126.970233,37.523428 126.970031,37.523500 126.969831,37.523583 126.969533,37.523728 126.966744,37.525375 126.966603,37.525464 126.966583,37.525478 126.966564,37.525489 126.966547,37.525503 126.966531,37.525517 126.966514,37.525531 126.966500,37.525547 126.966486,37.525564 126.966475,37.525581 126.966461,37.525594 126.966450,37.525614 126.966442,37.525631 126.966431,37.525650 126.966425,37.525667 126.966417,37.525686 126.966411,37.525703 126.966406,37.525722 126.966403,37.525742 126.966400,37.525761 126.966400,37.525781 126.966400,37.525800 126.966400,37.525819 126.966425,37.527406 126.966428,37.527431 126.966431,37.527456 126.966433,37.527481 126.966439,37.527506 126.966444,37.527528 126.966453,37.527553 126.966458,37.527578 126.966467,37.527600 126.966478,37.527625 126.966486,37.527647 126.966497,37.527669 126.966508,37.527692 126.966522,37.527714 126.966533,37.527739 126.966550,37.527758 126.966564,37.527781 126.966581,37.527803 126.966594,37.527822 126.966614,37.527844 126.966631,37.527864 126.966650,37.527883 126.967931,37.529222 126.968436,37.529753 126.971272,37.532750 126.972633,37.534133 126.973353,37.534864 126.973986,37.535603 126.973989,37.535606 126.974167,37.535861 126.974181,37.535883 126.974194,37.535906 126.974206,37.535928 126.974217,37.535950 126.974228,37.535975 126.974239,37.535997 126.974247,37.536019 126.974256,37.536044 126.974261,37.536069 126.974267,37.536092 126.974272,37.536117 126.974278,37.536142 126.974281,37.536167 126.974281,37.536192 126.974283,37.536217 126.974281,37.536239 126.974281,37.536264 126.974278,37.536289 126.974275,37.536314 126.974269,37.536339 126.974267,37.536364 126.972572,37.542839 126.972311,37.543833 126.972008,37.544989 126.971953,37.545361 126.971900,37.546225 126.971819,37.548156 126.971794,37.548819 126.971797,37.549114 126.971819,37.549381 126.971856,37.549678 126.971928,37.550069 126.972000,37.550425 126.972294,37.551683 126.972739,37.553319 126.972747,37.553344 126.972939,37.553928 126.973025,37.554339 126.973044,37.554517 126.973072,37.555861 126.973075,37.555892 126.973081,37.555922 126.973089,37.555953 126.973097,37.555981 126.973108,37.556008 126.973122,37.556039 126.973136,37.556067 126.973153,37.556094 126.973172,37.556119 126.973192,37.556144 126.973214,37.556169 126.973236,37.556194 126.973261,37.556219 126.973289,37.556242 126.973314,37.556261 126.973344,37.556281 126.973375,37.556300 126.973406,37.556317 126.973436,37.556333 126.973469,37.556350 126.973506,37.556361 126.973986,37.556536 126.977178,37.557736 126.977364,37.557817 126.977617,37.557967 126.977814,37.558117 126.978078,37.558394 126.978444,37.558789 126.978564,37.558883 126.978656,37.558956 126.978692,37.558981 126.978731,37.559006 126.978769,37.559028 126.978806,37.559050 126.978844,37.559075 126.978886,37.559094 126.978925,37.559117 126.978967,37.559139 126.979006,37.559158 126.979047,37.559178 126.979089,37.559197 126.979131,37.559217 126.979172,37.559233 126.979217,37.559253 126.979258,37.559269 126.979303,37.559286 126.979344,37.559303 126.979389,37.559319 126.979433,37.559333 126.979478,37.559350 126.979522,37.559364 126.980778,37.559744 126.980817,37.559756 126.980856,37.559769 126.980894,37.559783 126.980933,37.559797 126.980969,37.559811 126.981008,37.559825 126.981044,37.559839 126.981083,37.559856 126.981119,37.559869 126.981158,37.559886 126.981194,37.559903 126.981231,37.559919 126.981267,37.559936 126.981303,37.559953 126.981339,37.559972 126.981375,37.559989 126.981408,37.560008 126.981444,37.560028 126.981478,37.560044 126.981511,37.560067 126.981544,37.560083 126.981869,37.560261 126.982136,37.560417 126.982164,37.560431 126.982189,37.560444 126.982217,37.560456 126.982244,37.560469 126.982269,37.560481 126.982300,37.560494 126.982328,37.560506 126.982356,37.560514 126.982383,37.560525 126.982414,37.560533 126.982442,37.560544 126.982472,37.560553 126.982503,37.560561 126.982531,37.560569 126.982561,37.560575 126.982592,37.560583 126.982622,37.560589 126.982653,37.560594 126.982683,37.560600 126.982714,37.560603 126.982744,37.560608 126.984639,37.560767 126.986489,37.560978 126.986525,37.560983 126.988339,37.561144 126.989236,37.561203 126.989272,37.561206 126.989308,37.561206 126.989344,37.561208 126.989381,37.561208 126.989417,37.561211 126.989453,37.561211 126.989489,37.561211 126.989525,37.561208 126.989561,37.561208 126.989597,37.561208 126.989633,37.561206 126.989669,37.561203 126.989706,37.561203 126.989742,37.561200 126.989778,37.561194 126.989814,37.561192 126.989850,37.561189 126.989886,37.561183 126.989922,37.561178 126.989958,37.561175 126.989994,37.561167 126.990364,37.561100 126.990394,37.561094 126.990431,37.561092 126.990464,37.561086 126.990478,37.561083 126.990497,37.561083 126.990531,37.561078 126.990564,37.561075 126.990597,37.561072 126.990631,37.561069 126.990667,37.561067 126.990700,37.561064 126.990733,37.561061 126.990767,37.561058 126.990800,37.561058 126.990833,37.561056 126.990869,37.561056 126.990903,37.561053 126.990936,37.561053 126.990969,37.561053 126.991006,37.561053 126.991039,37.561053 126.991072,37.561053 126.992567,37.561075 126.993236,37.561078 126.993447,37.561100 126.994111,37.561242 126.994172,37.561256 126.995656,37.561569 127.004903,37.563719 127.005561,37.563858 127.006442,37.564022 127.006633,37.564058 127.006664,37.564067 127.006692,37.564075 127.006719,37.564081 127.006750,37.564092 127.006778,37.564100 127.006803,37.564111 127.006831,37.564122 127.006856,37.564133 127.006883,37.564147 127.006908,37.564158 127.006931,37.564175 127.006956,37.564189 127.006978,37.564206 127.007000,37.564219 127.007014,37.564231 127.007022,37.564239 127.007044,37.564256 127.007064,37.564272 127.007083,37.564292 127.007100,37.564311 127.007119,37.564331 127.007133,37.564350 127.007333,37.564619 127.007356,37.564644 127.008444,37.565839 127.008606,37.566017 127.008781,37.566247 127.008933,37.566497 127.009106,37.566825 127.009281,37.567164 127.009450,37.567483 127.009697,37.567964 127.009717,37.568006 127.009733,37.568050 127.009747,37.568092 127.009761,37.568133 127.009775,37.568178 127.009783,37.568219 127.009794,37.568264 127.009803,37.568308 127.009808,37.568353 127.009814,37.568397 127.009817,37.568442 127.009819,37.568483 127.009819,37.568528 127.009817,37.568575 127.009814,37.568617 127.009811,37.568661 127.009742,37.569244 127.009617,37.570203 127.009486,37.570547 127.009258,37.570911 127.009114,37.571139"
              }
            },
            {
              "mode": "WALK",
              "sectionTime": 400,
              "distance": 488,
              "start": {
                "name": "동대문",
                "lon": 127.00916111111111,
                "lat": 37.57105
              },
              "end": {
                "name": "동대문역.흥인지문",
                "lon": 127.01341666666667,
                "lat": 37.57225
              },
              "passShape": {
                "linestring": "127.009161,37.571050 127.009297,37.570950 127.009336,37.570900 127.009422,37.570789 127.009567,37.570453 127.009753,37.570547 127.010036,37.570781 127.010081,37.570789 127.010328,37.571033 127.010506,37.571222 127.010594,37.571322 127.010717,37.571475 127.010856,37.571600 127.010958,37.571617 127.011017,37.571625 127.011164,37.571642 127.011600,37.571736 127.011700,37.571739 127.011753,37.571750 127.011925,37.571786 127.012125,37.571831 127.012175,37.571844 127.012200,37.571867 127.012286,37.571886 127.012392,37.571911 127.012675,37.571983 127.013014,37.572086 127.013417,37.572250"
              }
            },
            {
              "mode": "BUS",
              "routeColor": "0068B7",
              "sectionTime": 927,
              "route": "간선:720",
              "routeId": "2870001",
              "distance": 3254,
              "service": 1,
              "start": {
                "name": "동대문역.흥인지문",
                "lon": 127.01341666666667,
                "lat": 37.57225
              },
              "passStopList": {
                "stationList": [
                  {
                    "index": 0,
                    "stationName": "동대문역.흥인지문",
                    "lon": "127.013417",
                    "lat": "37.572250",
                    "stationID": "775153"
                  },
                  {
                    "index": 1,
                    "stationName": "동묘앞",
                    "lon": "127.018736",
                    "lat": "37.573803",
                    "stationID": "775214"
                  },
                  {
                    "index": 2,
                    "stationName": "신설동역",
                    "lon": "127.025250",
                    "lat": "37.576150",
                    "stationID": "775282"
                  },
                  {
                    "index": 3,
                    "stationName": "용두동사거리",
                    "lon": "127.030111",
                    "lat": "37.577539",
                    "stationID": "775322"
                  },
                  {
                    "index": 4,
                    "stationName": "제기동역.서울약령시",
                    "lon": "127.037442",
                    "lat": "37.578072",
                    "stationID": "775336"
                  },
                  {
                    "index": 5,
                    "stationName": "청량리수산시장",
                    "lon": "127.041825",
                    "lat": "37.579014",
                    "stationID": "775365"
                  },
                  {
                    "index": 6,
                    "stationName": "청량리역환승센터(3번승강장)",
                    "lon": "127.045953",
                    "lat": "37.580481",
                    "stationID": "775408"
                  },
                  {
                    "index": 7,
                    "stationName": "청량리미주상가앞",
                    "lon": "127.047603",
                    "lat": "37.581906",
                    "stationID": "775453"
                  }
                ]
              },
              "end": {
                "name": "청량리미주상가앞",
                "lon": 127.04760277777778,
                "lat": 37.58190555555556
              },
              "type": 11,
              "passShape": {
                "linestring": "127.013406,37.572283 127.015558,37.572922 127.015733,37.572972 127.017636,37.573467 127.018711,37.573833 127.020483,37.574436 127.022508,37.575214 127.023142,37.575464 127.023414,37.575578 127.023914,37.575775 127.025200,37.576175 127.025228,37.576183 127.028344,37.577056 127.030089,37.577569 127.030400,37.577661 127.030581,37.577714 127.031100,37.577856 127.031703,37.577994 127.032378,37.578072 127.036372,37.578106 127.037419,37.578106 127.038231,37.578103 127.038414,37.578111 127.038728,37.578144 127.039303,37.578253 127.041822,37.579050 127.042503,37.579267 127.042672,37.579319 127.045083,37.580072 127.045256,37.580131 127.045558,37.580256 127.045828,37.580419 127.045908,37.580489 127.046206,37.580744 127.046731,37.581264 127.047583,37.581939"
              }
            },
            {
              "mode": "WALK",
              "sectionTime": 352,
              "distance": 452,
              "start": {
                "name": "청량리미주상가앞",
                "lon": 127.04760277777778,
                "lat": 37.58190555555556
              },
              "end": {
                "name": "도착지",
                "lon": 127.048958,
                "lat": 37.581381
              },
              "steps": [
                {
                  "streetName": "왕산로",
                  "distance": 78,
                  "description": "왕산로 을 따라 78m 이동",
                  "linestring": "127.04762,37.581886 127.04735,37.581673 127.04703,37.581406 127.04703,37.581375"
                },
                {
                  "streetName": "",
                  "distance": 16,
                  "description": "우측 횡단보도 후 16m 이동 ",
                  "linestring": "127.04982,37.581577 127.04971,37.58145"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 53,
                  "description": "우회전 후 보행자도로 을 따라 53m 이동 ",
                  "linestring": "127.04971,37.58145 127.04921,37.581703"
                }
              ]
            }
          ],
          "totalWalkTime": 853,
          "transferCount": 1,
          "totalDistance": 47144,
          "pathType": 3,
          "totalWalkDistance": 1061
        },
        {
          "fare": {
            "regular": {
              "totalFare": 2200,
              "currency": {
                "symbol": "￦",
                "currency": "원",
                "currencyCode": "KRW"
              }
            }
          },
          "totalTime": 7520,
          "legs": [
            {
              "mode": "WALK",
              "sectionTime": 101,
              "distance": 121,
              "start": {
                "name": "출발지",
                "lon": 126.8230369,
                "lat": 37.3168267
              },
              "end": {
                "name": "고잔",
                "lon": 126.82284444444444,
                "lat": 37.31684444444444
              },
              "steps": [
                {
                  "streetName": "",
                  "distance": 27,
                  "description": "27m 이동",
                  "linestring": "126.82303,37.3168 126.82306,37.3168 126.82333,37.316788"
                },
                {
                  "streetName": "",
                  "distance": 15,
                  "description": "고잔역 에서 좌회전 후 15m 이동 ",
                  "linestring": "126.82333,37.316788 126.82334,37.31693"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 20,
                  "description": "직진 후 보행자도로 을 따라 20m 이동 ",
                  "linestring": "126.82334,37.31693 126.82336,37.317104"
                },
                {
                  "streetName": "중앙대로",
                  "distance": 39,
                  "description": "좌회전 후 중앙대로 을 따라 39m 이동 ",
                  "linestring": "126.82336,37.317104 126.822914,37.317135"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 20,
                  "description": "좌회전 후 보행자도로 을 따라 20m 이동 ",
                  "linestring": "126.822914,37.317135 126.82289,37.316963"
                }
              ]
            },
            {
              "mode": "SUBWAY",
              "routeColor": "F5A200",
              "sectionTime": 7080,
              "route": "수인분당선",
              "routeId": "110101008",
              "distance": 73466,
              "service": 1,
              "start": {
                "name": "고잔",
                "lon": 126.82284444444444,
                "lat": 37.31684444444444
              },
              "passStopList": {
                "stationList": [
                  {
                    "index": 0,
                    "stationName": "고잔",
                    "lon": "126.822844",
                    "lat": "37.316844",
                    "stationID": "111045"
                  },
                  {
                    "index": 1,
                    "stationName": "중앙",
                    "lon": "126.838289",
                    "lat": "37.316067",
                    "stationID": "111044"
                  },
                  {
                    "index": 2,
                    "stationName": "한대앞",
                    "lon": "126.853231",
                    "lat": "37.310175",
                    "stationID": "111043"
                  },
                  {
                    "index": 3,
                    "stationName": "사리",
                    "lon": "126.857175",
                    "lat": "37.290869",
                    "stationID": "111042"
                  },
                  {
                    "index": 4,
                    "stationName": "야목",
                    "lon": "126.883808",
                    "lat": "37.261422",
                    "stationID": "111041"
                  },
                  {
                    "index": 5,
                    "stationName": "어천",
                    "lon": "126.908844",
                    "lat": "37.250086",
                    "stationID": "111040"
                  },
                  {
                    "index": 6,
                    "stationName": "오목천",
                    "lon": "126.963919",
                    "lat": "37.243394",
                    "stationID": "111039"
                  },
                  {
                    "index": 7,
                    "stationName": "고색",
                    "lon": "126.980594",
                    "lat": "37.249794",
                    "stationID": "111038"
                  },
                  {
                    "index": 8,
                    "stationName": "수원",
                    "lon": "127.000950",
                    "lat": "37.266575",
                    "stationID": "111037"
                  },
                  {
                    "index": 9,
                    "stationName": "매교",
                    "lon": "127.015772",
                    "lat": "37.265500",
                    "stationID": "111036"
                  },
                  {
                    "index": 10,
                    "stationName": "수원시청",
                    "lon": "127.031069",
                    "lat": "37.261867",
                    "stationID": "111035"
                  },
                  {
                    "index": 11,
                    "stationName": "매탄권선",
                    "lon": "127.040747",
                    "lat": "37.252569",
                    "stationID": "111034"
                  },
                  {
                    "index": 12,
                    "stationName": "망포",
                    "lon": "127.057606",
                    "lat": "37.245817",
                    "stationID": "111033"
                  },
                  {
                    "index": 13,
                    "stationName": "영통",
                    "lon": "127.071256",
                    "lat": "37.251428",
                    "stationID": "111032"
                  },
                  {
                    "index": 14,
                    "stationName": "청명",
                    "lon": "127.079056",
                    "lat": "37.259583",
                    "stationID": "111031"
                  },
                  {
                    "index": 15,
                    "stationName": "상갈",
                    "lon": "127.108728",
                    "lat": "37.261803",
                    "stationID": "111030"
                  },
                  {
                    "index": 16,
                    "stationName": "기흥",
                    "lon": "127.115950",
                    "lat": "37.275625",
                    "stationID": "111029"
                  },
                  {
                    "index": 17,
                    "stationName": "신갈",
                    "lon": "127.111278",
                    "lat": "37.286169",
                    "stationID": "111028"
                  },
                  {
                    "index": 18,
                    "stationName": "구성",
                    "lon": "127.105631",
                    "lat": "37.299019",
                    "stationID": "111027"
                  },
                  {
                    "index": 19,
                    "stationName": "보정",
                    "lon": "127.108306",
                    "lat": "37.312403",
                    "stationID": "111025"
                  },
                  {
                    "index": 20,
                    "stationName": "죽전",
                    "lon": "127.107386",
                    "lat": "37.324403",
                    "stationID": "111026"
                  },
                  {
                    "index": 21,
                    "stationName": "오리",
                    "lon": "127.108994",
                    "lat": "37.339828",
                    "stationID": "111024"
                  },
                  {
                    "index": 22,
                    "stationName": "미금",
                    "lon": "127.108961",
                    "lat": "37.349842",
                    "stationID": "111023"
                  },
                  {
                    "index": 23,
                    "stationName": "정자",
                    "lon": "127.108103",
                    "lat": "37.366036",
                    "stationID": "111022"
                  },
                  {
                    "index": 24,
                    "stationName": "수내",
                    "lon": "127.114247",
                    "lat": "37.378464",
                    "stationID": "111021"
                  },
                  {
                    "index": 25,
                    "stationName": "서현",
                    "lon": "127.123383",
                    "lat": "37.385019",
                    "stationID": "111020"
                  },
                  {
                    "index": 26,
                    "stationName": "이매",
                    "lon": "127.128267",
                    "lat": "37.394975",
                    "stationID": "111019"
                  },
                  {
                    "index": 27,
                    "stationName": "야탑",
                    "lon": "127.128672",
                    "lat": "37.411319",
                    "stationID": "111018"
                  },
                  {
                    "index": 28,
                    "stationName": "모란",
                    "lon": "127.129150",
                    "lat": "37.431644",
                    "stationID": "111017"
                  },
                  {
                    "index": 29,
                    "stationName": "태평",
                    "lon": "127.127794",
                    "lat": "37.439642",
                    "stationID": "111016"
                  },
                  {
                    "index": 30,
                    "stationName": "가천대",
                    "lon": "127.126719",
                    "lat": "37.448822",
                    "stationID": "111015"
                  },
                  {
                    "index": 31,
                    "stationName": "복정",
                    "lon": "127.126689",
                    "lat": "37.470275",
                    "stationID": "111014"
                  },
                  {
                    "index": 32,
                    "stationName": "수서",
                    "lon": "127.100922",
                    "lat": "37.487797",
                    "stationID": "111013"
                  },
                  {
                    "index": 33,
                    "stationName": "대모산입구",
                    "lon": "127.072750",
                    "lat": "37.491369",
                    "stationID": "111012"
                  },
                  {
                    "index": 34,
                    "stationName": "개포동",
                    "lon": "127.066042",
                    "lat": "37.489081",
                    "stationID": "111011"
                  },
                  {
                    "index": 35,
                    "stationName": "구룡",
                    "lon": "127.059747",
                    "lat": "37.486956",
                    "stationID": "111010"
                  },
                  {
                    "index": 36,
                    "stationName": "도곡",
                    "lon": "127.055450",
                    "lat": "37.490764",
                    "stationID": "111009"
                  },
                  {
                    "index": 37,
                    "stationName": "한티",
                    "lon": "127.052853",
                    "lat": "37.496294",
                    "stationID": "111008"
                  },
                  {
                    "index": 38,
                    "stationName": "선릉",
                    "lon": "127.048661",
                    "lat": "37.505253",
                    "stationID": "111007"
                  },
                  {
                    "index": 39,
                    "stationName": "선정릉",
                    "lon": "127.043678",
                    "lat": "37.510825",
                    "stationID": "111006"
                  },
                  {
                    "index": 40,
                    "stationName": "강남구청",
                    "lon": "127.041203",
                    "lat": "37.517428",
                    "stationID": "111005"
                  },
                  {
                    "index": 41,
                    "stationName": "압구정로데오",
                    "lon": "127.040758",
                    "lat": "37.527881",
                    "stationID": "111004"
                  },
                  {
                    "index": 42,
                    "stationName": "서울숲",
                    "lon": "127.044747",
                    "lat": "37.543767",
                    "stationID": "111003"
                  },
                  {
                    "index": 43,
                    "stationName": "왕십리",
                    "lon": "127.038769",
                    "lat": "37.561186",
                    "stationID": "111002"
                  },
                  {
                    "index": 44,
                    "stationName": "청량리",
                    "lon": "127.047519",
                    "lat": "37.580117",
                    "stationID": "111001"
                  }
                ]
              },
              "end": {
                "name": "청량리",
                "lon": 127.04751944444445,
                "lat": 37.58011666666667
              },
              "type": 100,
              "passShape": {
                "linestring": "126.822844,37.316844 126.823047,37.316833 126.824414,37.316767 126.826906,37.316628 126.830992,37.316458 126.832469,37.316378 126.833967,37.316292 126.835467,37.316217 126.836961,37.316139 126.838481,37.316061 126.839886,37.315986 126.842467,37.315819 126.844714,37.315669 126.845500,37.315581 126.846008,37.315494 126.846500,37.315406 126.846953,37.315281 126.847328,37.315158 126.847739,37.314989 126.848297,37.314700 126.848778,37.314403 126.849225,37.314086 126.849656,37.313689 126.850683,37.312733 126.851578,37.311858 126.852081,37.311344 126.852964,37.310425 126.853639,37.309722 126.857042,37.306233 126.857961,37.305256 126.858433,37.304658 126.858800,37.303908 126.859031,37.303219 126.859119,37.302611 126.859083,37.301506 126.859050,37.299936 126.858964,37.298331 126.858853,37.297417 126.858647,37.296117 126.858286,37.294664 126.857967,37.293525 126.857356,37.291475 126.857067,37.290503 126.855928,37.286456 126.855625,37.285378 126.855072,37.283472 126.854861,37.282600 126.854819,37.282008 126.854811,37.281367 126.854864,37.280722 126.854969,37.280125 126.855075,37.279653 126.855217,37.279189 126.855394,37.278703 126.855600,37.278289 126.855914,37.277786 126.856253,37.277364 126.857175,37.276200 126.858897,37.274058 126.860525,37.272028 126.860981,37.271494 126.861539,37.270925 126.862000,37.270492 126.862642,37.269981 126.863261,37.269528 126.864081,37.269103 126.864817,37.268808 126.866214,37.268414 126.870000,37.267636 126.872900,37.266975 126.874500,37.266633 126.875600,37.266386 126.876200,37.266203 126.876800,37.265956 126.877450,37.265617 126.879475,37.264278 126.881128,37.263192 126.883808,37.261422 126.885853,37.260058 126.888706,37.258181 126.889592,37.257583 126.890183,37.257214 126.890842,37.256800 126.891397,37.256467 126.891828,37.256308 126.893761,37.255614 126.898444,37.253897 126.900019,37.253308 126.902453,37.252444 126.906222,37.251044 126.907603,37.250547 126.908844,37.250086 126.913253,37.248481 126.914469,37.248044 126.915219,37.247831 126.915794,37.247703 126.916800,37.247564 126.920181,37.247083 126.923203,37.246656 126.929417,37.245783 126.933169,37.245242 126.935294,37.244942 126.936831,37.244731 126.938022,37.244525 126.939158,37.244278 126.940306,37.244014 126.941000,37.243825 126.943681,37.243133 126.949300,37.241669 126.950439,37.241356 126.951372,37.241142 126.952175,37.240978 126.952922,37.240878 126.953514,37.240831 126.954011,37.240825 126.954733,37.240867 126.955381,37.240931 126.956133,37.241053 126.958014,37.241397 126.959164,37.241583 126.960019,37.241761 126.960803,37.241933 126.961417,37.242108 126.962178,37.242381 126.962897,37.242725 126.963331,37.243003 126.963919,37.243394 126.964656,37.243867 126.965189,37.244211 126.965964,37.244650 126.966631,37.244975 126.967106,37.245164 126.968292,37.245567 126.970497,37.246325 126.974400,37.247667 126.980594,37.249794 126.983372,37.250742 126.986275,37.251742 126.987828,37.252258 126.989394,37.252814 126.989925,37.253028 126.990311,37.253172 126.990675,37.253372 126.991086,37.253622 126.991467,37.253928 126.991775,37.254242 126.992022,37.254578 126.992206,37.254878 126.992397,37.255269 126.992544,37.255650 126.992856,37.256567 126.993406,37.258167 126.993894,37.259547 126.994764,37.262017 126.994950,37.262494 126.995125,37.262956 126.995431,37.263525 126.995725,37.263972 126.996019,37.264272 126.996514,37.264683 126.997033,37.265017 126.997725,37.265336 126.998569,37.265678 127.000078,37.266167 127.000094,37.266200 127.000392,37.266375 127.000911,37.266558 127.007378,37.268950 127.008194,37.269244 127.008319,37.269264 127.008467,37.269264 127.008603,37.269242 127.008739,37.269192 127.008842,37.269111 127.008919,37.269011 127.009469,37.268089 127.009672,37.267819 127.009786,37.267697 127.009900,37.267589 127.010031,37.267494 127.010161,37.267408 127.010428,37.267242 127.011006,37.266914 127.011658,37.266556 127.011950,37.266428 127.012272,37.266303 127.012575,37.266200 127.012931,37.266092 127.013767,37.265883 127.014106,37.265817 127.014581,37.265736 127.015772,37.265500 127.016075,37.265439 127.017550,37.265103 127.019181,37.264728 127.019503,37.264664 127.019789,37.264614 127.020197,37.264581 127.020403,37.264564 127.020619,37.264539 127.020925,37.264456 127.024492,37.263528 127.025331,37.263339 127.031069,37.261864 127.033017,37.261364 127.034939,37.260864 127.035106,37.260803 127.035233,37.260725 127.035378,37.260631 127.035464,37.260544 127.035594,37.260356 127.035692,37.260147 127.035831,37.259514 127.036231,37.257267 127.036294,37.257125 127.036358,37.257019 127.036447,37.256892 127.036528,37.256797 127.036631,37.256714 127.036722,37.256644 127.036831,37.256581 127.036939,37.256528 127.037642,37.256294 127.038278,37.256078 127.038400,37.256025 127.038483,37.255967 127.038558,37.255908 127.038619,37.255844 127.038850,37.255519 127.040108,37.253494 127.040469,37.252958 127.040750,37.252569 127.040842,37.252442 127.041108,37.252100 127.041389,37.251778 127.041622,37.251556 127.041914,37.251344 127.042708,37.250839 127.048981,37.246694 127.050153,37.245919 127.050631,37.245594 127.050886,37.245519 127.051153,37.245506 127.052989,37.245586 127.056128,37.245753 127.057606,37.245817 127.058603,37.245858 127.060064,37.245928 127.062103,37.246019 127.064308,37.246244 127.065331,37.246361 127.065967,37.246456 127.066483,37.246633 127.066814,37.246844 127.067053,37.247047 127.067286,37.247289 127.070081,37.250144 127.071044,37.251194 127.071911,37.252142 127.075672,37.256036 127.078903,37.259419 127.079056,37.259583 127.079775,37.260367 127.079922,37.260492 127.080058,37.260611 127.080217,37.260731 127.080419,37.260831 127.080681,37.260903 127.081031,37.260967 127.081447,37.260994 127.082106,37.260989 127.087742,37.260819 127.089250,37.260775 127.090731,37.260756 127.093814,37.260778 127.096875,37.260931 127.099789,37.261186 127.101239,37.261264 127.102644,37.261358 127.104069,37.261503 127.105278,37.261556 127.106333,37.261600 127.106878,37.261644 127.107478,37.261731 127.107794,37.261767 127.108728,37.261803 127.111856,37.261925 127.112603,37.262056 127.113292,37.262281 127.113942,37.262567 127.114525,37.262953 127.114994,37.263339 127.115383,37.263764 127.115678,37.264247 127.115881,37.264792 127.115989,37.265456 127.116017,37.266158 127.115983,37.270875 127.115961,37.274794 127.115958,37.275008 127.115953,37.275483 127.115917,37.278300 127.115889,37.280372 127.115861,37.281458 127.115722,37.281931 127.115567,37.282306 127.115303,37.282642 127.115106,37.282853 127.114717,37.283250 127.112603,37.285075 127.112136,37.285472 127.111294,37.286144 127.111269,37.286164 127.110481,37.286814 127.108514,37.288464 127.108008,37.288922 127.107550,37.289408 127.107139,37.289922 127.106778,37.290458 127.106617,37.290733 127.106336,37.291303 127.106108,37.291883 127.105939,37.292478 127.105825,37.293081 127.105769,37.293689 127.105692,37.294556 127.105669,37.299019 127.105667,37.299800 127.105933,37.304906 127.105981,37.306047 127.105997,37.306219 127.106031,37.306422 127.106111,37.306697 127.106189,37.306900 127.106342,37.307203 127.106481,37.307419 127.106617,37.307578 127.106772,37.307736 127.107625,37.308489 127.107828,37.308706 127.108008,37.308936 127.108139,37.309150 127.108333,37.309556 127.108425,37.309792 127.108447,37.309892 127.108456,37.310008 127.108461,37.310244 127.108478,37.310669 127.108478,37.311033 127.108478,37.311306 127.108467,37.311558 127.108442,37.311747 127.108414,37.311914 127.108306,37.312403 127.108078,37.313442 127.107308,37.317222 127.107281,37.317372 127.107258,37.317522 127.107244,37.317672 127.107236,37.317825 127.107233,37.317972 127.107253,37.320481 127.107253,37.320544 127.107253,37.320608 127.107256,37.320672 127.107258,37.320739 127.107339,37.321600 127.107381,37.322467 127.107386,37.323342 127.107389,37.324281 127.107392,37.324403 127.107456,37.326669 127.107456,37.326672 127.107489,37.327492 127.107500,37.327778 127.107500,37.327828 127.107519,37.328414 127.107542,37.329075 127.107558,37.329792 127.107592,37.330328 127.107797,37.331117 127.108864,37.335342 127.108964,37.336331 127.108992,37.336869 127.108994,37.339828 127.109000,37.344864 127.108964,37.349667 127.108961,37.349842 127.108931,37.353142 127.108917,37.353683 127.108881,37.354378 127.108858,37.354642 127.108817,37.354944 127.108228,37.358803 127.108203,37.359011 127.108178,37.359292 127.108161,37.359522 127.108106,37.361117 127.108097,37.366036 127.108094,37.367483 127.108089,37.368414 127.108078,37.370103 127.108033,37.372003 127.108003,37.373217 127.108000,37.373292 127.108003,37.373364 127.108008,37.373436 127.108019,37.373508 127.108033,37.373581 127.108053,37.373653 127.108075,37.373725 127.108103,37.373794 127.108131,37.373864 127.108164,37.373931 127.108200,37.373997 127.108242,37.374064 127.108286,37.374128 127.108333,37.374189 127.108383,37.374250 127.108433,37.374303 127.108439,37.374308 127.108494,37.374367 127.108556,37.374422 127.108617,37.374475 127.108683,37.374525 127.108753,37.374575 127.112292,37.377053 127.112794,37.377414 127.113539,37.377956 127.114253,37.378458 127.115800,37.379544 127.120583,37.382997 127.122886,37.384678 127.123017,37.384767 127.124150,37.385544 127.124878,37.386053 127.125422,37.386444 127.127225,37.387747 127.127308,37.387814 127.127392,37.387883 127.127472,37.387956 127.127547,37.388031 127.127619,37.388108 127.127689,37.388189 127.127753,37.388267 127.127814,37.388350 127.127872,37.388433 127.127925,37.388519 127.127975,37.388608 127.128019,37.388697 127.128061,37.388789 127.128100,37.388881 127.128131,37.388972 127.128158,37.389064 127.128183,37.389158 127.128203,37.389253 127.128219,37.389350 127.128228,37.389444 127.128236,37.389542 127.128225,37.390978 127.128261,37.394692 127.128261,37.394697 127.128289,37.396353 127.128311,37.396606 127.128339,37.396783 127.128375,37.396944 127.128408,37.397053 127.128439,37.397161 127.128467,37.397269 127.128494,37.397378 127.128519,37.397483 127.128544,37.397594 127.128567,37.397703 127.128586,37.397811 127.128606,37.397922 127.128622,37.398031 127.128636,37.398142 127.128650,37.398250 127.128658,37.398361 127.128669,37.398469 127.128678,37.398581 127.128683,37.398692 127.128686,37.398803 127.128689,37.398914 127.128689,37.399022 127.128686,37.399133 127.128683,37.399244 127.128300,37.402569 127.128289,37.402631 127.128283,37.402694 127.128275,37.402756 127.128269,37.402819 127.128264,37.402881 127.128258,37.402944 127.128256,37.403008 127.128253,37.403069 127.128253,37.403133 127.128253,37.403194 127.128253,37.403258 127.128253,37.403319 127.128256,37.403383 127.128258,37.403447 127.128261,37.403508 127.128267,37.403572 127.128272,37.403633 127.128278,37.403697 127.128286,37.403758 127.128294,37.403822 127.128303,37.403883 127.128567,37.405172 127.128625,37.405708 127.128681,37.409783 127.128672,37.411319 127.128664,37.413050 127.128667,37.413206 127.128675,37.413547 127.128683,37.413908 127.128767,37.422553 127.128797,37.424569 127.128803,37.424658 127.128806,37.424747 127.128814,37.424836 127.128819,37.424928 127.128828,37.425017 127.128839,37.425106 127.128850,37.425194 127.128858,37.425281 127.128875,37.425369 127.128889,37.425458 127.128903,37.425547 127.128919,37.425636 127.128939,37.425725 127.128956,37.425814 127.128978,37.425900 127.128997,37.425989 127.129019,37.426075 127.129044,37.426164 127.129069,37.426250 127.129094,37.426339 127.129122,37.426425 127.129258,37.427219 127.129306,37.427464 127.129322,37.427561 127.129353,37.427747 127.129375,37.427967 127.129411,37.428233 127.129431,37.428403 127.129433,37.428511 127.129444,37.428667 127.129339,37.430136 127.129139,37.431644 127.128925,37.433239 127.128897,37.433417 127.128850,37.433689 127.127814,37.439558 127.127797,37.439642 127.127028,37.443683 127.126956,37.444222 127.126800,37.446225 127.126767,37.446767 127.126714,37.448150 127.126689,37.448822 127.126586,37.451456 127.126481,37.454989 127.126475,37.455069 127.126467,37.455150 127.126458,37.455228 127.126453,37.455306 127.126442,37.455386 127.126431,37.455467 127.126419,37.455544 127.126408,37.455625 127.126394,37.455703 127.126381,37.455781 127.126367,37.455861 127.126350,37.455939 127.126333,37.456017 127.126314,37.456094 127.126294,37.456175 127.126275,37.456253 127.126256,37.456331 127.126233,37.456408 127.126211,37.456486 127.126189,37.456564 127.126164,37.456639 127.126119,37.456808 127.126081,37.456975 127.126042,37.457144 127.126006,37.457314 127.125972,37.457483 127.125942,37.457653 127.125914,37.457822 127.125889,37.457992 127.125867,37.458164 127.125847,37.458333 127.125831,37.458503 127.125822,37.458608 127.125817,37.458675 127.125806,37.458844 127.125797,37.459017 127.125792,37.459189 127.125786,37.459358 127.125786,37.459531 127.125789,37.459703 127.125792,37.459872 127.125800,37.460044 127.125808,37.460217 127.126689,37.470275 127.126792,37.471467 127.126675,37.472189 127.126475,37.473072 127.126306,37.473578 127.126058,37.474133 127.125708,37.474664 127.125128,37.475242 127.124489,37.475650 127.123656,37.475986 127.122969,37.476153 127.122092,37.476331 127.116081,37.477411 127.114942,37.477650 127.114306,37.477806 127.113714,37.478008 127.113197,37.478236 127.112558,37.478597 127.112025,37.478933 127.111203,37.479583 127.108981,37.481389 127.106164,37.483675 127.104106,37.485228 127.103631,37.485583 127.102117,37.486831 127.101814,37.487078 127.101531,37.487306 127.100183,37.488394 127.100075,37.488481 127.099969,37.488564 127.099867,37.488650 127.099767,37.488739 127.099667,37.488828 127.099567,37.488919 127.099472,37.489011 127.099378,37.489103 127.099286,37.489197 127.099194,37.489294 127.099108,37.489392 127.099022,37.489489 127.098939,37.489589 127.098856,37.489689 127.098775,37.489789 127.098700,37.489892 127.098622,37.489994 127.098550,37.490100 127.098478,37.490203 127.098408,37.490308 127.098344,37.490417 127.097850,37.491250 127.097653,37.491572 127.096989,37.492733 127.096506,37.493431 127.096436,37.493508 127.096364,37.493589 127.096289,37.493664 127.096214,37.493739 127.096133,37.493814 127.096050,37.493886 127.095967,37.493956 127.095881,37.494025 127.095792,37.494089 127.095700,37.494156 127.095606,37.494217 127.095511,37.494278 127.095414,37.494336 127.095314,37.494392 127.095214,37.494447 127.095111,37.494500 127.095006,37.494550 127.094900,37.494597 127.094794,37.494644 127.094683,37.494686 127.094575,37.494728 127.089294,37.496542 127.089144,37.496583 127.088972,37.496606 127.088817,37.496603 127.088658,37.496575 127.085114,37.495456 127.079519,37.493650 127.074964,37.492178 127.074756,37.492108 127.073314,37.491606 127.072706,37.491353 127.072583,37.491289 127.072303,37.491139 127.071056,37.490358 127.070972,37.490311 127.070886,37.490267 127.070800,37.490222 127.070714,37.490178 127.070625,37.490136 127.070539,37.490094 127.070447,37.490056 127.070358,37.490017 127.070267,37.489978 127.070178,37.489939 127.070083,37.489903 127.069992,37.489867 127.069900,37.489833 127.069806,37.489800 127.069711,37.489769 127.069617,37.489739 127.069522,37.489708 127.069425,37.489678 127.069331,37.489650 127.069233,37.489622 127.069136,37.489597 127.068953,37.489558 127.068750,37.489519 127.067144,37.489281 127.066494,37.489181 127.066267,37.489131 127.065753,37.489017 127.065425,37.488931 127.064125,37.488514 127.064072,37.488492 127.064019,37.488472 127.063967,37.488450 127.063914,37.488428 127.063864,37.488406 127.063811,37.488383 127.063761,37.488358 127.063708,37.488333 127.063658,37.488308 127.063611,37.488283 127.063561,37.488256 127.063514,37.488228 127.063467,37.488200 127.063419,37.488172 127.063372,37.488142 127.063328,37.488114 127.063281,37.488083 127.063236,37.488053 127.063192,37.488019 127.063150,37.487989 127.063106,37.487956 127.062814,37.487717 127.062783,37.487694 127.062756,37.487675 127.062725,37.487656 127.062694,37.487636 127.062664,37.487617 127.062633,37.487600 127.062603,37.487581 127.062569,37.487564 127.062539,37.487547 127.062506,37.487531 127.062475,37.487514 127.062442,37.487497 127.062406,37.487483 127.062372,37.487469 127.062339,37.487456 127.062306,37.487442 127.062269,37.487428 127.062233,37.487414 127.062200,37.487403 127.062164,37.487392 127.062128,37.487378 127.061272,37.487097 127.061058,37.487042 127.060900,37.487011 127.060706,37.486981 127.060483,37.486956 127.060258,37.486939 127.059967,37.486928 127.059747,37.486956 127.059306,37.487014 127.058564,37.487175 127.058383,37.487225 127.058206,37.487281 127.058031,37.487342 127.057858,37.487411 127.057694,37.487486 127.057531,37.487567 127.057375,37.487656 127.057225,37.487747 127.057078,37.487847 127.056939,37.487950 127.056806,37.488061 127.056678,37.488175 127.056558,37.488294 127.056444,37.488417 127.056342,37.488544 127.056242,37.488675 127.056153,37.488811 127.056072,37.488950 127.056000,37.489089 127.055933,37.489233 127.055878,37.489378 127.055606,37.490194 127.055581,37.490283 127.055464,37.490714 127.055428,37.490844 127.055400,37.490917 127.055286,37.491228 127.054744,37.492353 127.052989,37.496011 127.052564,37.496897 127.050961,37.500322 127.049808,37.502819 127.049531,37.503408 127.049161,37.504192 127.049014,37.504506 127.048381,37.505850 127.048353,37.505906 127.048322,37.505958 127.048289,37.506014 127.048256,37.506067 127.048219,37.506117 127.048183,37.506169 127.048144,37.506219 127.048103,37.506269 127.048061,37.506317 127.048017,37.506367 127.047969,37.506414 127.047922,37.506458 127.047875,37.506503 127.047825,37.506547 127.047772,37.506589 127.047719,37.506631 127.047664,37.506669 127.047608,37.506711 127.047550,37.506747 127.047492,37.506783 127.047433,37.506819 127.047419,37.506828 127.045831,37.507781 127.045183,37.508194 127.045133,37.508228 127.045086,37.508258 127.045039,37.508292 127.044992,37.508325 127.044947,37.508361 127.044903,37.508397 127.044858,37.508433 127.044817,37.508469 127.044778,37.508508 127.044736,37.508550 127.044700,37.508589 127.044661,37.508631 127.044628,37.508669 127.044592,37.508714 127.044558,37.508756 127.044528,37.508800 127.044497,37.508844 127.044469,37.508889 127.044442,37.508933 127.044417,37.508981 127.044392,37.509025 127.044086,37.509736 127.043914,37.510192 127.043883,37.510278 127.043128,37.512289 127.041472,37.516667 127.041289,37.517178 127.041203,37.517428 127.040600,37.519125 127.039192,37.522858 127.039183,37.522892 127.039175,37.522928 127.039167,37.522961 127.039161,37.522994 127.039156,37.523031 127.039153,37.523067 127.039150,37.523100 127.039147,37.523136 127.039144,37.523169 127.039144,37.523206 127.039144,37.523242 127.039147,37.523275 127.039150,37.523311 127.039153,37.523344 127.039158,37.523381 127.039164,37.523414 127.039169,37.523450 127.039178,37.523483 127.039183,37.523519 127.039194,37.523553 127.039206,37.523586 127.040639,37.527586 127.040742,37.527883 127.041628,37.530467 127.041761,37.530903 127.041811,37.531081 127.043600,37.537894 127.043631,37.538028 127.043653,37.538161 127.043853,37.540408 127.043878,37.540589 127.043914,37.540764 127.043956,37.540942 127.044583,37.543011 127.044606,37.543108 127.044861,37.544292 127.044881,37.544392 127.044894,37.544481 127.044900,37.544569 127.044900,37.544661 127.044422,37.548458 127.044381,37.548783 127.044064,37.550931 127.044061,37.550958 127.044056,37.550989 127.044047,37.551017 127.044039,37.551044 127.044031,37.551075 127.044019,37.551103 127.044006,37.551131 127.043992,37.551158 127.043978,37.551183 127.043961,37.551211 127.043942,37.551236 127.043925,37.551264 127.043903,37.551289 127.043883,37.551311 127.043861,37.551333 127.043836,37.551358 127.043811,37.551381 127.043786,37.551403 127.043758,37.551422 127.043731,37.551442 127.043703,37.551461 127.043586,37.551525 127.043492,37.551575 127.040928,37.552842 127.040906,37.552856 127.040883,37.552867 127.040864,37.552878 127.040844,37.552892 127.040825,37.552906 127.040806,37.552919 127.040786,37.552933 127.040769,37.552950 127.040753,37.552967 127.040736,37.552983 127.040719,37.553000 127.040706,37.553017 127.040692,37.553033 127.040678,37.553053 127.040667,37.553072 127.040656,37.553092 127.040644,37.553108 127.040636,37.553131 127.040628,37.553150 127.040619,37.553169 127.040614,37.553189 127.040000,37.555642 127.039139,37.559358 127.038925,37.560358 127.038750,37.561292 127.037778,37.565417 127.037761,37.565531 127.037750,37.565644 127.037744,37.565758 127.037742,37.565872 127.037742,37.565986 127.037750,37.566103 127.037761,37.566217 127.037778,37.566328 127.037797,37.566442 127.037819,37.566556 127.037850,37.566667 127.037883,37.566778 127.037919,37.566889 127.037964,37.567000 127.038008,37.567106 127.038058,37.567214 127.038114,37.567319 127.038172,37.567425 127.038236,37.567528 127.038303,37.567628 127.038375,37.567728 127.038722,37.568219 127.040781,37.570906 127.042075,37.572578 127.043939,37.574981 127.045825,37.577750 127.046642,37.578867 127.047056,37.579456 127.048000,37.580675"
              }
            },
            {
              "mode": "WALK",
              "sectionTime": 339,
              "distance": 406,
              "start": {
                "name": "청량리",
                "lon": 127.04751944444445,
                "lat": 37.58011666666667
              },
              "end": {
                "name": "도착지",
                "lon": 127.048958,
                "lat": 37.581381
              },
              "steps": [
                {
                  "streetName": "",
                  "distance": 95,
                  "description": "95m 이동",
                  "linestring": "127.04749,37.58008 127.047714,37.579975 127.04787,37.5799 127.04841,37.57964"
                },
                {
                  "streetName": "",
                  "distance": 16,
                  "description": "직진 후 16m 이동 ",
                  "linestring": "127.04841,37.57964 127.04856,37.579563"
                },
                {
                  "streetName": "",
                  "distance": 242,
                  "description": "좌회전 후 242m 이동 ",
                  "linestring": "127.04856,37.579563 127.04963,37.580936 127.04958,37.58105 127.04961,37.58109 127.04965,37.58115 127.04975,37.58128 127.04977,37.581333 127.04976,37.581387 127.04971,37.58145"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 53,
                  "description": "10시 방향 좌회전 후 보행자도로 을 따라 53m 이동 ",
                  "linestring": "127.04971,37.58145 127.04921,37.581703"
                }
              ]
            }
          ],
          "totalWalkTime": 440,
          "transferCount": 0,
          "totalDistance": 73674,
          "pathType": 1,
          "totalWalkDistance": 527
        },
        {
          "fare": {
            "regular": {
              "totalFare": 10500,
              "currency": {
                "symbol": "￦",
                "currency": "원",
                "currencyCode": "KRW"
              }
            }
          },
          "totalTime": 6226,
          "legs": [
            {
              "mode": "WALK",
              "sectionTime": 101,
              "distance": 121,
              "start": {
                "name": "출발지",
                "lon": 126.8230369,
                "lat": 37.3168267
              },
              "end": {
                "name": "고잔",
                "lon": 126.82304722222223,
                "lat": 37.316833333333335
              },
              "steps": [
                {
                  "streetName": "",
                  "distance": 27,
                  "description": "27m 이동",
                  "linestring": "126.82303,37.3168 126.82306,37.3168 126.82333,37.316788"
                },
                {
                  "streetName": "",
                  "distance": 15,
                  "description": "고잔역 에서 좌회전 후 15m 이동 ",
                  "linestring": "126.82333,37.316788 126.82334,37.31693"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 20,
                  "description": "직진 후 보행자도로 을 따라 20m 이동 ",
                  "linestring": "126.82334,37.31693 126.82336,37.317104"
                },
                {
                  "streetName": "중앙대로",
                  "distance": 39,
                  "description": "좌회전 후 중앙대로 을 따라 39m 이동 ",
                  "linestring": "126.82336,37.317104 126.822914,37.317135"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 20,
                  "description": "좌회전 후 보행자도로 을 따라 20m 이동 ",
                  "linestring": "126.822914,37.317135 126.82289,37.316963"
                }
              ]
            },
            {
              "mode": "SUBWAY",
              "routeColor": "00A5DE",
              "sectionTime": 3841,
              "route": "수도권4호선",
              "routeId": "110041005",
              "distance": 39111,
              "service": 1,
              "start": {
                "name": "고잔",
                "lon": 126.82304722222223,
                "lat": 37.316833333333335
              },
              "passStopList": {
                "stationList": [
                  {
                    "index": 0,
                    "stationName": "고잔",
                    "lon": "126.823047",
                    "lat": "37.316833",
                    "stationID": "110446"
                  },
                  {
                    "index": 1,
                    "stationName": "중앙",
                    "lon": "126.838481",
                    "lat": "37.316058",
                    "stationID": "110445"
                  },
                  {
                    "index": 2,
                    "stationName": "한대앞",
                    "lon": "126.853636",
                    "lat": "37.309722",
                    "stationID": "110444"
                  },
                  {
                    "index": 3,
                    "stationName": "상록수",
                    "lon": "126.866383",
                    "lat": "37.302869",
                    "stationID": "110443"
                  },
                  {
                    "index": 4,
                    "stationName": "반월",
                    "lon": "126.903367",
                    "lat": "37.311867",
                    "stationID": "110442"
                  },
                  {
                    "index": 5,
                    "stationName": "대야미",
                    "lon": "126.916878",
                    "lat": "37.327964",
                    "stationID": "110441"
                  },
                  {
                    "index": 6,
                    "stationName": "수리산",
                    "lon": "126.925617",
                    "lat": "37.350569",
                    "stationID": "110440"
                  },
                  {
                    "index": 7,
                    "stationName": "산본",
                    "lon": "126.933019",
                    "lat": "37.357989",
                    "stationID": "110439"
                  },
                  {
                    "index": 8,
                    "stationName": "금정",
                    "lon": "126.943608",
                    "lat": "37.372211",
                    "stationID": "110438"
                  },
                  {
                    "index": 9,
                    "stationName": "범계",
                    "lon": "126.950675",
                    "lat": "37.389764",
                    "stationID": "110437"
                  },
                  {
                    "index": 10,
                    "stationName": "평촌",
                    "lon": "126.964106",
                    "lat": "37.394431",
                    "stationID": "110436"
                  },
                  {
                    "index": 11,
                    "stationName": "인덕원",
                    "lon": "126.976800",
                    "lat": "37.401900",
                    "stationID": "110435"
                  },
                  {
                    "index": 12,
                    "stationName": "정부과천청사",
                    "lon": "126.989942",
                    "lat": "37.426792",
                    "stationID": "110434"
                  },
                  {
                    "index": 13,
                    "stationName": "과천",
                    "lon": "126.996669",
                    "lat": "37.433281",
                    "stationID": "110433"
                  },
                  {
                    "index": 14,
                    "stationName": "대공원",
                    "lon": "127.006531",
                    "lat": "37.435831",
                    "stationID": "110432"
                  },
                  {
                    "index": 15,
                    "stationName": "경마공원",
                    "lon": "127.007944",
                    "lat": "37.443806",
                    "stationID": "110431"
                  },
                  {
                    "index": 16,
                    "stationName": "선바위",
                    "lon": "127.001972",
                    "lat": "37.452111",
                    "stationID": "110430"
                  },
                  {
                    "index": 17,
                    "stationName": "남태령",
                    "lon": "126.988883",
                    "lat": "37.464647",
                    "stationID": "110429"
                  },
                  {
                    "index": 18,
                    "stationName": "사당",
                    "lon": "126.981667",
                    "lat": "37.476808",
                    "stationID": "110428"
                  },
                  {
                    "index": 19,
                    "stationName": "총신대입구",
                    "lon": "126.982264",
                    "lat": "37.487347",
                    "stationID": "110427"
                  },
                  {
                    "index": 20,
                    "stationName": "동작",
                    "lon": "126.980344",
                    "lat": "37.502897",
                    "stationID": "110426"
                  },
                  {
                    "index": 21,
                    "stationName": "이촌",
                    "lon": "126.974444",
                    "lat": "37.522350",
                    "stationID": "110425"
                  },
                  {
                    "index": 22,
                    "stationName": "신용산",
                    "lon": "126.967928",
                    "lat": "37.529222",
                    "stationID": "110424"
                  },
                  {
                    "index": 23,
                    "stationName": "삼각지",
                    "lon": "126.972633",
                    "lat": "37.534133",
                    "stationID": "110423"
                  },
                  {
                    "index": 24,
                    "stationName": "숙대입구",
                    "lon": "126.972111",
                    "lat": "37.544600",
                    "stationID": "110422"
                  },
                  {
                    "index": 25,
                    "stationName": "서울역",
                    "lon": "126.972797",
                    "lat": "37.553497",
                    "stationID": "110421"
                  }
                ]
              },
              "end": {
                "name": "서울역",
                "lon": 126.97279722222223,
                "lat": 37.55349722222222
              },
              "type": 4,
              "passShape": {
                "linestring": "126.823047,37.316833 126.824414,37.316767 126.826906,37.316628 126.830992,37.316458 126.832469,37.316378 126.833967,37.316292 126.835467,37.316217 126.836961,37.316139 126.838481,37.316061 126.839886,37.315986 126.842467,37.315819 126.844714,37.315669 126.845500,37.315581 126.846008,37.315494 126.846500,37.315406 126.846953,37.315281 126.847328,37.315158 126.847739,37.314989 126.848297,37.314700 126.848778,37.314403 126.849225,37.314086 126.849656,37.313689 126.850683,37.312733 126.851578,37.311858 126.852081,37.311344 126.852964,37.310425 126.853639,37.309722 126.853753,37.309606 126.854828,37.308542 126.855744,37.307639 126.856328,37.307072 126.857392,37.306008 126.857856,37.305514 126.858386,37.304958 126.858814,37.304581 126.859214,37.304283 126.859542,37.304086 126.859942,37.303889 126.860364,37.303700 126.860919,37.303478 126.861364,37.303333 126.861831,37.303228 126.862433,37.303131 126.863000,37.303081 126.863614,37.303056 126.864522,37.302989 126.864847,37.302964 126.866383,37.302867 126.867319,37.302806 126.871492,37.302481 126.872903,37.302453 126.873700,37.302464 126.874558,37.302494 126.875353,37.302522 126.878306,37.302639 126.879389,37.302683 126.879444,37.302686 126.880114,37.302728 126.881167,37.302778 126.881628,37.302781 126.882600,37.302847 126.883436,37.303019 126.884569,37.303317 126.885189,37.303486 126.885669,37.303642 126.886589,37.303892 126.888092,37.304297 126.888661,37.304450 126.890783,37.305022 126.892403,37.305458 126.892564,37.305503 126.892981,37.305614 126.894681,37.306069 126.895633,37.306331 126.896247,37.306492 126.896492,37.306558 126.896719,37.306622 126.896942,37.306689 126.897150,37.306756 126.897361,37.306831 126.897625,37.306928 126.897911,37.307047 126.898361,37.307239 126.898828,37.307475 126.899233,37.307703 126.899661,37.307931 126.899892,37.308075 126.900097,37.308219 126.900678,37.308725 126.900789,37.308831 126.901239,37.309314 126.901683,37.309800 126.902156,37.310414 126.902169,37.310428 126.903367,37.311867 126.904406,37.313108 126.905708,37.314681 126.907406,37.316717 126.908850,37.318425 126.910258,37.320100 126.911522,37.321614 126.912292,37.322533 126.912761,37.323092 126.913981,37.324544 126.914781,37.325519 126.915297,37.326094 126.915756,37.326639 126.916767,37.327828 126.916878,37.327964 126.917661,37.328922 126.917814,37.329117 126.918161,37.329506 126.918344,37.329744 126.918844,37.330286 126.919256,37.330789 126.919794,37.331383 126.920208,37.331872 126.920333,37.332044 126.920756,37.332558 126.921186,37.333181 126.921350,37.333408 126.921494,37.333658 126.921644,37.333936 126.921775,37.334258 126.922056,37.334978 126.922269,37.335875 126.922333,37.336169 126.922636,37.337656 126.923267,37.340964 126.923528,37.342833 126.923919,37.344722 126.924153,37.345831 126.925533,37.350419 126.925583,37.350578 126.925914,37.351614 126.926225,37.352608 126.926347,37.352931 126.926475,37.353253 126.926664,37.353675 126.926964,37.354200 126.927178,37.354453 126.927636,37.354939 126.928156,37.355353 126.930842,37.356931 126.932967,37.358056 126.935383,37.359333 126.936328,37.359833 126.939789,37.361636 126.942525,37.363175 126.944708,37.364322 126.944894,37.364464 126.945069,37.364628 126.945267,37.364825 126.945411,37.365006 126.945514,37.365156 126.945586,37.365269 126.945653,37.365397 126.945731,37.365625 126.945792,37.365825 126.945825,37.366022 126.945839,37.366267 126.945839,37.366431 126.945808,37.366608 126.945775,37.366781 126.945708,37.366978 126.945628,37.367172 126.945536,37.367378 126.945350,37.367761 126.945236,37.367994 126.945072,37.368339 126.945006,37.368506 126.943858,37.371331 126.943644,37.371906 126.943539,37.372194 126.943325,37.372775 126.942281,37.374958 126.941906,37.375700 126.941767,37.375947 126.941228,37.376814 126.940958,37.377247 126.940797,37.377472 126.940664,37.377628 126.940542,37.377758 126.940036,37.378289 126.939917,37.378406 126.938700,37.379531 126.938669,37.379575 126.938653,37.379619 126.938642,37.379664 126.938633,37.379706 126.938636,37.379767 126.938644,37.379828 126.938872,37.380197 126.939181,37.380786 126.939919,37.382556 126.940189,37.383083 126.940436,37.383478 126.940786,37.383844 126.941064,37.384042 126.941464,37.384264 126.941989,37.384469 126.942383,37.384636 126.942733,37.384825 126.942994,37.384981 126.943428,37.385258 126.943864,37.385597 126.945186,37.386678 126.947628,37.388694 126.947642,37.388703 126.947653,37.388714 126.947672,37.388722 126.947686,37.388728 126.947700,37.388733 126.950675,37.389767 126.962272,37.393797 126.964086,37.394467 126.964389,37.394578 126.968514,37.395942 126.971767,37.397033 126.973083,37.397500 126.974419,37.397997 126.975417,37.398531 126.975703,37.398761 126.975883,37.398925 126.976144,37.399208 126.976603,37.399822 126.976797,37.400136 126.976881,37.400292 126.976914,37.400467 126.976833,37.401311 126.976833,37.401350 126.976783,37.402194 126.976803,37.402544 126.976883,37.402842 126.977764,37.404747 126.977808,37.404872 126.978228,37.406642 126.978494,37.407764 126.979033,37.410022 126.979122,37.410444 126.979419,37.411742 126.979725,37.412986 126.979781,37.413208 126.979867,37.413675 126.979908,37.413847 126.980011,37.414200 126.980928,37.416297 126.981547,37.417675 126.981750,37.418144 126.982047,37.418675 126.982469,37.419406 126.982989,37.420228 126.983950,37.421275 126.984853,37.422069 126.985869,37.422925 126.987003,37.423869 126.987781,37.424625 126.988981,37.425722 126.989956,37.426783 126.990411,37.427278 126.991381,37.428233 126.992906,37.429739 126.995314,37.432433 126.995839,37.432964 126.996069,37.433125 126.996378,37.433244 126.996656,37.433317 126.996756,37.433342 126.997372,37.433425 126.997742,37.433425 126.998319,37.433353 126.999225,37.433258 126.999853,37.433225 127.000758,37.433239 127.004331,37.433214 127.004481,37.433256 127.005314,37.433958 127.005433,37.434131 127.006244,37.435392 127.006450,37.435711 127.008228,37.438322 127.008297,37.438447 127.008344,37.438575 127.008364,37.438711 127.008375,37.438969 127.008203,37.440986 127.008019,37.442944 127.007969,37.443506 127.007944,37.443831 127.007819,37.444217 127.007717,37.444408 127.007436,37.444817 127.006928,37.445475 127.004367,37.448900 127.002544,37.451400 127.001972,37.452111 127.001836,37.452281 127.001633,37.452472 127.001442,37.452575 127.000942,37.452764 126.999947,37.453253 126.998028,37.454378 126.997922,37.454442 126.996819,37.455239 126.995569,37.456381 126.994317,37.457522 126.993775,37.458058 126.993728,37.458117 126.993250,37.458667 126.991156,37.461425 126.989422,37.463706 126.989111,37.464247 126.988883,37.464647 126.987964,37.466256 126.987881,37.466394 126.987797,37.466528 126.987708,37.466664 126.987617,37.466794 126.987525,37.466928 126.987428,37.467058 126.987328,37.467186 126.987225,37.467314 126.987122,37.467442 126.987014,37.467567 126.986903,37.467689 126.986792,37.467811 126.986675,37.467931 126.986558,37.468050 126.986439,37.468167 126.986314,37.468283 126.986189,37.468394 126.986061,37.468508 126.985931,37.468619 126.985800,37.468728 126.985664,37.468833 126.985211,37.469178 126.984864,37.469464 126.984636,37.469658 126.984472,37.469808 126.984356,37.469928 126.983858,37.470544 126.983769,37.470672 126.983256,37.471453 126.982953,37.472042 126.982397,37.473183 126.982053,37.474025 126.981878,37.474547 126.981769,37.474950 126.981675,37.475608 126.981669,37.475658 126.981633,37.476083 126.981656,37.476564 126.981658,37.476653 126.982111,37.485253 126.982117,37.485383 126.982122,37.485475 126.982158,37.486308 126.982228,37.487350 126.982233,37.487400 126.982378,37.488686 126.982531,37.489653 126.982592,37.490369 126.982647,37.494967 126.982628,37.496269 126.982639,37.496469 126.982664,37.496669 126.982692,37.496817 126.982744,37.497011 126.982794,37.497156 126.982983,37.497625 126.983100,37.497950 126.983239,37.498383 126.983267,37.498531 126.983264,37.498681 126.983233,37.498825 126.983178,37.498967 126.983067,37.499131 126.981803,37.500881 126.981647,37.501097 126.981553,37.501222 126.981425,37.501369 126.980922,37.501886 126.980856,37.501964 126.980794,37.502033 126.980644,37.502233 126.980547,37.502394 126.980464,37.502575 126.980381,37.502808 126.980294,37.503031 126.980269,37.503125 126.980183,37.503464 126.980156,37.503878 126.980156,37.504058 126.980164,37.504244 126.980186,37.504425 126.980258,37.504800 126.980719,37.506508 126.983114,37.515581 126.983161,37.515808 126.983203,37.516089 126.983231,37.516306 126.983269,37.517844 126.983278,37.518414 126.983261,37.518581 126.983225,37.518736 126.983183,37.518883 126.983128,37.519019 126.983053,37.519164 126.982961,37.519292 126.982853,37.519408 126.982725,37.519531 126.982633,37.519608 126.982606,37.519633 126.982456,37.519739 126.982292,37.519836 126.982108,37.519936 126.981919,37.520033 126.981719,37.520119 126.981503,37.520206 126.976931,37.521594 126.975139,37.522128 126.974444,37.522344 126.972900,37.522833 126.972414,37.522969 126.972056,37.523053 126.971644,37.523133 126.971183,37.523214 126.970789,37.523292 126.970444,37.523367 126.970233,37.523428 126.970031,37.523500 126.969831,37.523583 126.969533,37.523728 126.966744,37.525375 126.966603,37.525464 126.966583,37.525478 126.966564,37.525489 126.966547,37.525503 126.966531,37.525517 126.966514,37.525531 126.966500,37.525547 126.966486,37.525564 126.966475,37.525581 126.966461,37.525594 126.966450,37.525614 126.966442,37.525631 126.966431,37.525650 126.966425,37.525667 126.966417,37.525686 126.966411,37.525703 126.966406,37.525722 126.966403,37.525742 126.966400,37.525761 126.966400,37.525781 126.966400,37.525800 126.966400,37.525819 126.966425,37.527406 126.966428,37.527431 126.966431,37.527456 126.966433,37.527481 126.966439,37.527506 126.966444,37.527528 126.966453,37.527553 126.966458,37.527578 126.966467,37.527600 126.966478,37.527625 126.966486,37.527647 126.966497,37.527669 126.966508,37.527692 126.966522,37.527714 126.966533,37.527739 126.966550,37.527758 126.966564,37.527781 126.966581,37.527803 126.966594,37.527822 126.966614,37.527844 126.966631,37.527864 126.966650,37.527883 126.967931,37.529222 126.968436,37.529753 126.971272,37.532750 126.972633,37.534133 126.973353,37.534864 126.973986,37.535603 126.973989,37.535606 126.974167,37.535861 126.974181,37.535883 126.974194,37.535906 126.974206,37.535928 126.974217,37.535950 126.974228,37.535975 126.974239,37.535997 126.974247,37.536019 126.974256,37.536044 126.974261,37.536069 126.974267,37.536092 126.974272,37.536117 126.974278,37.536142 126.974281,37.536167 126.974281,37.536192 126.974283,37.536217 126.974281,37.536239 126.974281,37.536264 126.974278,37.536289 126.974275,37.536314 126.974269,37.536339 126.974267,37.536364 126.972572,37.542839 126.972311,37.543833 126.972008,37.544989 126.971953,37.545361 126.971900,37.546225 126.971819,37.548156 126.971794,37.548819 126.971797,37.549114 126.971819,37.549381 126.971856,37.549678 126.971928,37.550069 126.972000,37.550425 126.972294,37.551683 126.972739,37.553319 126.972747,37.553344"
              }
            },
            {
              "mode": "WALK",
              "sectionTime": 601,
              "distance": 762,
              "start": {
                "name": "서울역",
                "lon": 126.97279722222223,
                "lat": 37.55349722222222
              },
              "end": {
                "name": "서울",
                "lon": 126.97060833333333,
                "lat": 37.55463888888889
              },
              "passShape": {
                "linestring": "126.972797,37.553497 126.972717,37.553425 126.972650,37.553175 126.972453,37.553203 126.972478,37.553297 126.972439,37.553319 126.972450,37.553386 126.972417,37.553869 126.972378,37.554042 126.972275,37.554300 126.972208,37.554414 126.972150,37.554436 126.972089,37.554461 126.971981,37.554511 126.971869,37.554647 126.971869,37.554781 126.971867,37.554969 126.971869,37.555400 126.971906,37.555539 126.971983,37.555733 126.972078,37.555856 126.972164,37.556042 126.972192,37.556158 126.972206,37.556275 126.972181,37.556364 126.972142,37.556494 126.972017,37.556383 126.972025,37.556344 126.971706,37.556342 126.971703,37.555542 126.971700,37.555172 126.970778,37.555194 126.970792,37.554847 126.970608,37.554639"
              }
            },
            {
              "routeColor": "F75C4C",
              "distance": 7466,
              "start": {
                "name": "서울",
                "lon": 126.97060833333333,
                "lat": 37.55463888888889
              },
              "type": 3,
              "mode": "TRAIN",
              "sectionTime": 1268,
              "route": "KTX이음:서울-청량리",
              "routeId": "500001-500010-KTXEUM001",
              "service": 1,
              "passStopList": {
                "stationList": [
                  {
                    "index": 0,
                    "stationName": "서울",
                    "lon": "126.970608",
                    "lat": "37.554639",
                    "stationID": "500001"
                  },
                  {
                    "index": 1,
                    "stationName": "청량리",
                    "lon": "127.048500",
                    "lat": "37.580742",
                    "stationID": "500010"
                  }
                ]
              },
              "end": {
                "name": "청량리",
                "lon": 127.0485,
                "lat": 37.58074166666667
              },
              "passShape": {
                "linestring": "126.970608,37.554639 127.048500,37.580742"
              },
              "routePayment": 8400
            },
            {
              "mode": "WALK",
              "sectionTime": 415,
              "distance": 492,
              "start": {
                "name": "청량리",
                "lon": 127.0485,
                "lat": 37.58074166666667
              },
              "end": {
                "name": "도착지",
                "lon": 127.048958,
                "lat": 37.581381
              },
              "steps": [
                {
                  "streetName": "",
                  "distance": 109,
                  "description": "109m 이동",
                  "linestring": "127.04837,37.580803 127.04826,37.58067 127.04796,37.580284 127.047714,37.579975"
                },
                {
                  "streetName": "",
                  "distance": 72,
                  "description": "던킨도너츠 청량리역사점 에서 좌회전 후 72m 이동 ",
                  "linestring": "127.047714,37.579975 127.04787,37.5799 127.04841,37.57964"
                },
                {
                  "streetName": "",
                  "distance": 16,
                  "description": "직진 후 16m 이동 ",
                  "linestring": "127.04841,37.57964 127.04856,37.579563"
                },
                {
                  "streetName": "",
                  "distance": 242,
                  "description": "좌회전 후 242m 이동 ",
                  "linestring": "127.04856,37.579563 127.04963,37.580936 127.04958,37.58105 127.04961,37.58109 127.04965,37.58115 127.04975,37.58128 127.04977,37.581333 127.04976,37.581387 127.04971,37.58145"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 53,
                  "description": "10시 방향 좌회전 후 보행자도로 을 따라 53m 이동 ",
                  "linestring": "127.04971,37.58145 127.04921,37.581703"
                }
              ]
            }
          ],
          "totalWalkTime": 1117,
          "transferCount": 1,
          "totalDistance": 47423,
          "pathType": 5,
          "totalWalkDistance": 1375
        },
        {
          "fare": {
            "regular": {
              "totalFare": 2300,
              "currency": {
                "symbol": "￦",
                "currency": "원",
                "currencyCode": "KRW"
              }
            }
          },
          "totalTime": 6313,
          "legs": [
            {
              "mode": "WALK",
              "sectionTime": 101,
              "distance": 121,
              "start": {
                "name": "출발지",
                "lon": 126.8230369,
                "lat": 37.3168267
              },
              "end": {
                "name": "고잔",
                "lon": 126.82304722222223,
                "lat": 37.316833333333335
              },
              "steps": [
                {
                  "streetName": "",
                  "distance": 27,
                  "description": "27m 이동",
                  "linestring": "126.82303,37.3168 126.82306,37.3168 126.82333,37.316788"
                },
                {
                  "streetName": "",
                  "distance": 15,
                  "description": "고잔역 에서 좌회전 후 15m 이동 ",
                  "linestring": "126.82333,37.316788 126.82334,37.31693"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 20,
                  "description": "직진 후 보행자도로 을 따라 20m 이동 ",
                  "linestring": "126.82334,37.31693 126.82336,37.317104"
                },
                {
                  "streetName": "중앙대로",
                  "distance": 39,
                  "description": "좌회전 후 중앙대로 을 따라 39m 이동 ",
                  "linestring": "126.82336,37.317104 126.822914,37.317135"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 20,
                  "description": "좌회전 후 보행자도로 을 따라 20m 이동 ",
                  "linestring": "126.822914,37.317135 126.82289,37.316963"
                }
              ]
            },
            {
              "mode": "SUBWAY",
              "routeColor": "00A5DE",
              "sectionTime": 4319,
              "route": "수도권4호선",
              "routeId": "110041005",
              "distance": 42538,
              "service": 1,
              "start": {
                "name": "고잔",
                "lon": 126.82304722222223,
                "lat": 37.316833333333335
              },
              "passStopList": {
                "stationList": [
                  {
                    "index": 0,
                    "stationName": "고잔",
                    "lon": "126.823047",
                    "lat": "37.316833",
                    "stationID": "110446"
                  },
                  {
                    "index": 1,
                    "stationName": "중앙",
                    "lon": "126.838481",
                    "lat": "37.316058",
                    "stationID": "110445"
                  },
                  {
                    "index": 2,
                    "stationName": "한대앞",
                    "lon": "126.853636",
                    "lat": "37.309722",
                    "stationID": "110444"
                  },
                  {
                    "index": 3,
                    "stationName": "상록수",
                    "lon": "126.866383",
                    "lat": "37.302869",
                    "stationID": "110443"
                  },
                  {
                    "index": 4,
                    "stationName": "반월",
                    "lon": "126.903367",
                    "lat": "37.311867",
                    "stationID": "110442"
                  },
                  {
                    "index": 5,
                    "stationName": "대야미",
                    "lon": "126.916878",
                    "lat": "37.327964",
                    "stationID": "110441"
                  },
                  {
                    "index": 6,
                    "stationName": "수리산",
                    "lon": "126.925617",
                    "lat": "37.350569",
                    "stationID": "110440"
                  },
                  {
                    "index": 7,
                    "stationName": "산본",
                    "lon": "126.933019",
                    "lat": "37.357989",
                    "stationID": "110439"
                  },
                  {
                    "index": 8,
                    "stationName": "금정",
                    "lon": "126.943608",
                    "lat": "37.372211",
                    "stationID": "110438"
                  },
                  {
                    "index": 9,
                    "stationName": "범계",
                    "lon": "126.950675",
                    "lat": "37.389764",
                    "stationID": "110437"
                  },
                  {
                    "index": 10,
                    "stationName": "평촌",
                    "lon": "126.964106",
                    "lat": "37.394431",
                    "stationID": "110436"
                  },
                  {
                    "index": 11,
                    "stationName": "인덕원",
                    "lon": "126.976800",
                    "lat": "37.401900",
                    "stationID": "110435"
                  },
                  {
                    "index": 12,
                    "stationName": "정부과천청사",
                    "lon": "126.989942",
                    "lat": "37.426792",
                    "stationID": "110434"
                  },
                  {
                    "index": 13,
                    "stationName": "과천",
                    "lon": "126.996669",
                    "lat": "37.433281",
                    "stationID": "110433"
                  },
                  {
                    "index": 14,
                    "stationName": "대공원",
                    "lon": "127.006531",
                    "lat": "37.435831",
                    "stationID": "110432"
                  },
                  {
                    "index": 15,
                    "stationName": "경마공원",
                    "lon": "127.007944",
                    "lat": "37.443806",
                    "stationID": "110431"
                  },
                  {
                    "index": 16,
                    "stationName": "선바위",
                    "lon": "127.001972",
                    "lat": "37.452111",
                    "stationID": "110430"
                  },
                  {
                    "index": 17,
                    "stationName": "남태령",
                    "lon": "126.988883",
                    "lat": "37.464647",
                    "stationID": "110429"
                  },
                  {
                    "index": 18,
                    "stationName": "사당",
                    "lon": "126.981667",
                    "lat": "37.476808",
                    "stationID": "110428"
                  },
                  {
                    "index": 19,
                    "stationName": "총신대입구",
                    "lon": "126.982264",
                    "lat": "37.487347",
                    "stationID": "110427"
                  },
                  {
                    "index": 20,
                    "stationName": "동작",
                    "lon": "126.980344",
                    "lat": "37.502897",
                    "stationID": "110426"
                  },
                  {
                    "index": 21,
                    "stationName": "이촌",
                    "lon": "126.974444",
                    "lat": "37.522350",
                    "stationID": "110425"
                  },
                  {
                    "index": 22,
                    "stationName": "신용산",
                    "lon": "126.967928",
                    "lat": "37.529222",
                    "stationID": "110424"
                  },
                  {
                    "index": 23,
                    "stationName": "삼각지",
                    "lon": "126.972633",
                    "lat": "37.534133",
                    "stationID": "110423"
                  },
                  {
                    "index": 24,
                    "stationName": "숙대입구",
                    "lon": "126.972111",
                    "lat": "37.544600",
                    "stationID": "110422"
                  },
                  {
                    "index": 25,
                    "stationName": "서울역",
                    "lon": "126.972797",
                    "lat": "37.553497",
                    "stationID": "110421"
                  },
                  {
                    "index": 26,
                    "stationName": "회현",
                    "lon": "126.978564",
                    "lat": "37.558886",
                    "stationID": "110420"
                  },
                  {
                    "index": 27,
                    "stationName": "명동",
                    "lon": "126.986489",
                    "lat": "37.560981",
                    "stationID": "110419"
                  },
                  {
                    "index": 28,
                    "stationName": "충무로",
                    "lon": "126.994172",
                    "lat": "37.561256",
                    "stationID": "110418"
                  },
                  {
                    "index": 29,
                    "stationName": "동대문역사문화공원",
                    "lon": "127.007356",
                    "lat": "37.564644",
                    "stationID": "110417"
                  }
                ]
              },
              "end": {
                "name": "동대문역사문화공원",
                "lon": 127.00735555555555,
                "lat": 37.56464444444445
              },
              "type": 4,
              "passShape": {
                "linestring": "126.823047,37.316833 126.824414,37.316767 126.826906,37.316628 126.830992,37.316458 126.832469,37.316378 126.833967,37.316292 126.835467,37.316217 126.836961,37.316139 126.838481,37.316061 126.839886,37.315986 126.842467,37.315819 126.844714,37.315669 126.845500,37.315581 126.846008,37.315494 126.846500,37.315406 126.846953,37.315281 126.847328,37.315158 126.847739,37.314989 126.848297,37.314700 126.848778,37.314403 126.849225,37.314086 126.849656,37.313689 126.850683,37.312733 126.851578,37.311858 126.852081,37.311344 126.852964,37.310425 126.853639,37.309722 126.853753,37.309606 126.854828,37.308542 126.855744,37.307639 126.856328,37.307072 126.857392,37.306008 126.857856,37.305514 126.858386,37.304958 126.858814,37.304581 126.859214,37.304283 126.859542,37.304086 126.859942,37.303889 126.860364,37.303700 126.860919,37.303478 126.861364,37.303333 126.861831,37.303228 126.862433,37.303131 126.863000,37.303081 126.863614,37.303056 126.864522,37.302989 126.864847,37.302964 126.866383,37.302867 126.867319,37.302806 126.871492,37.302481 126.872903,37.302453 126.873700,37.302464 126.874558,37.302494 126.875353,37.302522 126.878306,37.302639 126.879389,37.302683 126.879444,37.302686 126.880114,37.302728 126.881167,37.302778 126.881628,37.302781 126.882600,37.302847 126.883436,37.303019 126.884569,37.303317 126.885189,37.303486 126.885669,37.303642 126.886589,37.303892 126.888092,37.304297 126.888661,37.304450 126.890783,37.305022 126.892403,37.305458 126.892564,37.305503 126.892981,37.305614 126.894681,37.306069 126.895633,37.306331 126.896247,37.306492 126.896492,37.306558 126.896719,37.306622 126.896942,37.306689 126.897150,37.306756 126.897361,37.306831 126.897625,37.306928 126.897911,37.307047 126.898361,37.307239 126.898828,37.307475 126.899233,37.307703 126.899661,37.307931 126.899892,37.308075 126.900097,37.308219 126.900678,37.308725 126.900789,37.308831 126.901239,37.309314 126.901683,37.309800 126.902156,37.310414 126.902169,37.310428 126.903367,37.311867 126.904406,37.313108 126.905708,37.314681 126.907406,37.316717 126.908850,37.318425 126.910258,37.320100 126.911522,37.321614 126.912292,37.322533 126.912761,37.323092 126.913981,37.324544 126.914781,37.325519 126.915297,37.326094 126.915756,37.326639 126.916767,37.327828 126.916878,37.327964 126.917661,37.328922 126.917814,37.329117 126.918161,37.329506 126.918344,37.329744 126.918844,37.330286 126.919256,37.330789 126.919794,37.331383 126.920208,37.331872 126.920333,37.332044 126.920756,37.332558 126.921186,37.333181 126.921350,37.333408 126.921494,37.333658 126.921644,37.333936 126.921775,37.334258 126.922056,37.334978 126.922269,37.335875 126.922333,37.336169 126.922636,37.337656 126.923267,37.340964 126.923528,37.342833 126.923919,37.344722 126.924153,37.345831 126.925533,37.350419 126.925583,37.350578 126.925914,37.351614 126.926225,37.352608 126.926347,37.352931 126.926475,37.353253 126.926664,37.353675 126.926964,37.354200 126.927178,37.354453 126.927636,37.354939 126.928156,37.355353 126.930842,37.356931 126.932967,37.358056 126.935383,37.359333 126.936328,37.359833 126.939789,37.361636 126.942525,37.363175 126.944708,37.364322 126.944894,37.364464 126.945069,37.364628 126.945267,37.364825 126.945411,37.365006 126.945514,37.365156 126.945586,37.365269 126.945653,37.365397 126.945731,37.365625 126.945792,37.365825 126.945825,37.366022 126.945839,37.366267 126.945839,37.366431 126.945808,37.366608 126.945775,37.366781 126.945708,37.366978 126.945628,37.367172 126.945536,37.367378 126.945350,37.367761 126.945236,37.367994 126.945072,37.368339 126.945006,37.368506 126.943858,37.371331 126.943644,37.371906 126.943539,37.372194 126.943325,37.372775 126.942281,37.374958 126.941906,37.375700 126.941767,37.375947 126.941228,37.376814 126.940958,37.377247 126.940797,37.377472 126.940664,37.377628 126.940542,37.377758 126.940036,37.378289 126.939917,37.378406 126.938700,37.379531 126.938669,37.379575 126.938653,37.379619 126.938642,37.379664 126.938633,37.379706 126.938636,37.379767 126.938644,37.379828 126.938872,37.380197 126.939181,37.380786 126.939919,37.382556 126.940189,37.383083 126.940436,37.383478 126.940786,37.383844 126.941064,37.384042 126.941464,37.384264 126.941989,37.384469 126.942383,37.384636 126.942733,37.384825 126.942994,37.384981 126.943428,37.385258 126.943864,37.385597 126.945186,37.386678 126.947628,37.388694 126.947642,37.388703 126.947653,37.388714 126.947672,37.388722 126.947686,37.388728 126.947700,37.388733 126.950675,37.389767 126.962272,37.393797 126.964086,37.394467 126.964389,37.394578 126.968514,37.395942 126.971767,37.397033 126.973083,37.397500 126.974419,37.397997 126.975417,37.398531 126.975703,37.398761 126.975883,37.398925 126.976144,37.399208 126.976603,37.399822 126.976797,37.400136 126.976881,37.400292 126.976914,37.400467 126.976833,37.401311 126.976833,37.401350 126.976783,37.402194 126.976803,37.402544 126.976883,37.402842 126.977764,37.404747 126.977808,37.404872 126.978228,37.406642 126.978494,37.407764 126.979033,37.410022 126.979122,37.410444 126.979419,37.411742 126.979725,37.412986 126.979781,37.413208 126.979867,37.413675 126.979908,37.413847 126.980011,37.414200 126.980928,37.416297 126.981547,37.417675 126.981750,37.418144 126.982047,37.418675 126.982469,37.419406 126.982989,37.420228 126.983950,37.421275 126.984853,37.422069 126.985869,37.422925 126.987003,37.423869 126.987781,37.424625 126.988981,37.425722 126.989956,37.426783 126.990411,37.427278 126.991381,37.428233 126.992906,37.429739 126.995314,37.432433 126.995839,37.432964 126.996069,37.433125 126.996378,37.433244 126.996656,37.433317 126.996756,37.433342 126.997372,37.433425 126.997742,37.433425 126.998319,37.433353 126.999225,37.433258 126.999853,37.433225 127.000758,37.433239 127.004331,37.433214 127.004481,37.433256 127.005314,37.433958 127.005433,37.434131 127.006244,37.435392 127.006450,37.435711 127.008228,37.438322 127.008297,37.438447 127.008344,37.438575 127.008364,37.438711 127.008375,37.438969 127.008203,37.440986 127.008019,37.442944 127.007969,37.443506 127.007944,37.443831 127.007819,37.444217 127.007717,37.444408 127.007436,37.444817 127.006928,37.445475 127.004367,37.448900 127.002544,37.451400 127.001972,37.452111 127.001836,37.452281 127.001633,37.452472 127.001442,37.452575 127.000942,37.452764 126.999947,37.453253 126.998028,37.454378 126.997922,37.454442 126.996819,37.455239 126.995569,37.456381 126.994317,37.457522 126.993775,37.458058 126.993728,37.458117 126.993250,37.458667 126.991156,37.461425 126.989422,37.463706 126.989111,37.464247 126.988883,37.464647 126.987964,37.466256 126.987881,37.466394 126.987797,37.466528 126.987708,37.466664 126.987617,37.466794 126.987525,37.466928 126.987428,37.467058 126.987328,37.467186 126.987225,37.467314 126.987122,37.467442 126.987014,37.467567 126.986903,37.467689 126.986792,37.467811 126.986675,37.467931 126.986558,37.468050 126.986439,37.468167 126.986314,37.468283 126.986189,37.468394 126.986061,37.468508 126.985931,37.468619 126.985800,37.468728 126.985664,37.468833 126.985211,37.469178 126.984864,37.469464 126.984636,37.469658 126.984472,37.469808 126.984356,37.469928 126.983858,37.470544 126.983769,37.470672 126.983256,37.471453 126.982953,37.472042 126.982397,37.473183 126.982053,37.474025 126.981878,37.474547 126.981769,37.474950 126.981675,37.475608 126.981669,37.475658 126.981633,37.476083 126.981656,37.476564 126.981658,37.476653 126.982111,37.485253 126.982117,37.485383 126.982122,37.485475 126.982158,37.486308 126.982228,37.487350 126.982233,37.487400 126.982378,37.488686 126.982531,37.489653 126.982592,37.490369 126.982647,37.494967 126.982628,37.496269 126.982639,37.496469 126.982664,37.496669 126.982692,37.496817 126.982744,37.497011 126.982794,37.497156 126.982983,37.497625 126.983100,37.497950 126.983239,37.498383 126.983267,37.498531 126.983264,37.498681 126.983233,37.498825 126.983178,37.498967 126.983067,37.499131 126.981803,37.500881 126.981647,37.501097 126.981553,37.501222 126.981425,37.501369 126.980922,37.501886 126.980856,37.501964 126.980794,37.502033 126.980644,37.502233 126.980547,37.502394 126.980464,37.502575 126.980381,37.502808 126.980294,37.503031 126.980269,37.503125 126.980183,37.503464 126.980156,37.503878 126.980156,37.504058 126.980164,37.504244 126.980186,37.504425 126.980258,37.504800 126.980719,37.506508 126.983114,37.515581 126.983161,37.515808 126.983203,37.516089 126.983231,37.516306 126.983269,37.517844 126.983278,37.518414 126.983261,37.518581 126.983225,37.518736 126.983183,37.518883 126.983128,37.519019 126.983053,37.519164 126.982961,37.519292 126.982853,37.519408 126.982725,37.519531 126.982633,37.519608 126.982606,37.519633 126.982456,37.519739 126.982292,37.519836 126.982108,37.519936 126.981919,37.520033 126.981719,37.520119 126.981503,37.520206 126.976931,37.521594 126.975139,37.522128 126.974444,37.522344 126.972900,37.522833 126.972414,37.522969 126.972056,37.523053 126.971644,37.523133 126.971183,37.523214 126.970789,37.523292 126.970444,37.523367 126.970233,37.523428 126.970031,37.523500 126.969831,37.523583 126.969533,37.523728 126.966744,37.525375 126.966603,37.525464 126.966583,37.525478 126.966564,37.525489 126.966547,37.525503 126.966531,37.525517 126.966514,37.525531 126.966500,37.525547 126.966486,37.525564 126.966475,37.525581 126.966461,37.525594 126.966450,37.525614 126.966442,37.525631 126.966431,37.525650 126.966425,37.525667 126.966417,37.525686 126.966411,37.525703 126.966406,37.525722 126.966403,37.525742 126.966400,37.525761 126.966400,37.525781 126.966400,37.525800 126.966400,37.525819 126.966425,37.527406 126.966428,37.527431 126.966431,37.527456 126.966433,37.527481 126.966439,37.527506 126.966444,37.527528 126.966453,37.527553 126.966458,37.527578 126.966467,37.527600 126.966478,37.527625 126.966486,37.527647 126.966497,37.527669 126.966508,37.527692 126.966522,37.527714 126.966533,37.527739 126.966550,37.527758 126.966564,37.527781 126.966581,37.527803 126.966594,37.527822 126.966614,37.527844 126.966631,37.527864 126.966650,37.527883 126.967931,37.529222 126.968436,37.529753 126.971272,37.532750 126.972633,37.534133 126.973353,37.534864 126.973986,37.535603 126.973989,37.535606 126.974167,37.535861 126.974181,37.535883 126.974194,37.535906 126.974206,37.535928 126.974217,37.535950 126.974228,37.535975 126.974239,37.535997 126.974247,37.536019 126.974256,37.536044 126.974261,37.536069 126.974267,37.536092 126.974272,37.536117 126.974278,37.536142 126.974281,37.536167 126.974281,37.536192 126.974283,37.536217 126.974281,37.536239 126.974281,37.536264 126.974278,37.536289 126.974275,37.536314 126.974269,37.536339 126.974267,37.536364 126.972572,37.542839 126.972311,37.543833 126.972008,37.544989 126.971953,37.545361 126.971900,37.546225 126.971819,37.548156 126.971794,37.548819 126.971797,37.549114 126.971819,37.549381 126.971856,37.549678 126.971928,37.550069 126.972000,37.550425 126.972294,37.551683 126.972739,37.553319 126.972747,37.553344 126.972939,37.553928 126.973025,37.554339 126.973044,37.554517 126.973072,37.555861 126.973075,37.555892 126.973081,37.555922 126.973089,37.555953 126.973097,37.555981 126.973108,37.556008 126.973122,37.556039 126.973136,37.556067 126.973153,37.556094 126.973172,37.556119 126.973192,37.556144 126.973214,37.556169 126.973236,37.556194 126.973261,37.556219 126.973289,37.556242 126.973314,37.556261 126.973344,37.556281 126.973375,37.556300 126.973406,37.556317 126.973436,37.556333 126.973469,37.556350 126.973506,37.556361 126.973986,37.556536 126.977178,37.557736 126.977364,37.557817 126.977617,37.557967 126.977814,37.558117 126.978078,37.558394 126.978444,37.558789 126.978564,37.558883 126.978656,37.558956 126.978692,37.558981 126.978731,37.559006 126.978769,37.559028 126.978806,37.559050 126.978844,37.559075 126.978886,37.559094 126.978925,37.559117 126.978967,37.559139 126.979006,37.559158 126.979047,37.559178 126.979089,37.559197 126.979131,37.559217 126.979172,37.559233 126.979217,37.559253 126.979258,37.559269 126.979303,37.559286 126.979344,37.559303 126.979389,37.559319 126.979433,37.559333 126.979478,37.559350 126.979522,37.559364 126.980778,37.559744 126.980817,37.559756 126.980856,37.559769 126.980894,37.559783 126.980933,37.559797 126.980969,37.559811 126.981008,37.559825 126.981044,37.559839 126.981083,37.559856 126.981119,37.559869 126.981158,37.559886 126.981194,37.559903 126.981231,37.559919 126.981267,37.559936 126.981303,37.559953 126.981339,37.559972 126.981375,37.559989 126.981408,37.560008 126.981444,37.560028 126.981478,37.560044 126.981511,37.560067 126.981544,37.560083 126.981869,37.560261 126.982136,37.560417 126.982164,37.560431 126.982189,37.560444 126.982217,37.560456 126.982244,37.560469 126.982269,37.560481 126.982300,37.560494 126.982328,37.560506 126.982356,37.560514 126.982383,37.560525 126.982414,37.560533 126.982442,37.560544 126.982472,37.560553 126.982503,37.560561 126.982531,37.560569 126.982561,37.560575 126.982592,37.560583 126.982622,37.560589 126.982653,37.560594 126.982683,37.560600 126.982714,37.560603 126.982744,37.560608 126.984639,37.560767 126.986489,37.560978 126.986525,37.560983 126.988339,37.561144 126.989236,37.561203 126.989272,37.561206 126.989308,37.561206 126.989344,37.561208 126.989381,37.561208 126.989417,37.561211 126.989453,37.561211 126.989489,37.561211 126.989525,37.561208 126.989561,37.561208 126.989597,37.561208 126.989633,37.561206 126.989669,37.561203 126.989706,37.561203 126.989742,37.561200 126.989778,37.561194 126.989814,37.561192 126.989850,37.561189 126.989886,37.561183 126.989922,37.561178 126.989958,37.561175 126.989994,37.561167 126.990364,37.561100 126.990394,37.561094 126.990431,37.561092 126.990464,37.561086 126.990478,37.561083 126.990497,37.561083 126.990531,37.561078 126.990564,37.561075 126.990597,37.561072 126.990631,37.561069 126.990667,37.561067 126.990700,37.561064 126.990733,37.561061 126.990767,37.561058 126.990800,37.561058 126.990833,37.561056 126.990869,37.561056 126.990903,37.561053 126.990936,37.561053 126.990969,37.561053 126.991006,37.561053 126.991039,37.561053 126.991072,37.561053 126.992567,37.561075 126.993236,37.561078 126.993447,37.561100 126.994111,37.561242 126.994172,37.561256 126.995656,37.561569 127.004903,37.563719 127.005561,37.563858 127.006442,37.564022 127.006633,37.564058 127.006664,37.564067 127.006692,37.564075 127.006719,37.564081 127.006750,37.564092 127.006778,37.564100 127.006803,37.564111 127.006831,37.564122 127.006856,37.564133 127.006883,37.564147 127.006908,37.564158 127.006931,37.564175 127.006956,37.564189 127.006978,37.564206 127.007000,37.564219 127.007014,37.564231 127.007022,37.564239 127.007044,37.564256 127.007064,37.564272 127.007083,37.564292 127.007100,37.564311 127.007119,37.564331 127.007133,37.564350 127.007333,37.564619 127.007356,37.564644"
              }
            },
            {
              "mode": "WALK",
              "sectionTime": 240,
              "distance": 274,
              "start": {
                "name": "동대문역사문화공원",
                "lon": 127.00735555555555,
                "lat": 37.56464444444445
              },
              "end": {
                "name": "을지로6가.국립중앙의료원",
                "lon": 127.006225,
                "lat": 37.56619444444444
              },
              "passShape": {
                "linestring": "127.007356,37.564644 127.007442,37.564703 127.007164,37.564858 127.007211,37.564989 127.007206,37.565003 127.007217,37.565044 127.007306,37.565292 127.007361,37.565428 127.007464,37.565703 127.007472,37.565739 127.007475,37.565769 127.007467,37.565828 127.007444,37.565889 127.007386,37.565950 127.007342,37.565969 127.007294,37.565994 127.007192,37.566014 127.007161,37.566014 127.006753,37.566078 127.006706,37.566092 127.006592,37.566106 127.006225,37.566194"
              }
            },
            {
              "routeColor": "0068B7",
              "distance": 3948,
              "start": {
                "name": "을지로6가.국립중앙의료원",
                "lon": 127.006225,
                "lat": 37.56619444444444
              },
              "Lane": [
                {
                  "routeColor": "0068B7",
                  "route": "간선:105",
                  "routeId": "11426001",
                  "service": 1,
                  "type": 11
                },
                {
                  "routeColor": "0068B7",
                  "route": "간선:202",
                  "routeId": "3078001",
                  "service": 1,
                  "type": 11
                }
              ],
              "type": 11,
              "mode": "BUS",
              "sectionTime": 1129,
              "route": "간선:261",
              "routeId": "11444001",
              "service": 1,
              "passStopList": {
                "stationList": [
                  {
                    "index": 0,
                    "stationName": "을지로6가.국립중앙의료원",
                    "lon": "127.006225",
                    "lat": "37.566194",
                    "stationID": "774962"
                  },
                  {
                    "index": 1,
                    "stationName": "동대문역사문화공원",
                    "lon": "127.009017",
                    "lat": "37.567569",
                    "stationID": "775011"
                  },
                  {
                    "index": 2,
                    "stationName": "동대문역.흥인지문",
                    "lon": "127.013417",
                    "lat": "37.572250",
                    "stationID": "775153"
                  },
                  {
                    "index": 3,
                    "stationName": "동묘앞",
                    "lon": "127.018736",
                    "lat": "37.573803",
                    "stationID": "775214"
                  },
                  {
                    "index": 4,
                    "stationName": "신설동역",
                    "lon": "127.025250",
                    "lat": "37.576150",
                    "stationID": "775282"
                  },
                  {
                    "index": 5,
                    "stationName": "용두동사거리",
                    "lon": "127.030111",
                    "lat": "37.577539",
                    "stationID": "775322"
                  },
                  {
                    "index": 6,
                    "stationName": "제기동역.서울약령시",
                    "lon": "127.037442",
                    "lat": "37.578072",
                    "stationID": "775336"
                  },
                  {
                    "index": 7,
                    "stationName": "청량리수산시장",
                    "lon": "127.041825",
                    "lat": "37.579014",
                    "stationID": "775365"
                  },
                  {
                    "index": 8,
                    "stationName": "청량리역환승센터(2번승강장)",
                    "lon": "127.045728",
                    "lat": "37.580225",
                    "stationID": "775401"
                  }
                ]
              },
              "end": {
                "name": "청량리역환승센터(2번승강장)",
                "lon": 127.04572777777778,
                "lat": 37.580225
              },
              "passShape": {
                "linestring": "127.006228,37.566231 127.007200,37.566053 127.007714,37.565956 127.007939,37.566056 127.008106,37.566431 127.008947,37.567617 127.009025,37.567725 127.009686,37.568731 127.009750,37.568869 127.009828,37.569236 127.009844,37.569428 127.009833,37.569608 127.009778,37.569958 127.009708,37.570239 127.009578,37.570614 127.009378,37.571011 127.009239,37.571231 127.009181,37.571314 127.009489,37.571400 127.010361,37.571564 127.010828,37.571633 127.012281,37.571950 127.013406,37.572283 127.015558,37.572922 127.015733,37.572972 127.017636,37.573467 127.018711,37.573833 127.020483,37.574436 127.022508,37.575214 127.023142,37.575464 127.023414,37.575578 127.023914,37.575775 127.025200,37.576175 127.025228,37.576183 127.028344,37.577056 127.030089,37.577569 127.030400,37.577661 127.030581,37.577714 127.031100,37.577856 127.031703,37.577994 127.032378,37.578072 127.036372,37.578106 127.037419,37.578106 127.038231,37.578103 127.038414,37.578111 127.038728,37.578144 127.039303,37.578253 127.041822,37.579050 127.042503,37.579267 127.042672,37.579319 127.045083,37.580072 127.045256,37.580131 127.045558,37.580256 127.045669,37.580322"
              }
            },
            {
              "mode": "WALK",
              "sectionTime": 524,
              "distance": 570,
              "start": {
                "name": "청량리역환승센터(2번승강장)",
                "lon": 127.04572777777778,
                "lat": 37.580225
              },
              "end": {
                "name": "도착지",
                "lon": 127.048958,
                "lat": 37.581381
              },
              "steps": [
                {
                  "streetName": "보행자도로",
                  "distance": 8,
                  "description": "보행자도로 을 따라 8m 이동",
                  "linestring": "127.04572,37.58023 127.045784,37.580273"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 11,
                  "description": "청량리역  4번출구 에서 우측 횡단보도 후 보행자도로 을 따라 11m 이동 ",
                  "linestring": "127.045784,37.580273 127.045876,37.5802"
                },
                {
                  "streetName": "왕산로",
                  "distance": 122,
                  "description": "청량리역  4번출구 에서 좌회전 후 왕산로 을 따라 122m 이동 ",
                  "linestring": "127.045876,37.5802 127.04594,37.58025 127.04599,37.580303 127.046104,37.58046 127.04615,37.580517 127.046196,37.580578 127.046295,37.58071 127.04644,37.580803 127.04654,37.58086 127.04664,37.58092 127.04674,37.580936 127.046814,37.580925"
                },
                {
                  "streetName": "",
                  "distance": 25,
                  "description": "KFC 청량리점 에서 좌측 횡단보도 후 25m 이동 ",
                  "linestring": "127.046814,37.580925 127.04683,37.580956 127.04684,37.580994 127.04685,37.58104 127.04687,37.581104 127.046875,37.58114"
                },
                {
                  "streetName": "",
                  "distance": 16,
                  "description": "직진 후 16m 이동 ",
                  "linestring": "127.046875,37.58114 127.04689,37.58125 127.046906,37.581284"
                },
                {
                  "streetName": "",
                  "distance": 15,
                  "description": "횡단보도 후 15m 이동 ",
                  "linestring": "127.046906,37.581284 127.04703,37.581375"
                },
                {
                  "streetName": "",
                  "distance": 304,
                  "description": "훼미리마트 청량리역점 에서 우회전 후 304m 이동 ",
                  "linestring": "127.04703,37.581375 127.04703,37.581326 127.04703,37.581295 127.04706,37.581257 127.0471,37.581226 127.047165,37.581207 127.047226,37.581196 127.04729,37.581196 127.047356,37.581207 127.047424,37.581226 127.04754,37.581272 127.047646,37.581345 127.04775,37.581417 127.04815,37.581753 127.04833,37.581894 127.04845,37.581947 127.04858,37.581978 127.04877,37.581997 127.04888,37.58199 127.04908,37.581936 127.04913,37.58191 127.04942,37.58177 127.04982,37.581577"
                },
                {
                  "streetName": "",
                  "distance": 17,
                  "description": "우측 횡단보도 후 17m 이동 ",
                  "linestring": "127.04982,37.581577 127.04971,37.58145"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 52,
                  "description": "우회전 후 보행자도로 을 따라 52m 이동 ",
                  "linestring": "127.04971,37.58145 127.04921,37.581703"
                }
              ]
            }
          ],
          "totalWalkTime": 865,
          "transferCount": 1,
          "totalDistance": 47075,
          "pathType": 3,
          "totalWalkDistance": 965
        },
        {
          "fare": {
            "regular": {
              "totalFare": 10400,
              "currency": {
                "symbol": "￦",
                "currency": "원",
                "currencyCode": "KRW"
              }
            }
          },
          "totalTime": 8810,
          "legs": [
            {
              "mode": "WALK",
              "sectionTime": 101,
              "distance": 121,
              "start": {
                "name": "출발지",
                "lon": 126.8230369,
                "lat": 37.3168267
              },
              "end": {
                "name": "고잔",
                "lon": 126.82304722222223,
                "lat": 37.316833333333335
              },
              "steps": [
                {
                  "streetName": "",
                  "distance": 27,
                  "description": "27m 이동",
                  "linestring": "126.82303,37.3168 126.82306,37.3168 126.82333,37.316788"
                },
                {
                  "streetName": "",
                  "distance": 15,
                  "description": "고잔역 에서 좌회전 후 15m 이동 ",
                  "linestring": "126.82333,37.316788 126.82334,37.31693"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 20,
                  "description": "직진 후 보행자도로 을 따라 20m 이동 ",
                  "linestring": "126.82334,37.31693 126.82336,37.317104"
                },
                {
                  "streetName": "중앙대로",
                  "distance": 39,
                  "description": "좌회전 후 중앙대로 을 따라 39m 이동 ",
                  "linestring": "126.82336,37.317104 126.822914,37.317135"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 20,
                  "description": "좌회전 후 보행자도로 을 따라 20m 이동 ",
                  "linestring": "126.822914,37.317135 126.82289,37.316963"
                }
              ]
            },
            {
              "mode": "SUBWAY",
              "routeColor": "00A5DE",
              "sectionTime": 3361,
              "route": "수도권4호선",
              "routeId": "110041005",
              "distance": 35317,
              "service": 1,
              "start": {
                "name": "고잔",
                "lon": 126.82304722222223,
                "lat": 37.316833333333335
              },
              "passStopList": {
                "stationList": [
                  {
                    "index": 0,
                    "stationName": "고잔",
                    "lon": "126.823047",
                    "lat": "37.316833",
                    "stationID": "110446"
                  },
                  {
                    "index": 1,
                    "stationName": "중앙",
                    "lon": "126.838481",
                    "lat": "37.316058",
                    "stationID": "110445"
                  },
                  {
                    "index": 2,
                    "stationName": "한대앞",
                    "lon": "126.853636",
                    "lat": "37.309722",
                    "stationID": "110444"
                  },
                  {
                    "index": 3,
                    "stationName": "상록수",
                    "lon": "126.866383",
                    "lat": "37.302869",
                    "stationID": "110443"
                  },
                  {
                    "index": 4,
                    "stationName": "반월",
                    "lon": "126.903367",
                    "lat": "37.311867",
                    "stationID": "110442"
                  },
                  {
                    "index": 5,
                    "stationName": "대야미",
                    "lon": "126.916878",
                    "lat": "37.327964",
                    "stationID": "110441"
                  },
                  {
                    "index": 6,
                    "stationName": "수리산",
                    "lon": "126.925617",
                    "lat": "37.350569",
                    "stationID": "110440"
                  },
                  {
                    "index": 7,
                    "stationName": "산본",
                    "lon": "126.933019",
                    "lat": "37.357989",
                    "stationID": "110439"
                  },
                  {
                    "index": 8,
                    "stationName": "금정",
                    "lon": "126.943608",
                    "lat": "37.372211",
                    "stationID": "110438"
                  },
                  {
                    "index": 9,
                    "stationName": "범계",
                    "lon": "126.950675",
                    "lat": "37.389764",
                    "stationID": "110437"
                  },
                  {
                    "index": 10,
                    "stationName": "평촌",
                    "lon": "126.964106",
                    "lat": "37.394431",
                    "stationID": "110436"
                  },
                  {
                    "index": 11,
                    "stationName": "인덕원",
                    "lon": "126.976800",
                    "lat": "37.401900",
                    "stationID": "110435"
                  },
                  {
                    "index": 12,
                    "stationName": "정부과천청사",
                    "lon": "126.989942",
                    "lat": "37.426792",
                    "stationID": "110434"
                  },
                  {
                    "index": 13,
                    "stationName": "과천",
                    "lon": "126.996669",
                    "lat": "37.433281",
                    "stationID": "110433"
                  },
                  {
                    "index": 14,
                    "stationName": "대공원",
                    "lon": "127.006531",
                    "lat": "37.435831",
                    "stationID": "110432"
                  },
                  {
                    "index": 15,
                    "stationName": "경마공원",
                    "lon": "127.007944",
                    "lat": "37.443806",
                    "stationID": "110431"
                  },
                  {
                    "index": 16,
                    "stationName": "선바위",
                    "lon": "127.001972",
                    "lat": "37.452111",
                    "stationID": "110430"
                  },
                  {
                    "index": 17,
                    "stationName": "남태령",
                    "lon": "126.988883",
                    "lat": "37.464647",
                    "stationID": "110429"
                  },
                  {
                    "index": 18,
                    "stationName": "사당",
                    "lon": "126.981667",
                    "lat": "37.476808",
                    "stationID": "110428"
                  },
                  {
                    "index": 19,
                    "stationName": "총신대입구",
                    "lon": "126.982264",
                    "lat": "37.487347",
                    "stationID": "110427"
                  },
                  {
                    "index": 20,
                    "stationName": "동작",
                    "lon": "126.980344",
                    "lat": "37.502897",
                    "stationID": "110426"
                  },
                  {
                    "index": 21,
                    "stationName": "이촌",
                    "lon": "126.974444",
                    "lat": "37.522350",
                    "stationID": "110425"
                  }
                ]
              },
              "end": {
                "name": "이촌",
                "lon": 126.97444444444444,
                "lat": 37.52235
              },
              "type": 4,
              "passShape": {
                "linestring": "126.823047,37.316833 126.824414,37.316767 126.826906,37.316628 126.830992,37.316458 126.832469,37.316378 126.833967,37.316292 126.835467,37.316217 126.836961,37.316139 126.838481,37.316061 126.839886,37.315986 126.842467,37.315819 126.844714,37.315669 126.845500,37.315581 126.846008,37.315494 126.846500,37.315406 126.846953,37.315281 126.847328,37.315158 126.847739,37.314989 126.848297,37.314700 126.848778,37.314403 126.849225,37.314086 126.849656,37.313689 126.850683,37.312733 126.851578,37.311858 126.852081,37.311344 126.852964,37.310425 126.853639,37.309722 126.853753,37.309606 126.854828,37.308542 126.855744,37.307639 126.856328,37.307072 126.857392,37.306008 126.857856,37.305514 126.858386,37.304958 126.858814,37.304581 126.859214,37.304283 126.859542,37.304086 126.859942,37.303889 126.860364,37.303700 126.860919,37.303478 126.861364,37.303333 126.861831,37.303228 126.862433,37.303131 126.863000,37.303081 126.863614,37.303056 126.864522,37.302989 126.864847,37.302964 126.866383,37.302867 126.867319,37.302806 126.871492,37.302481 126.872903,37.302453 126.873700,37.302464 126.874558,37.302494 126.875353,37.302522 126.878306,37.302639 126.879389,37.302683 126.879444,37.302686 126.880114,37.302728 126.881167,37.302778 126.881628,37.302781 126.882600,37.302847 126.883436,37.303019 126.884569,37.303317 126.885189,37.303486 126.885669,37.303642 126.886589,37.303892 126.888092,37.304297 126.888661,37.304450 126.890783,37.305022 126.892403,37.305458 126.892564,37.305503 126.892981,37.305614 126.894681,37.306069 126.895633,37.306331 126.896247,37.306492 126.896492,37.306558 126.896719,37.306622 126.896942,37.306689 126.897150,37.306756 126.897361,37.306831 126.897625,37.306928 126.897911,37.307047 126.898361,37.307239 126.898828,37.307475 126.899233,37.307703 126.899661,37.307931 126.899892,37.308075 126.900097,37.308219 126.900678,37.308725 126.900789,37.308831 126.901239,37.309314 126.901683,37.309800 126.902156,37.310414 126.902169,37.310428 126.903367,37.311867 126.904406,37.313108 126.905708,37.314681 126.907406,37.316717 126.908850,37.318425 126.910258,37.320100 126.911522,37.321614 126.912292,37.322533 126.912761,37.323092 126.913981,37.324544 126.914781,37.325519 126.915297,37.326094 126.915756,37.326639 126.916767,37.327828 126.916878,37.327964 126.917661,37.328922 126.917814,37.329117 126.918161,37.329506 126.918344,37.329744 126.918844,37.330286 126.919256,37.330789 126.919794,37.331383 126.920208,37.331872 126.920333,37.332044 126.920756,37.332558 126.921186,37.333181 126.921350,37.333408 126.921494,37.333658 126.921644,37.333936 126.921775,37.334258 126.922056,37.334978 126.922269,37.335875 126.922333,37.336169 126.922636,37.337656 126.923267,37.340964 126.923528,37.342833 126.923919,37.344722 126.924153,37.345831 126.925533,37.350419 126.925583,37.350578 126.925914,37.351614 126.926225,37.352608 126.926347,37.352931 126.926475,37.353253 126.926664,37.353675 126.926964,37.354200 126.927178,37.354453 126.927636,37.354939 126.928156,37.355353 126.930842,37.356931 126.932967,37.358056 126.935383,37.359333 126.936328,37.359833 126.939789,37.361636 126.942525,37.363175 126.944708,37.364322 126.944894,37.364464 126.945069,37.364628 126.945267,37.364825 126.945411,37.365006 126.945514,37.365156 126.945586,37.365269 126.945653,37.365397 126.945731,37.365625 126.945792,37.365825 126.945825,37.366022 126.945839,37.366267 126.945839,37.366431 126.945808,37.366608 126.945775,37.366781 126.945708,37.366978 126.945628,37.367172 126.945536,37.367378 126.945350,37.367761 126.945236,37.367994 126.945072,37.368339 126.945006,37.368506 126.943858,37.371331 126.943644,37.371906 126.943539,37.372194 126.943325,37.372775 126.942281,37.374958 126.941906,37.375700 126.941767,37.375947 126.941228,37.376814 126.940958,37.377247 126.940797,37.377472 126.940664,37.377628 126.940542,37.377758 126.940036,37.378289 126.939917,37.378406 126.938700,37.379531 126.938669,37.379575 126.938653,37.379619 126.938642,37.379664 126.938633,37.379706 126.938636,37.379767 126.938644,37.379828 126.938872,37.380197 126.939181,37.380786 126.939919,37.382556 126.940189,37.383083 126.940436,37.383478 126.940786,37.383844 126.941064,37.384042 126.941464,37.384264 126.941989,37.384469 126.942383,37.384636 126.942733,37.384825 126.942994,37.384981 126.943428,37.385258 126.943864,37.385597 126.945186,37.386678 126.947628,37.388694 126.947642,37.388703 126.947653,37.388714 126.947672,37.388722 126.947686,37.388728 126.947700,37.388733 126.950675,37.389767 126.962272,37.393797 126.964086,37.394467 126.964389,37.394578 126.968514,37.395942 126.971767,37.397033 126.973083,37.397500 126.974419,37.397997 126.975417,37.398531 126.975703,37.398761 126.975883,37.398925 126.976144,37.399208 126.976603,37.399822 126.976797,37.400136 126.976881,37.400292 126.976914,37.400467 126.976833,37.401311 126.976833,37.401350 126.976783,37.402194 126.976803,37.402544 126.976883,37.402842 126.977764,37.404747 126.977808,37.404872 126.978228,37.406642 126.978494,37.407764 126.979033,37.410022 126.979122,37.410444 126.979419,37.411742 126.979725,37.412986 126.979781,37.413208 126.979867,37.413675 126.979908,37.413847 126.980011,37.414200 126.980928,37.416297 126.981547,37.417675 126.981750,37.418144 126.982047,37.418675 126.982469,37.419406 126.982989,37.420228 126.983950,37.421275 126.984853,37.422069 126.985869,37.422925 126.987003,37.423869 126.987781,37.424625 126.988981,37.425722 126.989956,37.426783 126.990411,37.427278 126.991381,37.428233 126.992906,37.429739 126.995314,37.432433 126.995839,37.432964 126.996069,37.433125 126.996378,37.433244 126.996656,37.433317 126.996756,37.433342 126.997372,37.433425 126.997742,37.433425 126.998319,37.433353 126.999225,37.433258 126.999853,37.433225 127.000758,37.433239 127.004331,37.433214 127.004481,37.433256 127.005314,37.433958 127.005433,37.434131 127.006244,37.435392 127.006450,37.435711 127.008228,37.438322 127.008297,37.438447 127.008344,37.438575 127.008364,37.438711 127.008375,37.438969 127.008203,37.440986 127.008019,37.442944 127.007969,37.443506 127.007944,37.443831 127.007819,37.444217 127.007717,37.444408 127.007436,37.444817 127.006928,37.445475 127.004367,37.448900 127.002544,37.451400 127.001972,37.452111 127.001836,37.452281 127.001633,37.452472 127.001442,37.452575 127.000942,37.452764 126.999947,37.453253 126.998028,37.454378 126.997922,37.454442 126.996819,37.455239 126.995569,37.456381 126.994317,37.457522 126.993775,37.458058 126.993728,37.458117 126.993250,37.458667 126.991156,37.461425 126.989422,37.463706 126.989111,37.464247 126.988883,37.464647 126.987964,37.466256 126.987881,37.466394 126.987797,37.466528 126.987708,37.466664 126.987617,37.466794 126.987525,37.466928 126.987428,37.467058 126.987328,37.467186 126.987225,37.467314 126.987122,37.467442 126.987014,37.467567 126.986903,37.467689 126.986792,37.467811 126.986675,37.467931 126.986558,37.468050 126.986439,37.468167 126.986314,37.468283 126.986189,37.468394 126.986061,37.468508 126.985931,37.468619 126.985800,37.468728 126.985664,37.468833 126.985211,37.469178 126.984864,37.469464 126.984636,37.469658 126.984472,37.469808 126.984356,37.469928 126.983858,37.470544 126.983769,37.470672 126.983256,37.471453 126.982953,37.472042 126.982397,37.473183 126.982053,37.474025 126.981878,37.474547 126.981769,37.474950 126.981675,37.475608 126.981669,37.475658 126.981633,37.476083 126.981656,37.476564 126.981658,37.476653 126.982111,37.485253 126.982117,37.485383 126.982122,37.485475 126.982158,37.486308 126.982228,37.487350 126.982233,37.487400 126.982378,37.488686 126.982531,37.489653 126.982592,37.490369 126.982647,37.494967 126.982628,37.496269 126.982639,37.496469 126.982664,37.496669 126.982692,37.496817 126.982744,37.497011 126.982794,37.497156 126.982983,37.497625 126.983100,37.497950 126.983239,37.498383 126.983267,37.498531 126.983264,37.498681 126.983233,37.498825 126.983178,37.498967 126.983067,37.499131 126.981803,37.500881 126.981647,37.501097 126.981553,37.501222 126.981425,37.501369 126.980922,37.501886 126.980856,37.501964 126.980794,37.502033 126.980644,37.502233 126.980547,37.502394 126.980464,37.502575 126.980381,37.502808 126.980294,37.503031 126.980269,37.503125 126.980183,37.503464 126.980156,37.503878 126.980156,37.504058 126.980164,37.504244 126.980186,37.504425 126.980258,37.504800 126.980719,37.506508 126.983114,37.515581 126.983161,37.515808 126.983203,37.516089 126.983231,37.516306 126.983269,37.517844 126.983278,37.518414 126.983261,37.518581 126.983225,37.518736 126.983183,37.518883 126.983128,37.519019 126.983053,37.519164 126.982961,37.519292 126.982853,37.519408 126.982725,37.519531 126.982633,37.519608 126.982606,37.519633 126.982456,37.519739 126.982292,37.519836 126.982108,37.519936 126.981919,37.520033 126.981719,37.520119 126.981503,37.520206 126.976931,37.521594 126.975139,37.522128 126.974444,37.522344"
              }
            },
            {
              "mode": "WALK",
              "sectionTime": 148,
              "distance": 135,
              "start": {
                "name": "이촌",
                "lon": 126.97444444444444,
                "lat": 37.52235
              },
              "end": {
                "name": "이촌",
                "lon": 126.97354722222222,
                "lat": 37.5224
              },
              "passShape": {
                "linestring": "126.974444,37.522350 126.973544,37.522650 126.973483,37.522525 126.973397,37.522556 126.973356,37.522542 126.973547,37.522400"
              }
            },
            {
              "routeColor": "77C4A3",
              "distance": 16441,
              "start": {
                "name": "이촌",
                "lon": 126.97354722222222,
                "lat": 37.5224
              },
              "Lane": [
                {
                  "routeColor": "77C4A3",
                  "route": "경의중앙선(급행)",
                  "routeId": "111121016",
                  "service": 0,
                  "type": 122
                }
              ],
              "type": 104,
              "mode": "SUBWAY",
              "sectionTime": 1800,
              "route": "경의중앙선",
              "routeId": "110121034",
              "service": 0,
              "passStopList": {
                "stationList": [
                  {
                    "index": 0,
                    "stationName": "이촌",
                    "lon": "126.973547",
                    "lat": "37.522400",
                    "stationID": "111228"
                  },
                  {
                    "index": 1,
                    "stationName": "용산",
                    "lon": "126.964453",
                    "lat": "37.529425",
                    "stationID": "111229"
                  },
                  {
                    "index": 2,
                    "stationName": "효창공원앞",
                    "lon": "126.962394",
                    "lat": "37.538711",
                    "stationID": "111230"
                  },
                  {
                    "index": 3,
                    "stationName": "공덕",
                    "lon": "126.952117",
                    "lat": "37.542575",
                    "stationID": "111231"
                  },
                  {
                    "index": 4,
                    "stationName": "서강대",
                    "lon": "126.935378",
                    "lat": "37.552161",
                    "stationID": "111232"
                  },
                  {
                    "index": 5,
                    "stationName": "홍대입구",
                    "lon": "126.928183",
                    "lat": "37.556753",
                    "stationID": "111233"
                  },
                  {
                    "index": 6,
                    "stationName": "가좌",
                    "lon": "126.914889",
                    "lat": "37.568833",
                    "stationID": "111236"
                  },
                  {
                    "index": 7,
                    "stationName": "디지털미디어시티",
                    "lon": "126.900639",
                    "lat": "37.577411",
                    "stationID": "111237"
                  },
                  {
                    "index": 8,
                    "stationName": "수색",
                    "lon": "126.895600",
                    "lat": "37.580919",
                    "stationID": "111238"
                  },
                  {
                    "index": 9,
                    "stationName": "한국항공대",
                    "lon": "126.868378",
                    "lat": "37.602919",
                    "stationID": "111239"
                  },
                  {
                    "index": 10,
                    "stationName": "강매",
                    "lon": "126.844686",
                    "lat": "37.612272",
                    "stationID": "111240"
                  },
                  {
                    "index": 11,
                    "stationName": "행신",
                    "lon": "126.834283",
                    "lat": "37.612139",
                    "stationID": "111241"
                  }
                ]
              },
              "end": {
                "name": "행신",
                "lon": 126.83428333333333,
                "lat": 37.612138888888886
              },
              "passShape": {
                "linestring": "126.973550,37.522408 126.973425,37.522450 126.972692,37.522664 126.972542,37.522700 126.972392,37.522733 126.972239,37.522764 126.972083,37.522789 126.971931,37.522814 126.971775,37.522833 126.971619,37.522853 126.971464,37.522867 126.971306,37.522878 126.971150,37.522886 126.970992,37.522892 126.970836,37.522894 126.970678,37.522894 126.970522,37.522889 126.970364,37.522881 126.970208,37.522869 126.970053,37.522858 126.969894,37.522842 126.969739,37.522819 126.969586,37.522797 126.969431,37.522772 126.969333,37.522756 126.969239,37.522739 126.969142,37.522719 126.969044,37.522703 126.968947,37.522683 126.968850,37.522664 126.968753,37.522644 126.968656,37.522625 126.968561,37.522606 126.968464,37.522586 126.968369,37.522567 126.968272,37.522544 126.968178,37.522525 126.968081,37.522503 126.967983,37.522481 126.967889,37.522458 126.967794,37.522436 126.967700,37.522414 126.967603,37.522389 126.967508,37.522367 126.967414,37.522342 126.967253,37.522303 126.967089,37.522264 126.966928,37.522231 126.966764,37.522200 126.966597,37.522172 126.966431,37.522147 126.966264,37.522125 126.966097,37.522106 126.965931,37.522089 126.965761,37.522078 126.965592,37.522067 126.965425,37.522061 126.965256,37.522058 126.965086,37.522058 126.964917,37.522061 126.964750,37.522069 126.964581,37.522078 126.964411,37.522092 126.964244,37.522106 126.964078,37.522125 126.963908,37.522147 126.963525,37.522219 126.963150,37.522325 126.962789,37.522456 126.962450,37.522617 126.962128,37.522803 126.961833,37.523017 126.961567,37.523250 126.961369,37.523497 126.961214,37.523758 126.961075,37.524033 126.960958,37.524342 126.960853,37.524681 126.960783,37.525003 126.960747,37.525292 126.960744,37.525597 126.960811,37.525892 126.960917,37.526208 126.961064,37.526522 126.961253,37.526814 126.961497,37.527108 126.961672,37.527306 126.961894,37.527511 126.964453,37.529425 126.964469,37.529439 126.965803,37.530392 126.965850,37.530428 126.965900,37.530461 126.965947,37.530497 126.965997,37.530531 126.966047,37.530564 126.966097,37.530597 126.966147,37.530631 126.966200,37.530661 126.966250,37.530692 126.966303,37.530725 126.966356,37.530756 126.966408,37.530783 126.966461,37.530814 126.966517,37.530844 126.966569,37.530872 126.966625,37.530900 126.966681,37.530928 126.966736,37.530956 126.966792,37.530981 126.966850,37.531006 126.966906,37.531031 126.967703,37.531439 126.967922,37.531581 126.968128,37.531736 126.968319,37.531903 126.968494,37.532081 126.968656,37.532267 126.968794,37.532464 126.968917,37.532667 126.969019,37.532878 126.969103,37.533092 126.969106,37.533106 126.969164,37.533314 126.969206,37.533536 126.969228,37.533761 126.969228,37.533986 126.969206,37.534211 126.969164,37.534436 126.969100,37.534656 126.969017,37.534869 126.968914,37.535081 126.968792,37.535283 126.968642,37.535481 126.968508,37.535658 126.968367,37.535844 126.968050,37.536242 126.967764,37.536561 126.967528,37.536806 126.967214,37.537069 126.966875,37.537294 126.966528,37.537483 126.966144,37.537658 126.965739,37.537803 126.965317,37.537914 126.962369,37.538647 126.957778,37.539792 126.957719,37.539806 126.957661,37.539822 126.957606,37.539842 126.957556,37.539856 126.957547,37.539858 126.957489,37.539878 126.957433,37.539897 126.957378,37.539917 126.957322,37.539939 126.957267,37.539961 126.957214,37.539986 126.957158,37.540008 126.957131,37.540022 126.957106,37.540033 126.957053,37.540058 126.957003,37.540086 126.956950,37.540114 126.956900,37.540142 126.956850,37.540169 126.956800,37.540200 126.956753,37.540231 126.956706,37.540261 126.956658,37.540292 126.956453,37.540425 126.956078,37.540669 126.955833,37.540806 126.955664,37.540894 126.955547,37.540956 126.953331,37.542003 126.952117,37.542575 126.951197,37.543008 126.950553,37.543297 126.946175,37.545258 126.946033,37.545322 126.945892,37.545383 126.945747,37.545444 126.945603,37.545506 126.945458,37.545564 126.945314,37.545622 126.945167,37.545678 126.945022,37.545733 126.944875,37.545789 126.944728,37.545842 126.944578,37.545894 126.944428,37.545947 126.944281,37.545997 126.944128,37.546044 126.943914,37.546117 126.943675,37.546186 126.943522,37.546231 126.943369,37.546275 126.943217,37.546317 126.943064,37.546358 126.942278,37.546600 126.941472,37.546878 126.941303,37.546939 126.941147,37.547003 126.940958,37.547086 126.940742,37.547194 126.940617,37.547272 126.940503,37.547333 126.940389,37.547406 126.940281,37.547475 126.940136,37.547572 126.939964,37.547703 126.939850,37.547797 126.939758,37.547878 126.939656,37.547975 126.939619,37.548006 126.939553,37.548072 126.939486,37.548136 126.939450,37.548172 126.939364,37.548264 126.939269,37.548367 126.939156,37.548497 126.938958,37.548742 126.937733,37.550358 126.937417,37.550711 126.937208,37.550892 126.935417,37.552197 126.935311,37.552275 126.933925,37.553239 126.932031,37.554556 126.931169,37.555178 126.930500,37.555617 126.929956,37.555939 126.929450,37.556208 126.928869,37.556492 126.928594,37.556611 126.928297,37.556742 126.928208,37.556783 126.928031,37.556869 126.927831,37.556975 126.927594,37.557100 126.927389,37.557214 126.927194,37.557325 126.927031,37.557419 126.926928,37.557486 126.926817,37.557567 126.926731,37.557631 126.926653,37.557683 126.926578,37.557742 126.926503,37.557792 126.926406,37.557864 126.926331,37.557919 126.926256,37.557978 126.926189,37.558036 126.926103,37.558100 126.926028,37.558161 126.925936,37.558239 126.925839,37.558328 126.925822,37.558342 126.925714,37.558436 126.925631,37.558514 126.925564,37.558572 126.925489,37.558647 126.925403,37.558736 126.923878,37.560714 126.919286,37.566456 126.919178,37.566572 126.918953,37.566797 126.918764,37.566964 126.918531,37.567139 126.918442,37.567175 126.918319,37.567228 126.918200,37.567278 126.918081,37.567333 126.917961,37.567389 126.916583,37.568019 126.916000,37.568292 126.915139,37.568706 126.915108,37.568722 126.914564,37.568997 126.913564,37.569456 126.911331,37.570475 126.910764,37.570733 126.910381,37.570903 126.910108,37.571033 126.909833,37.571169 126.909547,37.571317 126.909358,37.571428 126.909083,37.571594 126.908711,37.571828 126.908133,37.572206 126.907372,37.572725 126.906578,37.573242 126.905667,37.573839 126.905181,37.574164 126.904792,37.574428 126.904231,37.574842 126.903308,37.575517 126.902408,37.576175 126.901878,37.576564 126.901872,37.576569 126.900989,37.577178 126.900639,37.577411 126.900378,37.577581 126.900056,37.577794 126.899525,37.578150 126.899192,37.578369 126.898906,37.578572 126.898667,37.578742 126.897594,37.579531 126.896678,37.580194 126.896439,37.580353 126.895956,37.580672 126.895600,37.580919 126.895133,37.581239 126.894772,37.581483 126.893358,37.582422 126.893089,37.582603 126.892775,37.582786 126.892458,37.582975 126.892111,37.583186 126.891653,37.583464 126.891328,37.583653 126.890314,37.584256 126.889958,37.584481 126.889503,37.584781 126.889294,37.584919 126.889139,37.585036 126.888978,37.585158 126.888678,37.585414 126.888319,37.585731 126.887836,37.586133 126.880081,37.592778 126.877667,37.594844 126.876967,37.595444 126.876044,37.596239 126.875000,37.597181 126.873456,37.598550 126.872911,37.599039 126.872442,37.599453 126.871708,37.600053 126.871169,37.600503 126.870883,37.600744 126.870458,37.601114 126.869925,37.601572 126.869419,37.602006 126.868994,37.602394 126.868433,37.602869 126.868378,37.602919 126.867239,37.603883 126.866167,37.604781 126.865342,37.605492 126.864750,37.605983 126.864447,37.606211 126.864017,37.606508 126.863542,37.606786 126.863203,37.606953 126.862878,37.607094 126.862589,37.607206 126.861803,37.607494 126.861331,37.607658 126.860883,37.607825 126.860400,37.608006 126.859806,37.608250 126.859203,37.608489 126.858406,37.608858 126.857881,37.609108 126.857336,37.609383 126.856589,37.609767 126.855828,37.610178 126.855078,37.610600 126.854586,37.610889 126.853614,37.611478 126.853489,37.611544 126.853236,37.611675 126.852975,37.611794 126.852706,37.611906 126.852433,37.612006 126.852294,37.612053 126.852014,37.612139 126.851728,37.612214 126.851583,37.612247 126.851292,37.612308 126.850939,37.612364 126.850706,37.612394 126.850297,37.612431 126.849886,37.612444 126.849256,37.612450 126.848786,37.612436 126.847906,37.612417 126.846472,37.612350 126.844889,37.612281 126.844686,37.612272 126.843569,37.612233 126.842706,37.612208 126.840575,37.612197 126.839539,37.612183 126.837825,37.612156 126.836406,37.612139 126.835306,37.612131 126.834286,37.612117"
              }
            },
            {
              "mode": "WALK",
              "sectionTime": 45,
              "distance": 52,
              "start": {
                "name": "행신",
                "lon": 126.83428333333333,
                "lat": 37.612138888888886
              },
              "end": {
                "name": "행신",
                "lon": 126.83428333333333,
                "lat": 37.61199444444444
              },
              "passShape": {
                "linestring": "126.834283,37.612139 126.834219,37.612117 126.834108,37.612117 126.834106,37.611928 126.834283,37.611994"
              }
            },
            {
              "routeColor": "F75C4C",
              "distance": 19233,
              "start": {
                "name": "행신",
                "lon": 126.83428333333333,
                "lat": 37.61199444444444
              },
              "type": 3,
              "mode": "TRAIN",
              "sectionTime": 2940,
              "route": "KTX이음:행신-청량리",
              "routeId": "500069-500010-KTXEUM001",
              "service": 0,
              "passStopList": {
                "stationList": [
                  {
                    "index": 0,
                    "stationName": "행신",
                    "lon": "126.834283",
                    "lat": "37.611994",
                    "stationID": "500069"
                  },
                  {
                    "index": 1,
                    "stationName": "청량리",
                    "lon": "127.048500",
                    "lat": "37.580742",
                    "stationID": "500010"
                  }
                ]
              },
              "end": {
                "name": "청량리",
                "lon": 127.0485,
                "lat": 37.58074166666667
              },
              "passShape": {
                "linestring": "126.834283,37.611994 127.048500,37.580742"
              },
              "routePayment": 8400
            },
            {
              "mode": "WALK",
              "sectionTime": 415,
              "distance": 492,
              "start": {
                "name": "청량리",
                "lon": 127.0485,
                "lat": 37.58074166666667
              },
              "end": {
                "name": "도착지",
                "lon": 127.048958,
                "lat": 37.581381
              },
              "steps": [
                {
                  "streetName": "",
                  "distance": 109,
                  "description": "109m 이동",
                  "linestring": "127.04837,37.580803 127.04826,37.58067 127.04796,37.580284 127.047714,37.579975"
                },
                {
                  "streetName": "",
                  "distance": 72,
                  "description": "던킨도너츠 청량리역사점 에서 좌회전 후 72m 이동 ",
                  "linestring": "127.047714,37.579975 127.04787,37.5799 127.04841,37.57964"
                },
                {
                  "streetName": "",
                  "distance": 16,
                  "description": "직진 후 16m 이동 ",
                  "linestring": "127.04841,37.57964 127.04856,37.579563"
                },
                {
                  "streetName": "",
                  "distance": 242,
                  "description": "좌회전 후 242m 이동 ",
                  "linestring": "127.04856,37.579563 127.04963,37.580936 127.04958,37.58105 127.04961,37.58109 127.04965,37.58115 127.04975,37.58128 127.04977,37.581333 127.04976,37.581387 127.04971,37.58145"
                },
                {
                  "streetName": "보행자도로",
                  "distance": 53,
                  "description": "10시 방향 좌회전 후 보행자도로 을 따라 53m 이동 ",
                  "linestring": "127.04971,37.58145 127.04921,37.581703"
                }
              ]
            }
          ],
          "totalWalkTime": 709,
          "transferCount": 2,
          "totalDistance": 71262,
          "pathType": 5,
          "totalWalkDistance": 800
        }
      ]
    }
  }
}'''

# JSON 문자열을 Python 딕셔너리로 변환
data = json.loads(json_data)

# 딕셔너리 확인
print(data)

{'metaData': {'requestParameters': {'busCount': 0, 'expressbusCount': 0, 'subwayCount': 4, 'airplaneCount': 0, 'locale': 'ko', 'endY': '37.581381', 'endX': '127.048958', 'wideareaRouteCount': 0, 'subwayBusCount': 3, 'startY': '37.3168267', 'startX': '126.8230369', 'ferryCount': 0, 'trainCount': 3, 'reqDttm': '20241024224044'}, 'plan': {'itineraries': [{'fare': {'regular': {'totalFare': 2200, 'currency': {'symbol': '￦', 'currency': '원', 'currencyCode': 'KRW'}}}, 'totalTime': 4802, 'legs': [{'mode': 'WALK', 'sectionTime': 101, 'distance': 121, 'start': {'name': '출발지', 'lon': 126.8230369, 'lat': 37.3168267}, 'end': {'name': '고잔', 'lon': 126.82304722222223, 'lat': 37.316833333333335}, 'steps': [{'streetName': '', 'distance': 27, 'description': '27m 이동', 'linestring': '126.82303,37.3168 126.82306,37.3168 126.82333,37.316788'}, {'streetName': '', 'distance': 15, 'description': '고잔역 에서 좌회전 후 15m 이동 ', 'linestring': '126.82333,37.316788 126.82334,37.31693'}, {'streetName': '보행자도로', 'distance':

최종) 경로 정보 활용하기 위해 route_data 전역 변수 선언

In [ ]:
from flask import Flask, request, jsonify
import requests
from datetime import datetime

app = Flask(__name__)

API_KEY = 'YEWVxfrK4j8xTNQZURJ4z1Te4JTZs26v45fgmfn7'
TMAP_URL = "https://apis.openapi.sk.com/transit/routes"

# 전역 변수로 선언
route_data = None

@app.route('/find_route', methods=['POST'])
def find_route():
    global route_data  # 전역 변수 사용 선언
    
    # 프론트엔드에서 경도, 위도 좌표를 받아옴
    data = request.get_json()
    start_lon = data.get('start_lon')
    start_lat = data.get('start_lat')
    end_lon = data.get('end_lon')
    end_lat = data.get('end_lat')

    # Tmap API에 요청할 페이로드
    payload = {
        "startX": start_lon,
        "startY": start_lat,
        "endX": end_lon,
        "endY": end_lat,
        "lang": 0,
        "format": "json",
        "count": 10,
        "searchDttm": datetime.now().strftime("%Y%m%d%H%M")
    }

    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "appKey": API_KEY
    }

    # Tmap API 요청
    try:
        response = requests.post(TMAP_URL, json=payload, headers=headers)
        route_data = response.json()  # 전역 변수에 할당
        return jsonify(route_data), 200
    except requests.RequestException as e:
        return jsonify({"error": "Unable to find route"}), 500

if __name__ == '__main__':
    app.run(debug=True)


경로 상 지하철역 이름과 호선 정보 추출

In [6]:
import json

# 'itineraries'는 경로 정보를 담고 있는 배열
itineraries = route_data['metaData']['plan']['itineraries']

# 각 'itinerary'에서 legs 확인
for idx, itinerary in enumerate(itineraries):
    print(f"### 경로 {idx + 1} ###")  # 경로 번호 출력
    
    # 경로에 포함된 지하철역 이름과 호선 정보를 담을 리스트
    subway_station_info = []
    
    for leg in itinerary['legs']:
        # 지하철 경로인지 확인
        if leg['mode'] == 'SUBWAY':
            route_name = leg['route']  # 호선 정보
            # 'passStopList'에서 지하철역 목록 추출
            for station in leg['passStopList']['stationList']:
                station_name = station['stationName']
                # 역 이름과 호선 정보 함께 저장
                subway_station_info.append({
                    'station_name': station_name,
                    'line': route_name
                })
    
    # 추출된 지하철역 이름과 호선 정보 출력
    if subway_station_info:
        for station_info in subway_station_info:
            print(f"Station: {station_info['station_name']}, Line: {station_info['line']}")
    else:
        print("해당 경로에 지하철 정보가 없습니다.")
    
    print()  # 경로별로 출력 간격을 두기 위해 공백 라인 추가


### 경로 1 ###
Station: 동묘앞, Line: 수도권1호선
Station: 신설동, Line: 수도권1호선

### 경로 2 ###
해당 경로에 지하철 정보가 없습니다.

### 경로 3 ###
해당 경로에 지하철 정보가 없습니다.

### 경로 4 ###
해당 경로에 지하철 정보가 없습니다.

### 경로 5 ###
해당 경로에 지하철 정보가 없습니다.

### 경로 6 ###
해당 경로에 지하철 정보가 없습니다.

[]


최종) 경로상 지하철역 정보 추출 + 경로별로 저장, 지하철 없는 경로는 저장하지 않음

In [9]:
import json

# 'itineraries'는 경로 정보를 담고 있는 배열
itineraries = route_data['metaData']['plan']['itineraries'] 
### 실제로는 data(X) route_data(O) data는 테스트용! ###

# 경로별 지하철 정보를 저장할 딕셔너리
all_subway_info = {}

# 각 'itinerary'에서 legs 확인
for idx, itinerary in enumerate(itineraries):
    
    # 경로에 포함된 지하철역 이름과 호선 정보를 담을 리스트
    subway_station_info = []
    
    for leg in itinerary['legs']:
        # 지하철 경로인지 확인
        if leg['mode'] == 'SUBWAY':
            route_name = leg['route']  # 호선 정보
            if '수도권' in route_name or '(급행)' in route_name:
                route_name = route_name.replace('수도권', '').replace('(급행)', '').strip()
            
            # 'passStopList'에서 지하철역 목록 추출
            for station in leg['passStopList']['stationList']:
                station_name = station['stationName']
                
                # 역 이름 뒤에 '역'을 붙임 (중복 방지 포함)
                if not station_name.endswith('역'):
                    station_name += '역'
                
                # 역 이름과 호선 정보 함께 저장
                subway_station_info.append({
                    'station_name': station_name,
                    'line': route_name
                })
    
    # 지하철 정보가 있는 경로만 저장
    if subway_station_info:
        all_subway_info[f'route_{idx + 1}'] = subway_station_info

# 결과 출력
print(all_subway_info)


{'route_1': [{'station_name': '동묘앞역', 'line': '1호선'}, {'station_name': '신설동역', 'line': '1호선'}]}


지하철 혼잡도 api 호출 기본 형태

In [ ]:
import requests

url = "https://apis.openapi.sk.com/transit/puzzle/subway/congestion/stat/train?routeNm=1%ED%98%B8%EC%84%A0&stationNm=%EC%84%9C%EC%9A%B8%EC%97%AD"

headers = {
    "accept": "application/json",
    "appKey": "YEWVxfrK4j8xTNQZURJ4z1Te4JTZs26v45fgmfn7"
}

response = requests.get(url, headers=headers)

print(response.text)

In [8]:
import requests
import urllib.parse
from datetime import datetime

# 노선명과 역명칭
route_name = "1호선"
station_name = "동묘앞역"

# 현재 시간 반영 (요일과 시간)
now = datetime.now()
day_of_week = now.strftime('%a').upper()  # 요일을 MON, TUE 형식으로 반환
hour = int(now.strftime('%H'))  # 현재 시간을 24시간 형식으로 반환

# 시간 범위 제한 (05 ~ 23시)
if hour < 5:
    hour = 5
elif hour > 23:
    hour = 23
hour_str = f"{hour:02d}"  # 두 자리 형식으로 맞추기

# URL 인코딩 (노선명과 역명칭)
encoded_route_name = urllib.parse.quote(route_name)
encoded_station_name = urllib.parse.quote(station_name)

# 완성된 URL
url = f"https://apis.openapi.sk.com/transit/puzzle/subway/congestion/stat/train?routeNm={encoded_route_name}&stationNm={encoded_station_name}&dow={day_of_week}&hh={hour_str}"

# 요청 보내기 (예시)
headers = {
    "accept": "application/json",
    "appKey": "YEWVxfrK4j8xTNQZURJ4z1Te4JTZs26v45fgmfn7"
}

response = requests.get(url, headers=headers)

# 결과 출력
if response.status_code == 200:
    congestion_data = response.json()
    print(congestion_data)
else:
    print(f"Request failed with status code {response.status_code}")


{'status': {'code': '00', 'message': 'success', 'totalCount': 1}, 'contents': {'subwayLine': '1호선', 'stationName': '동묘앞역', 'stationCode': '127', 'stat': [{'startStationCode': '100', 'startStationName': '소요산역', 'endStationCode': '161', 'endStationName': '인천역', 'prevStationCode': '126', 'prevStationName': '신설동역', 'updnLine': 1, 'directAt': 0, 'data': [{'dow': 'THU', 'hh': '23', 'mm': '00', 'congestionTrain': 13}, {'dow': 'THU', 'hh': '23', 'mm': '10', 'congestionTrain': 13}, {'dow': 'THU', 'hh': '23', 'mm': '20', 'congestionTrain': 6}, {'dow': 'THU', 'hh': '23', 'mm': '30', 'congestionTrain': 13}, {'dow': 'THU', 'hh': '23', 'mm': '40', 'congestionTrain': 12}, {'dow': 'THU', 'hh': '23', 'mm': '50', 'congestionTrain': 12}]}, {'startStationCode': '100-3', 'startStationName': '연천역', 'endStationCode': '141', 'endStationName': '구로역', 'prevStationCode': '126', 'prevStationName': '신설동역', 'updnLine': 1, 'directAt': 0, 'data': [{'dow': 'THU', 'hh': '23', 'mm': '00', 'congestionTrain': 7}, {'dow': 

지하철 혼잡도 api 호출 - 현재 시간에 가장 가까운 혼잡도 데이터 추출

In [ ]:
import requests
import urllib.parse
from datetime import datetime
from flask import Flask, jsonify

app = Flask(__name__)

# all_subway_info에서 경로별로 노선명과 역명칭을 가져와서 차례대로 인코딩 및 API 호출
def get_congestion_data(all_subway_info):
    congestion_results = []
    
    # 현재 시간 반영 (요일과 시간)
    now = datetime.now()
    day_of_week = now.strftime('%a').upper()  # 요일을 MON, TUE 형식으로 반환
    current_hour = int(now.strftime('%H'))  # 현재 시간을 24시간 형식으로 반환
    current_minute = int(now.strftime('%M'))  # 현재 분을 숫자로 반환

    # 시간 범위 제한 (05 ~ 23시)
    if current_hour < 5:
        current_hour = 5
    elif current_hour > 23:
        current_hour = 23
    hour_str = f"{current_hour:02d}"  # 두 자리 형식으로 맞추기

    headers = {
        "accept": "application/json",
        "appKey": "YOUR_TMAP_API_KEY"  # 실제 Tmap API Key 입력
    }

    # all_subway_info에서 경로별로 노선명과 역명칭을 가져와서 API 호출
    for route, stations in all_subway_info.items():
        for station_info in stations:
            route_name = station_info['line']
            station_name = station_info['station_name']

            # URL 인코딩 (노선명과 역명칭)
            encoded_route_name = urllib.parse.quote(route_name)
            encoded_station_name = urllib.parse.quote(station_name)

            # 완성된 URL
            url = f"https://apis.openapi.sk.com/transit/puzzle/subway/congestion/stat/train?routeNm={encoded_route_name}&stationNm={encoded_station_name}&dow={day_of_week}&hh={hour_str}"

            # Tmap API 호출
            try:
                response = requests.get(url, headers=headers)
                if response.status_code == 200:
                    # 응답 데이터 저장
                    congestion_data = response.json()

                    # 현재 시간에 가장 가까운 데이터를 찾기
                    if 'stat' in congestion_data['contents']:
                        closest_congestion = None
                        closest_time_diff = float('inf')

                        for stat in congestion_data['contents']['stat']:
                            for data in stat['data']:
                                hh = int(data['hh'])
                                mm = int(data['mm'])

                                time_diff = abs((current_hour * 60 + current_minute) - (hh * 60 + mm))

                                if time_diff < closest_time_diff:
                                    closest_time_diff = time_diff
                                    closest_congestion = {
                                        'station_name': station_name,
                                        'route_name': route_name,
                                        'congestion_data': data['congestionTrain']
                                    }
                        if closest_congestion:
                            congestion_results.append(closest_congestion)
            except requests.RequestException as e:
                print(f"Request failed for station: {station_name}, route: {route_name}")

    return congestion_results


# 프론트엔드로 혼잡도 데이터를 전송하는 API 엔드포인트
@app.route('/get_congestion', methods=['GET'])
def send_congestion_data():
   
    # 혼잡도 데이터 가져오기
    congestion_results = get_congestion_data(all_subway_info)
    
    # 프론트엔드로 JSON 형식으로 반환
    return jsonify(congestion_results)


if __name__ == '__main__':
    app.run(debug=True)


최종) 경로별로 구분된 결과 반환하는 기능 추가

In [ ]:
import requests
import urllib.parse
from datetime import datetime
from flask import Flask, jsonify

app = Flask(__name__)

# all_subway_info에서 경로별로 노선명과 역명칭을 가져와서 차례대로 인코딩 및 API 호출
def get_congestion_data(all_subway_info):
    congestion_results = {}
    
    # 현재 시간 반영 (요일과 시간)
    now = datetime.now()
    day_of_week = now.strftime('%a').upper()  # 요일을 MON, TUE 형식으로 반환
    current_hour = int(now.strftime('%H'))  # 현재 시간을 24시간 형식으로 반환
    current_minute = int(now.strftime('%M'))  # 현재 분을 숫자로 반환

    # 시간 범위 제한 (05 ~ 23시)
    if current_hour < 5:
        current_hour = 5
    elif current_hour > 23:
        current_hour = 23
    hour_str = f"{current_hour:02d}"  # 두 자리 형식으로 맞추기

    headers = {
        "accept": "application/json",
        "appKey": "YOUR_TMAP_API_KEY"  # 실제 Tmap API Key 입력
    }

    # all_subway_info에서 경로별로 노선명과 역명칭을 가져와서 API 호출
    for route, stations in all_subway_info.items():
        route_results = []  # 경로별로 혼잡도 데이터를 저장할 리스트

        for station_info in stations:
            route_name = station_info['line']
            station_name = station_info['station_name']

            # URL 인코딩 (노선명과 역명칭)
            encoded_route_name = urllib.parse.quote(route_name)
            encoded_station_name = urllib.parse.quote(station_name)

            # 완성된 URL
            url = f"https://apis.openapi.sk.com/transit/puzzle/subway/congestion/stat/train?routeNm={encoded_route_name}&stationNm={encoded_station_name}&dow={day_of_week}&hh={hour_str}"

            # Tmap API 호출
            try:
                response = requests.get(url, headers=headers)
                if response.status_code == 200:
                    # 응답 데이터 저장
                    congestion_data = response.json()

                    # 현재 시간에 가장 가까운 데이터를 찾기
                    if 'stat' in congestion_data['contents']:
                        closest_congestion = None
                        closest_time_diff = float('inf')

                        for stat in congestion_data['contents']['stat']:
                            for data in stat['data']:
                                hh = int(data['hh'])
                                mm = int(data['mm'])

                                time_diff = abs((current_hour * 60 + current_minute) - (hh * 60 + mm))

                                if time_diff < closest_time_diff:
                                    closest_time_diff = time_diff
                                    closest_congestion = {
                                        'station_name': station_name,
                                        'route_name': route_name,
                                        'congestion_data': data['congestionTrain']
                                    }
                        if closest_congestion:
                            route_results.append(closest_congestion)
            except requests.RequestException as e:
                print(f"Request failed for station: {station_name}, route: {route_name}")

        # 경로별 결과를 딕셔너리에 추가
        if route_results:
            congestion_results[route] = route_results

    return congestion_results


# 프론트엔드로 혼잡도 데이터를 전송하는 API 엔드포인트
@app.route('/get_congestion', methods=['GET'])
def send_congestion_data():
    # 혼잡도 데이터 가져오기
    congestion_results = get_congestion_data(all_subway_info)
    
    # 프론트엔드로 JSON 형식으로 반환
    return jsonify(congestion_results)


if __name__ == '__main__':
    app.run(debug=True)
